<a href="https://colab.research.google.com/github/EmilSeyfullayev/Work/blob/main/Strings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Run to import data

In [ ]:
# pip install pyspark
# pip install koalas

In [ ]:
# import databricks.koalas as pd
import pandas as pd
# import dask.dataframe as pd
df = pd.read_excel("/content/drive/MyDrive/OktyabrAQTİS_AEM.xlsx")

df = df.iloc[:-1, :]
df_copy = df.copy()

#Format dates to timestamp
date = pd.to_datetime(
    df['NGA tarixi'],
    format='%d/%m/%Y %H:%M'
)
df['nga_tarixi'] = date
ay_sirasi = pd.DatetimeIndex(date).month
df['ay_sirasi'] = ay_sirasi

fitosanitar = [
               "Aqrokimyəvi maddələrin dövlət sınağı, təkrar qeydiyyatı və ekspertizası",
               "Bitki mühafizə vasitələrində toksikoloji analizlər",
               "Bitki və bitkiçilik məhsullarında toksikoloji analizlər",
               "Fiziki və hüquqi şəxslərin müraciətləri əsasında fitosanitar müayinəsinin və ekspertizasının aparılması",
               "Gübrələr",
               "Pestisidlərin və bioloji preparatların dövlət sınağı, təkrar qeydiyyatı və ekspertizası"
  ]

baytarliq = [
                 "Heyvan xəstəliklərinin diaqnostikası",
                 "Yem və yem əlavələri",
                 "Yoluxucu xəstəliklərin siyahısına daxil edilmiş xüsusi təhlükəli heyvan xəstəlikləri istisna olmaqla, digər xəstəliklərin diaqnostikası (müayinəsi)",
                 "Baytarlıq preparatlarının, yem və yem əlavələrinin istifadə üçün təqdim edilən seriyalarının (partiyalarının) keyfiyyətinin müəyyən edilməsi",
                 "Nəzarət analizləri metodikalarının öyrənilməsi üzrə məsləhətlər",
                 "Baytarlıq preparatlarının, yem və yem əlavələrinin sınağı, ekspertizası, qeydə alınması",
                 "Yoluxucu xəstəliklərin siyahısına daxil edilmiş xüsusi təhlükəli heyvan xəstəlikləri istisna olmaqla, digər xəstəliklərin diaqnostikası (müayinəsi)",
                 "Mikrobioloji göstəricilər"

  ]

qida = [
                 "Bioloji Fəallığa malik Qida Əlavələri və Qida Əlavələri",
                 "Qida mikrobiologiyası",
                 "Qida məhsullarının fiziki-kimyəvi göstəriciləri",
                 "Qablaşdırma materialları",
                 "Qida məhsullarının toksikologiyası",
                 "Radiologiya",
                 "Molekulyar-biologiya (Polimeraz Zəncirvari Reaksiya)",
                 "Torpaqda toksikoloji analizlər"
  ]

# for i in df['Analiz qrupu'].unique():
#   print(i)

#Based on "Analiz qrupu" column we create Three groups

groups_advanced = []

for index, i in enumerate(df['Analiz qrupu']):

  if i in fitosanitar:
    groups_advanced.append("Fitosanitar nəzarət")

  elif i in baytarliq:
    groups_advanced.append("Baytarlıq nəzarəti")

  elif i in qida:
    groups_advanced.append("Qida təhlükəsizliyi nəzarəti")
##################################################################################################################
  elif i == "Digər":
    ############ Here we use "Laboratoriya" column in addition to "Analiz qrupu"
    ############ These labs work only with Baytarliq sphere
    ############ This info about sphere of these labs should be checked
    if df.iloc[index].loc['Laboratoriya'] in [
                                              'Mərkəzi Baytarlıq Laboratoriyası',
                                              'Bərdə Regional Sınaq Laboratoriyası',
                                              'Göygöl Regional Sınaq Laboratoriyası',
                                              'Qax Regionl Sınaq Laboratoriyası',
                                              'Quba Reqional Sınaq Laboratoriyası',
                                              'Sabirabad Regional Sınaq Laboratoriyası',


    ]:
      groups_advanced.append("Baytarlıq nəzarəti")
      ############ These labs work only with Fitosanitar sphere
    elif df.iloc[index].loc['Laboratoriya'] in [
                                                'Meyvəli Sınaq Laboratoriyası',
                                                'Mərkəzi Fitosanitar Laboratoriyası',
                                                'Xaçmaz Regional Sınaq Laboratoriyası'

    ]:
      groups_advanced.append("Fitosanitar nəzarət")

    else:
      groups_advanced.append(i)
    
############################################################################################################
  elif i == "":
    ############ Here we check not only for "Diger", but also ""
    if df.iloc[index].loc['Laboratoriya'] in [
                                              'Mərkəzi Baytarlıq Laboratoriyası',
                                              'Bərdə Regional Sınaq Laboratoriyası',
                                              'Göygöl Regional Sınaq Laboratoriyası',
                                              'Qax Regionl Sınaq Laboratoriyası',
                                              'Quba Reqional Sınaq Laboratoriyası',
                                              'Sabirabad Regional Sınaq Laboratoriyası',
                                              'GöyGöl Regional SIınaq Laboratoriyası'


    ]:
      groups_advanced.append("Baytarlıq nəzarəti")
    elif df.iloc[index].loc['Laboratoriya'] in [
                                                'Meyvəli Sınaq Laboratoriyası',
                                                'Mərkəzi Fitosanitar Laboratoriyası',
                                                'Xaçmaz Regional Sınaq Laboratoriyası'

    ]:
      groups_advanced.append("Fitosanitar nəzarət")

    else:
      groups_advanced.append(i)
  else:
    groups_advanced.append(i)

# Here, instead of nesting we simply use previous grouping
# Also we use another column "Analizin tipi"
groups_advanced_analiz_tipi = []

for index, value in enumerate(groups_advanced):

  if value not in ['', "Digər"]:
    groups_advanced_analiz_tipi.append(value)

  elif value in ['', "Digər"]:

    if df.iloc[index].loc['Analizin tipi'] == 'Phytosanitary':
      groups_advanced_analiz_tipi.append("Fitosanitar nəzarət")
    elif df.iloc[index].loc['Analizin tipi'] == 'Food':
      groups_advanced_analiz_tipi.append("Qida təhlükəsizliyi nəzarəti")
    elif df.iloc[index].loc['Analizin tipi'] == 'Veterinary':
      groups_advanced_analiz_tipi.append("Baytarlıq nəzarəti")
    else:
     groups_advanced_analiz_tipi.append(value) 

df['groups_advanced_analiz_tipi'] = groups_advanced_analiz_tipi    
# So we did use Analiz qrupu, Laboratoriya, Analiz tipi  


import numpy as np
df = df.replace(r'^\s*$', np.nan, regex=True)

analiz_neticesi = []

for i in df['Analiz nəticəsi']:

  if i is not np.nan:
    analiz_neticesi.append(i)

  elif i is np.nan:
    analiz_neticesi.append("Qeyd olunmayıb")

df['analiz_neticesi'] = analiz_neticesi
df['analiz_neticesi'].value_counts()

# df = df[~df['analiz_neticesi'].str.contains("Qeyd olunmayıb")]

# Here we use one more column which is "Mehsul" for categorization

groups_advanced_analiz_tipi_nas = []

for index, value in enumerate(df['groups_advanced_analiz_tipi']):
  if value is not np.nan:
    groups_advanced_analiz_tipi_nas.append(value)
  elif value is np.nan:
    if df.iloc[index].loc['Məhsul'] in [
                                        'Pomidor', 'Pendir', 'Biskvit', 'Mal əti', 'ÇIY SÜD',
                                        'Balık yağı' , 'Balıq unu','Donuz piyi','Dondurulmuş toyuq',
                                        'Şor','Makaron','Hazır yemək','Su','Kolbasa hisə verilmiş',
                                        'Kolbasa','Ərzaqlıq kartof', 'Ət', 'Bərk pendir','Yoqurt',
                                        'Konfet','Alkoqolsuz içki','Pasterizə edilmiş süd','Hisə verilmiş toyuq hissələri',
                                        'Qatıq','Soyudulmuş toyuq əti','Tort','Kərə yağı',
                                        'Pomidor bitkisi yarpaqları','Sirop','tort üçün krem tozu','Meyvə Şirəsi',
                                        'Günəbaxan jmıxı','Vafli','Pomidor şirəsi','Peçenye'

    ]:
      groups_advanced_analiz_tipi_nas.append("Qida təhlükəsizliyi nəzarəti")
    else:
      groups_advanced_analiz_tipi_nas.append(value)
    
df['groups_advanced_analiz_tipi_nas'] = groups_advanced_analiz_tipi_nas


#suyun kutle payi and koranovairu muayinesi we consider as qida sphere

final_groups = []

for index, value in enumerate(df['groups_advanced_analiz_tipi_nas']):

  if value != "Digər":
    final_groups.append(value)

  elif value == "Digər":

    if df.iloc[index].loc['Analizin adı'] == "Koronovirus müayinəsi":
      final_groups.append("Qida təhlükəsizliyi nəzarəti")
    
    elif df.iloc[index].loc['Analizin adı'] == "Suyun kütlə payı":
      final_groups.append("Qida təhlükəsizliyi nəzarəti")

df['final_groups'] = final_groups

numunenin_teyinati = []

for i in df['Nümunənin təyinatı']:
  if i in [
          "İxrac prosesi",
          "İxrac (prosessiz)",
          'İxrac (prosessiz) ',
          "Heyvan məhsulları ixrac"
  ]:
    numunenin_teyinati.append('İxrac')
    #####################################
  elif i in [
             'İdxal prosesi',
             'İdxal (prosessiz)'
  ]:
    numunenin_teyinati.append('İdxal')
    #####################################
  elif i in [
             'Dövlət sifarişi',
             'Regional baytarlıq idarəsi',
             'Dövlət qurumu'
  ]:
    numunenin_teyinati.append('Dövlət qurumu')
    #########################################
  elif i in [
             'Monitoring (prosessiz)',
             'Monitoring',
             'Monitorinq'
  ]:
    numunenin_teyinati.append("Monitorinq")
    #########################################
  elif i in [
             'Müqavilə üzrə fərdi müraciət',
             'Fərdi müraciət'
  ]:
    numunenin_teyinati.append("Fərdi istək")
    ##########################################
  elif i in ['Şikayət']:
    numunenin_teyinati.append('Şikayət')
    ##########################################
  elif i in [
             'Yoxlama (planlı)',
             'Yoxlama (plandan kənar)'
  ]:
    numunenin_teyinati.append('Yoxlama')
    ##########################################
  elif i in [
           'Daxili karantin sertifikatı',
           'Qiymətləndirmə',
           'Baytarlıq ekspertiza aktı',
           'Elmi araşdırma',
           'Qeydiyyat prosesi',
           'Zəhərlənmə'
  ]:
    numunenin_teyinati.append('Digər')
  else:
    numunenin_teyinati.append(i)


df['numunenin_teyinati'] = numunenin_teyinati

# "Yoluxucu xəstəliklərin siyahısına daxil edilmiş xüsusi təhlükəli heyvan xəstəlikləri istisna olmaqla, digər xəstəliklərin diaqnostikası (müayinəsi)",
# "Baytarlıq preparatlarının, yem və yem əlavələrinin istifadə üçün təqdim edilən seriyalarının (partiyalarının) keyfiyyətinin müəyyən edilməsi",
# "Nəzarət analizləri metodikalarının öyrənilməsi üzrə məsləhətlər"

numunenin_novu = []

for i in df['Analiz qrupu']:
##########################################################################
  if i in [
           'Aqrokimyəvi maddələrin dövlət sınağı, təkrar qeydiyyatı və ekspertizası',
           'Gübrələr'
  ]:
    numunenin_novu.append("Aqrokimyəvi maddələr")
##########################################################################
  elif i in [
           'Bioloji Fəallığa malik Qida Əlavələri və Qida Əlavələri'
  ]:
    numunenin_novu.append("BFMQƏ")
########################################################################## 
  elif i in [
           'Heyvan xəstəliklərinin diaqnostikası'
  ]:
    numunenin_novu.append("Heyvan xəstəliklərinin diaqnostikası")
########################################################################## 
  elif i in [
           'Mikrobioloji göstəricilər'
  ]:
    numunenin_novu.append('Mikrobioloji göstəricilər')
########################################################################## 
  elif i in [
           'Heyvan xəstəliklərinin diaqnostikası',
           "Yoluxucu xəstəliklərin siyahısına daxil edilmiş xüsusi təhlükəli heyvan xəstəlikləri istisna olmaqla, digər xəstəliklərin diaqnostikası (müayinəsi)"
           
  ]:
    numunenin_novu.append('Heyvan xəstəliklərinin diaqnostikası')
########################################################################## 
  elif i in [
           'Pestisidlərin və bioloji preparatların dövlət sınağı, təkrar qeydiyyatı və ekspertizası'
  ]:
    numunenin_novu.append('Pestesidlər')
########################################################################## 
  elif i in [
           'Fiziki və hüquqi şəxslərin müraciətləri əsasında fitosanitar müayinəsinin və ekspertizasının aparılması',
           'Qablaşdırma materialları'
  ]:
    numunenin_novu.append("Qeyri-qida məhsulları")
########################################################################## 
  elif i in [
           'Bitki mühafizə vasitələrində toksikoloji analizlər',
           'Bitki və bitkiçilik məhsullarında toksikoloji analizlər',
           'Qida mikrobiologiyası',
           'Qida məhsullarının fiziki-kimyəvi göstəriciləri',
           'Qida məhsullarının toksikologiyası',
           'Radiologiya',
           'Torpaqda toksikoloji analizlər'
  ]:
    numunenin_novu.append("Qida məhsulları")
########################################################################## 
  elif i in [
           'Yem və yem əlavələri',
  ]:
    numunenin_novu.append("Yem və yem əlavələri")
########################################################################## 
  #   elif i in [
  #          '',
  # ]:
  #   numunenin_novu.append("BFMQƏ")
  else:
      numunenin_novu.append(i)

df['numunenin_novu'] = numunenin_novu


# Məhsul

In [ ]:
df['Məhsul'].value_counts().reset_index().head(15)

,index,Məhsul
0,QAN,72751
1,Pomidor,48267
2,Alma,27495
3,Ərzaqlıq kartof,7743
4,Xurma,7513
5,Pambıq lifi,7287
6,ƏRZAQLIQ BUĞDA,7205
7,ALMA,4688
8,TƏZƏ KARTOF,4103
9,YAXMA,4076


In [ ]:
checkpoint = df

In [ ]:
df = checkpoint

In [ ]:
df['mehsul'] = df['Məhsul']
df['mehsul'] = df['mehsul'].apply(lambda x: str(x).lower())

In [ ]:
def see_similar(string):
  similar = df[
    (pd.notna(df['mehsul'])) &
    (df['mehsul'].str.contains(string))
  ]['mehsul'].unique()

  len_similar = len(similar)
  return len_similar, similar

In [ ]:
see_similar("balıg") #flakon, tablet, kapsul, ml, gr ətli

(3, array(['soyudulmuş beluqa balıgı',
        'soyudulmuş  sazan, belamor, xanı balıgı',
        'soyudulmuş forel balıgı'], dtype=object))

### Baliqlar

In [ ]:
# balıg, balığ, balig, baliğ, balig and with q, and bal
balıq_baliq_mehsullari_ve_baliqlara_aid_numuneler = [
                                                     'skumbriyanın filesi tomat sousunda',
                                                     'kilkə baltik qizardilmiş tomatda 240qr',
                                                     'balıq (çapaq)',
'forel (alabaliq)',                                                     
'soyudulmuş si̇bri̇m baliği', 'bölgə  baliği soyudulmuş',
        'si̇yənək  baliği dondurulmuş', 'karp  baliği dondurulmuş',
        'beluga  baliği soyudulmuş', 'xəşəm baliği dondurulmuş',
        'sazan  baliği soyudulmuş', 'çapaq  baliği dondurulmuş',
        'skumbri̇ya baliği  dondurulmuş', 'farel  baliği soyudulmuş',
        'sazan  baliği dondurulmuş', 'sudak  baliği dondurulmuş',
        'dondurulmuş qizil baliğin onurğa hissələri',
        'dondurulmuş qizil baliğin hissələri', 'tuna baliğinin fi̇lesi̇',
        'soyudulmuş si̇bas baliği', 'nərə  baliği dondurulmuş',
        'qizil baliğinlin tikələri', 'belamur baliği  soyudulmuş',
        'xani̇ baliği  soyudulmuş', 'i̇lan  baliği dondurulmuş',
        'tuna baliği  dondurulmuş',
        'dondurulmuş qizil baliğin qarin hissələri',
        'dondurulmuş panqasius baliği', 'si̇yənək  baliği soyudulmuş',
        'uzunburun  baliği soyudulmuş', 'nərə  baliği soyudulmuş',
        'ton baliğin filesi', 'si̇yənək baliği konservi',
        'ki̇lkə baliği konservi', 'skumbri̇ya baliği konservi',
        'sardi̇na baliği konservi',
        'konservləşdi̇ri̇lmi̇ş sardi̇na baliği', 'soyudulmuş farel baliği',
        'belamur baliği soyudulmuş', 'xani baliği soyudulmuş',
        'sazan baliği soyudulmuş', 'dondurulmuş skumbriya baliği',
        'dondurulmuş xek baliğı', 'soyudulmuş forel baliği',
        'tunes baliği konservi', 'şprotı baliği konservi',
        'soyudulmuş sazan baliği', 'soyudulmuş amur baliği',
        'kütüm baliğı  soyudulmuş', 'sazan baliği  soyudulmuş',
        'kütüm baliği  soyudulmuş', 'belamor baliğı  soyudulmuş',
        'xanı baliği  soyudulmuş', 'soyudulmuş kütüm baliği',
        'atlantik okeani qizilbaliği (salmo salar) və dunay qizilbaliği',
        'çəki̇ baliği  soyudulmuş', 'dondurulmuş çapaq baliği',
        'saki̇t oken hake baliği (merlucci̇us spp)',
        'saki̇t oken hake baliği (merlucci̇us spp',
        'dondurulmuş saki̇t oken hake baliği (merlucci̇us spp)',
        'dondurulmuş saki̇t oken hake baliği (merlucci̇us spp',
        'taze sog. çipura butun (400-600gr) - təzə soyudulmuş çupra baliği bütün (400-600 qr)',
        'taze sog. levrek butun (400-600gr) - təzə soyudulmuş levrek baliği bütün (400-600qr)',
        'kefal baliği dondurulmuş', 'adi̇ sif(sudak) baliği dondurulmuş',
        'dondurulmuş siyənək baliği', 'bülgə baliğı  soyudulmuş',
        'dondurulmuş forel baliği  200-300',
        'dondurulmuş çapaq (leş) baliği', 'soyudulmuş belamur baliği',
        'ton baliğinin parçalari duzlu suda', 'sazan baliği dondurulmuş',
        'sakit okean qizilbaliği', 'konservləşdi̇ri̇lmi̇ş  ton baliği',
        'dondurulmuş sazan baliği',
        'skumbriya baliği başsız dondurulmuş 200-300 qr',
        'dondurulmuş syemqa baliği, syemqa baliğinin fi̇lesi̇, ti̇kələnmi̇ş syemqa baliği',
        'dondurulmuş ki̇juç baliği', 'unaqi̇ baliği',


'konservleşdirilmiş ton baligi', 'soyudulmuş sea bream baligi',
        'soyudulmuş sea bass baligi 400-600', 'soyudulmus belamor baligi',                                                     

'syomqa balığın filesi', 'konservləşdirilmiş ton balığı',
        'dondurulmuş çapaq balığı', 'soyudulmuş forel(salmon) balığı',
        'soyudulmuş beluqa & siberi uzunburun  balığı',
        'soyudulmuş sazan balığı', 'dondurulmuş siyənək balığı',
        'salmon balığının dondurulmuş onurğa hissəsi',
        'dondurulmuş sazan balığı', 'soyudulmuş çəki balığı',
        'soyudulmuş belamur balığı', 'dondurulmuş xəşəm balığı',
        'dondurulmuş skumbriya balığı', 'çəki balığı soyudulmuş',
        'kütüm balığı soyudulmuş', 'soyudulmuş kütüm balığı',
        'soyudulmuş çəki balığlı', 'dondurulmuş xəşəm (jerex) balığı',
        'soyudulmuş xanı balığı', 'soyudulmuş ağ amur balığı',
        'dondurulmuş yayın balığı', 'dondurulmuş cənub putassu balığı',
        'salmon balığının dondurulmuş qarınaltı hissesi',
        'dondurulmuş siyənək (seld) balığı', 'dondurulmuş  kefal balığı',
        'siyənək balığı yağda duzlu "atlantik seld"  1300qr',
        'forel balığının filesi "hisə/verilmiş"  300qr',
        'syomqa balığının filesi "hisə/verilmiş"  200qr',
        'qorbuşa balığının tikələri "soyuq/hisə/verilmiş" 230qr',
        'qızılbalığın kremi krevetka ilə', 'dondurulmuş salmon balığı',
        'qurudulmuş duzlu tuneç balığı', 'xek balığı',
        'dondurulmuş xek balığı',
        'qizil balığın kürüsü qorbuşa "original" 210qr',
        'liza (kefal) balığı(лат. liza haematocheilus)',
        'seld siyənək balığının file tikələri şüyüd ilə  200qr',
        'dondurulmuş salmon  (syomqa)   balığı', 'soyudulmuş  karp balığı',                                                     

'soyudulmuş beluqa balıgı',
        'soyudulmuş  sazan, belamor, xanı balıgı',
        'soyudulmuş forel balıgı',


'soyuq hisə verilmiş qızılbalığın filesinin  dilimləri vakuumda  100 q',
'dondurulmuş baliqlar','balık yağı','goody balikli (fi̇sh) pi̇şi̇k yemi̇',
'”purina one” (yüksək tərkibli qizilbaliq ətli və buğda dənli, sterilizasiya olunmuş pişiklər üçün)',
'“gourmet perle” sousda yumuşaq file (qizilbaliq ilə)',
'baliq (panqasus)', 'baliq', 'pro plan qizilbaliq ilə',
'kitekat 85 q dovşan,mal, baliq, quzu və toyuq əti ilə (sousda)',
'təzə soyudulmuş baliqlar', 'baliq (tikələri)',
'vitamama qizil baliq kapsul 75 q', 'baliq konservi̇',
'kitekat baliqçinin ovu, iştahaçan dana əti, ət bayrami və iştahaçan toyuq əti çeşidli yetkin pişiklər üçün yem',
'ocean baliq yaği portağal dadi i̇lə, daxilə qəbul üçün məhlul 150 ml',
'asotri̇n baliq',
'“whiskas” iştahaçici qarişiq. (i̇ldən 1+). qizil baliq və krevet ilə qaymaqli sousda yetkin pişiklər üçün tam rasionlu konservləşdirilmiş yem-pişiklər üçün',
'baliq yaği',
'whiskas qizilbaliqli və krivetli qaymaqli sous iştahaçan qarişiq',
'whiskas qizilbaliqli, mal və quzu, mal və dovşan, toyuq, toyuq və hinduşka ətli jele',
'whiskas toyuq və hinduşka ətli, qizilbaliqli iştahaçan',
'whiskas alabaliq, qizilbaliq, toyuq, dana, quzu, mal, dovşan-hinduşka ətli raqu (1 yaşdan başlayaraq)-yetkin pişiklər üçün',
'nutrigen growmega baliq yaği, şərbət 200 ml',
'konservləşdiril-miş baliq məhsullari',
'treska baliq ciyəri 120qr', 'baliq farşi',
'qurudulmus duzlanmis baliq 18 qr', 'alabaliq baliği soyudulmuş',
'konservləşdirilmiş baliq', 'təzə soyudulmuş alabaliq',
'dondurulmuş çapaq (leş baliqı)',
'fetofak(xəşəmbaş) soyudulmuş təzə baliq', 'quru baliq 36 qr',
'dondurulmuş qizil baliq', 'dondurulmus forel baliqi',
'qizilbaliq (qorbuşa) kürüsü (qirmizi kürü)',
'baliq məhsulu-pasta antarktik kril "dəniz məhsullarından qaymaq sarımsaqlı"  150qr',
'baliq konvervasi 24*180gr',
'taze sog. ci̇pura butun (400-600 gr)- təzə soyudulmuş çupra bütöv baliq (400-600gr)',
'alabaliq baliq  soyudulmuş', 'baliq konservləri',
        'baliq konservləşdi̇ri̇lmi̇ş', 'forel (alabaliq)'
'balıq (çapaq)', 'balıq (salmon)', 'balıq (qızılbalıq)',
'balıq konservi', 'balıq (çipura)', 'balıq (levrek)',
'balıq (xanı)', 'balıq (forel)', 'balıq (skumbriya)', 'balıq yemi',
'soyuq hisə verilmiş balıq', 'balıq preservi', 'balıq kürüsü',
'balıq unu', 'dondurulmuş balıq', 'balıq filesi',
'dondurulmuş hisə verilmiş balıq filesi', 'balıq (sazan)',
'balıq (belamur)', 'balıq (xek)', 'marinada qoyulmuş balıq',
'balıq (sudak)', 'yarımfabrikat balıq tikələri', 'balıq (beluqa)',
'soyudulmuş balıq', 'balıq (hamsi)', 'təzə balıq',
'qaxac edilmiş balıq', 'balıq (kütüm)', 'balıq (çəki)', 'balıq',
'balıq (karp)',
'konservləşdirilmiş balıqlar file skumbriya yağda,tomat sousunda,skumbriya tomat sousunda, yağda',
'balıq merluza-merluzzi decapitati          kg.1,00x06l',
'costa lachsfilet nişanlı balıq filesi',
'konservləşdirilmiş balıq', 'konservlesdirilmis balıq',
'balıq filesi qurudulmuş və qizardilmiş(dondurulmuş) 1/5000q,"fi̇sh day"1/350q,"prioriti" 1/300q',
'kapour(sazan) soyudulmuş təzə balıq', 'qızıl balıq filesi',
'dondurulmuş alabalıq', 'balıq  konservləşdirilmiş',
'forel(alabalıq) təzə', 'balıq (putassu)',
'ruegenf kieler sprotten nişanlı balıq konservi',
'balıq konservi(kapitan vkusov)',
'taze sog. levrek butun (400-600 gr)- levrek təzə soyudulmuş bütöv balıq (400-600gr)',
'balıq konservləri']

balıq_baliq_mehsullari_ve_baliqlara_aid_numuneler = list(set(balıq_baliq_mehsullari_ve_baliqlara_aid_numuneler))

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Balıq, balıq məhsulları və balıqlara aid nümunələr" if x in balıq_baliq_mehsullari_ve_baliqlara_aid_numuneler else x)

In [ ]:
df[
    (pd.notna(df['mehsul'])) &
    (df['mehsul'].str.contains("dərİ".lower())) #&
   #~(df['mehsul'].str.contains("ş"))
  ]['mehsul'].unique()

array(['hi̇nd quşu si̇nəsi̇ni̇n dəri̇li̇ tri̇mi̇ngi̇',
       'hi̇nd quşu ayağinin dəri̇si̇z tri̇mi̇ngi̇',
       'dondurulmuş toyuq dəri̇si̇ məhsulu',
       'dondurulmuş toyuq əti̇ məhsullari- dəri̇',
       'dondurulmuş toyuq əti məhsullari - dəri̇',
       'dondurulmuş toyuq dəri̇si̇'], dtype=object)

In [ ]:
# df[
#     (pd.notna(df['mehsul'])) &
#    (df['mehsul'].str.contains("şərbət")==False) &
#    (df['mehsul'].str.contains("yemlər")==False) &
#    (df['mehsul'].str.contains("diliml")==False) &
#    (df['mehsul'].str.contains("toxumlar")==False) &
#     (df['mehsul'].str.contains("ml")) #|
#   #   (df['mehsul'].str.contains("məhlul")) |
#   #  (df['mehsul'].str.contains("damcı")) |
#   #  (df['mehsul'].str.contains("sprey")) |
#   #  (df['mehsul'].str.contains("qəbul üçün")) |
#   #  (df['mehsul'].str.contains("damci")) |
#   #  (df['mehsul'].str.contains("damci")) 
#   ]['mehsul'].unique()



### Dermanlar

In [ ]:
# df[
#     (pd.notna(df['mehsul'])) &
#    (df['mehsul'].str.contains("şərbət")==False) &
#    (df['mehsul'].str.contains("yemlər")==False) &
#    (df['mehsul'].str.contains("diliml")==False) &
#    (df['mehsul'].str.contains("toxumlar")==False) &
#     (df['mehsul'].str.contains("ml")) #|
#   #   (df['mehsul'].str.contains("məhlul")) |
#   #  (df['mehsul'].str.contains("damcı")) |
#   #  (df['mehsul'].str.contains("sprey")) |
#   #  (df['mehsul'].str.contains("qəbul üçün")) |
#   #  (df['mehsul'].str.contains("damci")) |
#   #  (df['mehsul'].str.contains("damci")) 
#   ]['mehsul'].unique()


In [ ]:
see_similar('toz')

(158, array(['venatura i̇nulin ve fruktooligosakkarit (fos), toz, saşedə №30',
        'venatura sam-e, toz, saşedə №30',
        'venatura kids vitamin c, toz / saşe №30',
        'mega mass 4000 (chocolate flavour), toz 3 kq',
        'premium whey (chocolate-nougat), toz 500 q',
        'premium whey (chocolate-nougat), toz 2.3 kq',
        'gold whey (milk chocolate flavour), toz 500 q',
        'gold whey (milk chocolate flavour), toz 2 kq', 'kakao tozu',
        'laktoz 100ml şərbət', 'ferti̇na,toz(saşedə) №30', 'frappe tozu',
        'şəkər tozu', 'velanil toz 4q saşe №40',
        'bulardi junior toz 2q, saşe №10',
        'whey protein complex 100% chocolate flavour, toz 2270qr',
        'whey protein complex 100% chocolate flavour, toz 700 q',
        'gai̇n boli̇c 6000, toz (paketdə) 1000 q',
        'aspermi̇n toz, saşe n30', 'gain bolic 6000, toz, 6800qr',
        'bcaa xplode powder (xplosive cola flavour), toz, 500qr',
        'bcaa xplode powder (xplosive cola flavour),

In [ ]:
tablet_flakon_kapsul_mehlul_damci_dermanlar = [ # vitaminler, preparatlar, serbetler fromasinda toz

'biogren +, toz, saşedə n20', 'biovac reo', 'biovac nd-i̇b',
        'biocillin-500 ws', 'biorest, toz-saşe №10',
        'rovabio advance p 50', 'biomax g2', 'bio-bitpest g2',
        'rehidron bio toz , saşe №10', 'biocan r inj', 'biobos ibr',
        'biocan dhppi̇+l', 'biobos respi 4', 'biocan dhppi̇ inj',
        'biofel pch inj', 'biospekta, toz saşedə №40', 'biovit olkar 80',
        'bio-vac la sota', 'biopirox', 'biovit formaton 150q',
        'biostim universal', 'canvit biocal plus for dogs',
        'rehidron bio, toz saşe №10', 'biocal', 'biovit-olkar-80',
        'simbiomax №10 5q saşe', 'probiocsin kids ,toz(saşede) №15',
        'rehidron bio toz saşe №10', 'isabion: 4800 litr',
        'biosillin 200 ws', 'biohumus', 'biototal', 'biosoil', 'bioard 20',
        'esbioful', 'nbl probiotic gold, saşe №20',
        'nbl probiotic atp, saşe №20', 'bioplane', 'seleda bio gas',
        'seleda bio sek 3-1-1', 'esobiotik n15 saşe',
        'probiomax, toz (saşedə) n20', 'biotracius cab',

        'rehidron bio toz,saşe  no 10', 'biobac wp', 'biostim',
 'biotracusius cab',
        'biotracius k',

'colidaza, şərbət 150 ml', 'largovit kids şərbət 120 ml',
        'grengam, şərbət 150 ml', 'venatura sambu-c, şərbət 150 ml',
        'venatura beta-qlükan, propolis, exinaseya, c vitamini və sink, şərbət 150 ml',
        'peksan, şərbət 150 ml', 'kri̇lom şərbət 150 ml',
        'bone baby, şərbət, 150 ml', 'urotonic 200ml şərbət',
        'calcelux şərbət 150ml', 'laktoz 100ml şərbət',
        'n-as 150ml şərbət', 'nikovit c şərbət 100ml',
        'ci̇nkofer, şərbət 150 ml', 'aci̇dekzol şərbət 200ml',
        'zincomega şərbət №20', 'nanopar, şərbət 150 ml',
        'krimberq,şərbət 150 ml', 'radibest,şərbət 100 ml',
        'blokalin,şərbət, 150 ml', 'dr. asos, şərbət 150 ml',
        'furiya , şərbət 150 ml', 'epifos , şərbət 150 ml',
        'i̇mukan 100 ml şərbət', 't-mix, şərbət 150 ml',
        'zekomax şərbət 100ml', 'betastim, şərbət 150 ml',
        'novalopin kids ,şərbət 150 ml',

        'rina baby/rina beybi, şərbət 150 ml',
        'bilberyx/bilberiks, şərbət 150 ml', 'polmegold, şərbət 150 ml',
        'avakid, şərbət 150 ml', 'venatura kids sink, şərbət 100 ml',
        'vi̇tagood şərbət 150 ml', 'sibulax, şərbət 150 ml',
        'calcivit m/ kalsivit m şərbət 150 ml', 'bio-h, şərbət 100 ml',
        'anti simbioz, şərbət 100ml', 'tussor, şərbət 100ml',
        'ufron, şərbət 150ml', 'fosfobut, şərbət 150 ml',
        'ofta mix şərbət 100 ml', 'cartisin, şərbət 150 ml',
        'asbon, şərbət 150 ml', 'gamino baby, şərbət 150ml',
        'i̇muvir, şərbət 150ml', 'i̇namiton, şərbət 150ml',
        'i̇nameston, şərbət 150ml', 'elsaton, şərbət 100 ml',
        'benelux, şərbət 100 ml', 'muc-uno şərbət 150 ml',
        'venatura kids maqnezium, şərbət, 100 ml',
        'nemokal, şərbət 150 ml', 'grenovit, şərbət 150 ml',
        'qamasin vitamin c 100 ml şərbət', 'vita-lizin, şərbət 200 ml',
        'fumax ,şərbət 150 ml', 'pi̇nstone şərbət 200ml',
        'delarest şərbət 200ml', 'megalan şərbət 100 ml',
        'sedikum şərbət 150 ml', 'nyam-nyam kids şərbət 200 ml',
        'pçelavit şərbət 190 ml', 'karel şərbət 200ml',
        'bi̇en şərbət 200ml', 'verfen şərbət 200ml',

        'apoksi-l şərbət 150 ml', 'anfolak şərbət 150 ml',
        'slim fort, şərbət 200 ml', 'versa, şərbət 200 ml',
        'vetodan s ,şərbət 150 ml', 'parametabol, şərbət 150 ml',
        'osteokal-d kids , şərbət 150 ml', 'holicin şərbət 90 ml',
        'citelat şərbət 90 ml', 'vitamama i̇mmunotops, şərbət 95ml.',
        'cereox, şərbət 100ml', 'kargi plus, şərbət, 150 ml',
        'tridekal/ tridecal , şərbət 150 ml', 'ferapin şərbət 200 ml',
        'detenzim, şərbət 150 ml', 'nutrigen supra, şərbət 200 ml',
        "möller's omega-3 alma aromali, şərbət 250 ml",
        'megaller, şərbət 100 ml', 'proline 200 ml şərbət',
        'detoxsfer, şərbət 150 ml', 'rolly multivitamin şərbət 150 ml',
        'oksimega, şərbət 150 ml', 'detokid, şərbət 120 ml',
        'sefalin, şərbət 120 ml', 'calsiton, şərbət 150 ml',
        'sonefro, şərbət 120 ml', 'omejuni̇or şərbət 150 ml',
        'day moon, şərbət 150 ml', 'di̇ktum, şərbət 150 ml',
        'ostin, şərbət 150 ml', 'sedifactor şərbət 100ml',
        'i̇mmunolife şərbət 100 ml', 'reviforce, şərbət 250 ml',
        'novasan kids,şərbət 150 ml', 'kardiotonus 250ml şərbət',
        'desol omega 3 ,şərbət 150 ml', 'desol multivit ,şərbət 150 ml',
        'bi̇omunal 120 ml şərbət', 'bonestone , şərbət 150 ml',
        'alsokar 150 ml şərbət', 'vi̇kofer 150 ml şərbət',
        'demorin şərbət 100 ml', 'friska ben 150 ml şərbət',

        'grendecal, şərbət 150 ml',
        'venatura premium kids omega 3 tutti frutti dadli, şərbət 150 ml',
        'miroksera, şərbət, 100 ml', 'daunason 150 ml şərbət',
        'vekarmol, şərbət 150 ml', 'gammasi̇ron 150 ml şərbət',
        'aso-h 150 ml şərbət', 'realizin, şərbət 100 ml',
        'nevroxol, şərbət 100 ml', 'neyrobut, şərbət 120 ml',
        'munitas, şərbət 150 ml', '3 metabol 150 ml şərbət',
        'meri̇car 150 ml şərbət', 'sentocal calcium 150 ml şərbət',
        'asolin oksi, şərbət 150 ml', 'prokal d, şərbət 200 ml',
        'omegapin şərbət 190 ml', 'portaze plus, şərbət 120 ml',
        'odezim, şərbət 150 ml', 'evitar, şərbət 120 ml',
        'nanolakt, şərbət 120 ml', 'helmizin-p, 100 ml şərbət n1',
        'fosforash 150 ml şərbət', 'i̇mmuno-generator, şərbət, 100 ml',
        'novarest kids ,şərbət 150 ml', 'apitol, şərbət, 125 ml',
        'elfatol, şərbət 100 ml', 'zeluvit kids şərbət 150 ml',
        'nomerin forte ,şərbət 120 ml', 'sterovit  ,şərbət 150 ml',
        'legam 150 ml şərbət №1', 'lungast 150 ml şərbət',
        'delong 120 ml şərbət', 'acetan 120 ml şərbət',
        'rembol 120 ml şərbət', 'ronosil, şərbət, 250 ml',
        'qalitolin, şərbət, 250 ml', 'pnevmanol, şərbət, 250 ml',
        'nefromax, şərbət 100 ml', 'aminogamma, şərbət 100 ml',
        'breynavit, şərbət 150 ml', 'vitamemor kids 200 ml şərbət',
        'lespiforte 120 ml şərbət', 'demikar 120 ml şərbət',
        'vitatrop 150 ml şərbət', 'sambovin kids,150 ml şərbət№1',
        'altapin şərbət 200 ml', 'altekid, şərbət 120 ml',
        'seriton forte, şərbət 300 ml', 'apimol, şərbət 100 ml',
        'florita şərbət 150 ml', 'renepas, şərbət 200 ml',
        'omegito şərbət 200 ml', 'yasno oko, şərbət 100 ml',
        'g-mix şərbət 100ml', 'altekid şərbət 120 ml',
        'mari̇fi̇sh ,şərbət 150 ml', 'fi̇ti̇um ki̇ds ,şərbət 150 ml',
        'vi̇tali̇n forte ,şərbət 150 ml', 'vi̇mari̇n ki̇ds,şərbət 150 ml',

        'vi̇tmi̇x multi̇vi̇tami̇n ,şərbət 100 ml',
        'holi̇ci̇n şərbət 90 ml', 'pantotin şərbət 100 ml',
        'fumax ,şərbət ,150 ml', 'osteohard ki̇ds,şərbət 150 ml',
        'sambovin kids,150ml şərbət №1', 'calci̇gol 125ml şərbət',
        'altekid,şərbət 120 ml', 'megaglukankids şərbət 150 ml',
        'nanospan şərbət 150 ml', 'bi̇fonk  şərbət 100 ml',
        'ostimol, şərbət 150 ml', 'nanopar 150 ml şərbət',
        'fobrain, şərbət 150 ml', 'olmega h-33 125 ml şərbət',
        'omega 3 multi baby 150 ml şərbət', 'ki̇ti̇for , şərbət 120 ml',
        'li̇tased ki̇ds , şərbət 120 ml', 'restomi̇x , şərbət 120 ml',
        'i̇nfason , şərbət 120 ml', 'caldep, şərbət 150 ml',
        'askopal , şərbət 150 ml', 'alfased , şərbət 120 ml',
        'neurosleep , 120 ml şərbət № 1', 'b-vital baby ,şərbət 150 ml',
        'strepsore şərbət 100 ml', 'paros k , şərbət 150 ml',
        'nişasta şərbəti', 'bonacal d3, şərbət150 ml',
        'normavit 200 ml şərbət', 'aceffect/aseffekt, şərbət 150 ml',
        'vi̇tababy şərbət 150 ml', 'mersolakt şərbət 200ml',
        'krimberq 150 ml şərbət', 'blokalin 150 ml şərbət',
        'radibest 100 ml şərbət', 'romans jele şərbət 10 ml №10',
        'brocsevi̇n,şərbət 150 ml', 'cholika, şərbət 150 ml',
        'bone baby, şərbət 150 ml', 've-bixol, şərbət, 100 ml',
        'galax şərbət 120 ml', 'urotonic şərbət 200ml',
        'montevi̇t 150 ml şərbət', 'reni̇di̇ol, şərbət 300 ml',
        'reni̇con, şərbət 300 ml', 'şərbət(aqav,quds enginarı,ağcaqayin)',

'vetimisin rtu 50 ml',
       'mamacare şenkov və kormyaşix it (u208), juniorcomlex şenkov üçün (u207) unitabs multikompleks cto 55025582-0006-2015',
       'bitkisəl qarişimli məcun', 'tilozin 200 100 ml',
       'neozidin m; 20 ml', 'mastomisin 10 ml', 'tilozin 50 50 ml',
       'klozatrem 250 ml', 'nitoks 200 50 ml', 'nitoks 200 100 ml',
       'mastisan 100 ml', 'tilozin 50 20 ml', 'ampronet 2500 ml',
       'yemli̇k maya', 'guarana shots 1.800 orange flavour 25 ml n20',
       'yemlik qarğidali', 'forever aloe vera gel, gel 1000ml',
       'benzilpenisilin 1mln',
       'herbisidlər, bitkilərin cücərməsinə qarşı vasitələr və bitkilərin boy artımının tənzimləyiciləri:',
       'i̇vermi̇s –fac 100 ml', 'oximic 20% la x 100 ml',
       'zuletel 10% iny x 50 ml', 'estreptopenicilina x 50 ml',
       'ivermic + ad3e x 100 ml', 'estreptopenicilina x 250 ml',
       'oximic 20% la x 50 ml', 'calciomic x 100 ml',
       'zuletel 10% iny x 250 ml', 'oximic 20% la x 250 ml',
       'calciomic x 250 ml', 'estreptopenicilina x 100 ml',
       'zi̇mec-fc injectable solution 250ml', 'tilmicure 250ml',
       'fitolkas konsentrat mehlul 100 ml',
       'blotsi̇m oral emulsion 100ml',
       'amoxyni̇l la injectable suspension 250 ml',
       'cefti̇si̇n injectable suspension 250 ml',
       'bupartek injectable solution 20ml',
       'calject 50 injectable solution 100ml',
       'amoxyni̇l la injectable suspension 100ml',
       'enrocure %10 oral solution 100ml',
       'amoxygen injectable suspension 100ml',
       'amoxygen injectable suspension 250 ml',
       'calject 50 injectable solution 250ml',
       'cefti̇si̇n injectable suspension 50ml',
       'cefti̇si̇n injectable suspension 100ml', 'ketosi̇l 250 ml',
       'enroci̇d %20- 50 ml-630', 'tylonet %30 - 100 ml',
       'si̇prasol 50 ml', 'netflor 100 ml', 'netflor 250 ml',
       'enroci̇d %20- 100 ml', 'ketosi̇l 50 ml', 'ketosi̇l 100 ml',
       'floral %30 - 100 ml', 'si̇prasol - 250 ml', 'si̇prasol 100 ml',
       'netflor 50 ml', 'santaket enjeksiyonluk çözelti 50 ml',
       'mastikloks 10 ml', 'piroteks 7%+b12 100ml', 'nimline', 'cemlər',
       'kremlin emtee  nishanli arag', 'şokolad kremləri 400 qr',
       'quick sleep-sueno rapid/kvik slip-sueno rapid,mehlul 10 ml',
       'nar eşkisi sous 250 ml', 'bitkisəl qarışımlı məcun',
       'süd 200 ml 1x27', 'akmi̇na li̇mon + c 200 ml 6*4',
       'akmina qirmizi meyvələr 200 ml 6*4',
       'akmina mandalin + c 200 ml 6*4', 'akmina limon 200 ml 6*4',
       'alteya altay çaytikanı yağı 100ml',
       'alteya altay küncüt yağı 100ml',
       'alteya altay balqabaq yağı 100ml', 'alteya altay qoz yağı 100ml',
       'alteya altay kətan yağı 100ml',
       'alteya altay alaqanqal yağı 100ml', 'caldesun mehlul 200 ml',
       '- magnesium + b6 shots orange ampulda 25ml n20',
       'guarana 1.800 shots orange 25 ml n20', 'qara zi̇rə yaği 250ml',
       'krem i̇xracat 200 ml',
#damci, mehlul, qebul üçün
'delife (delayf), məhlul 30 ml',
       'donsir advance, daxilə qəbul üçün suspenziya 250ml',
       'cystoliberin advance, daxilə qəbul üçün suspensiya 125 ml',
       'treataprost, daxilə qəbul üçün suspenziya,250 ml',
       'disurinorm, daxilə qəbul üçün suspenziya 250 ml',
       'comfovita advance, daxilə qəbul üçün suspenziya 250 ml',
       'devive, daxilə qəbul üçün məhlul-damcı, 15 ml',
       'fondomix, daxilə qəbul üçün məhlul, 15 ml №10',
       'maribiol kids daxilə qəbul üçün məhlul 240ml',
       'amino power liquid (cola flavour), məhlul 1000 ml',
       'lebemax,daxilə qəbul üçün məhlul damci. 50 ml',
       'neym, daxilə qəbul üçün məhlul 150 ml',
       'devi̇ta, məhlul-damci 30ml', 'neo-entelakt, məhlul 200 ml',
       'protec 20, daxilə qəbul üçün məhlul 200ml',
       'zeytun yarpaği ekstrakti, daxilə qəbul ücün məhlul,30ml',
       'lentavi̇t forte d3, daxi̇lə qəbul üçün məhlul-damci, 15 ml',
       'smart-d, daxilə qəbul üçün məhlul-damci, 30 ml',
       'composor 24 uva-ursi complex, daxilə qəbul üçün məhlul, 50 ml',
       'maxi d3 daxilə qəbul üçün məhlul-damci 30 ml',
       'vitamin c daxilə qəbul üçün məhlul 5 ml №10',
       'vitamin c daxilə qəbul üçün məhlul 2 ml №10',
       'aktiv flora beybi / acive flora baby , oral damci 5ml',
       'funtin daxilə qəbul üçün məhlul-damci 30 ml',
       'vitamin c,daxilə qəbul üçün məhlul 5 ml №10',
       'fulor, boğaz üçün sprey 30 ml',
       'care-h, daxilə qəbul üçün məhlul-damci 30 ml',
       'botamin, daxilə qəbul üçün məhlul-damci, 30 ml',
       'mielotrof , 10 ml daxilə qəbul üçün məhlul №10',
       'l-eliksir / l-elixir, daxilə qəbul üçün məhlul-damcı 30 ml',
       'çistotel maksimum damci it üçün',
       'çistotel maksimum damci pişik üçün',
       'ocean picozinc, daxilə qəbul üçün məhlul-damci 30 ml',
       'epam 7,daxilə qəbul üçün məhlul 30ml',
       'epam 44, daxilə qəbul üçün məhlul 30 ml',
       'esokart, 10ml daxilə qəbul üçün məhlul №10',
       'factor d3, daxilə qəbul üçün məhlul-damci 15 ml',
       'bars sprey it üçün', 'bars sprey pişik üçün',
       'composor 05 valerian complex, daxilə qəbul üçün məhlul 50 ml',
       'melastim ,daxilə qəbul üçün məhlul , 50 ml',
       'ofio, daxilə qəbul üçün məhlul-damci 30 ml',
       'epam 4, daxilə qəbul üçün məhlul 30ml',
       'nevron, daxilə qəbul üçün məhlul 30 ml',
       'dream drops daxilə qəbul üçün məhlul damci 30 ml',
       'ocean microfer, daxilə qəbul üçün məhlul-damci 30 ml',
       'amino forte plus, məhlul 10 ml №10',
       'degold, daxilə qəbul üçün məhlul-damci 30 ml',
       'depronax d3, məhlul-damci 15 ml',
       'simela ,daxilə qəbul üçün məhlul 50 ml',
       'neocar baby daxilə qəbul üçün məhlul-damci 50 ml',
       'alfadol məhlul-damci 30 ml',
       'flegamix daxilə qəbul üçün məhlul 150 ml',
       'asebronx məhlul 200 ml',
       'mega a&d 50 ml daxilə qəbul üçün məhlul-damci',
       'magnesium + b6 shots orange , 25ml daxilə qəbul üçün məhlul №20',
       'mohsinal, daxilə qəbul üçün məhlul-damci 35 ml',
       'slogren daxilə qəbul ucun məhlul-damcı, 10 ml',
       'grenkid, daxilə qəbul üçün məhlul-damçi, 30 ml',
       'calcivit m məhlul 150 ml', 'tenat, məhlul(damci) 30 ml',
       'nutri̇comp d məhlul 500 ml',
       'nutri̇comp dri̇nk plus chocolate məhlul 200 ml',
       'nutri̇comp dri̇nk plus strawberryl məhlul 200 ml',
       'forever freedom (orange flavored aloe vera juise)/ forever azadliq şirəsi(portağal ətirli aloe vera şirəsi), məhlul 1000ml',
       'nuravit d3 daxilə qəbul üçün məhlul 15 ml',
       'memodrop b6 daxi̇lə qəbul üçün məhlul-damci 50ml',
       'eskavit daxilə qəbul üçün məhlul-damci 15 ml',
       'bioarginina orale vials / bioarginin daxilə qəbul üçün məhlul, məhlul №20',
       'neyrostar daxilə qəbul üçün məhlul-damci 50 ml',
       'lugn, daxilə qəbul üçün məhlul-damci 30 ml',
       'reginat duo, daxilə qəbul üçün suspenziya 200 ml',
       'notser, məhlul, saşe №20', 'neyromart, məhlul 50 ml',
       'oroliv ,daxilə qəbul üçün məhlul-damci,50ml',
       'flushedix 200 ml məhlul/2 qr toz saşedə',
       'sultepri̇m oral məhlul', 'li̇nkovi̇k ineksiya məhlulu',
       'akti̇vi̇ton i̇nyeksi̇ya üçün məhlul',
       'enroflon 5% i̇nyeksi̇ya üçün məhlul',
       'afden daxilə qəbul üçün məhlul-damci 15 ml',
       'simde daxilə qəbul üçün məhlul 30 ml',
       'vitaber plus daxilə qəbul üçün məhlul 150 ml',
       'qipokar daxilə qəbul üçün məhlul, 100 ml',
       'd-large daxi̇lə qəbul üçün məhlul 30ml',
       'denkos, daxi̇lə qəbul üçün məhlul-damci 50 ml',
       'demi̇ca, məhlul 30 ml', 'caputan məhlul 100 ml',
       'b-vit , daxilə qəbul üçün məhlul - damci ,50 ml',
       'formalin (formaldehid 37 %-li məhlul)',
       'devitor ,daxi̇lə qəbul üçün məhlul - damci 30 ml',
       'levami̇zol 10%-li̇ məhlul (i̇nyeksi̇ya üçün)',
       'çaytikanı yağı, məhlul 100 ml', 'askor-zn məhlul 30 ml',
       'defaktor d daxilə qəbul üçün məhlul 30 ml',
       'ultra ginkgo plus, daxilə qəbul üçün məhlul 10 ml, №10',
       'demaks, daxi̇lə qəbul üçün 10 ml məhlul-damci', 'oksisprey',
       'aminostyl daxilə qəbul üçün məhlul-damci 50 ml',
       'legam, daxilə qəbul üçün məhlul 150 ml',
       'deca, daxi̇lə qəbul üçün məhlul 150 ml',
       'reks və musya insekto-akarisid damci it və pişiklər üçün',
       'dezi sprey', 'deglos , daxilə qəbul üçün məhlul damci 15 ml',
       'senovit daxilə qəbul üçün məhlul-damci 15 ml',
       'kaprovit daxilə qəbul üçün məhlul damci 15 ml',
       'melonop f, daxilə qəbul üçün məhlul damci 50 ml',
       'desol-d3 ,daxilə qəbul üçün məhlul-damcı 15 ml',
       'devi sprey, məhlul 20ml', 'melasprey, oral sprey 10ml',
       'nefroles, daxilə qəbul üçün məhlul, 100 ml',
       'amino eaa xplode drink zero lemon flavour, daxilə qəbul üçün məhlul 330 ml',
       'cortex, məhlul 10ml №10',
       'movi̇ment-d3, 5 ml daxi̇lə qəbul üçün məhlul-damci',
       'caldesun,daxilə qəbul üçün məhlul,200 ml',
       'venatura pelarim, daxilə qəbul üçün məhlul 20 ml',
       'essential sorbent joint comfort/essential sorbents oynaq üçün fitosorbent. daxilə qəbul olunan məhlul hazirlamaq üçün toz, 80q',
       'essential vitamins vitamin d3, daxilə qəbul üçün məhlul-damci 30ml',
       'urotropin məhlulu 40%', 'refamin, məhlul 100ml',
       'novapol b12 məhlul',
       'kontrseks neo neo (dişi pişiklər və itlər üçün damcilar)',
       '“fedomax” + daxilə qəbul üçün məhlul-damci 15 ml',
       'nutri̇comp standart neutral məhlul 500 ml',
       'solukar 10 ml daxilə qəbul üçün məhlul №10',
       'mersobil ,daxi̇lə qəbul üçün məhlul - damci 50 ml',
       'thermo çay,daxilə qəbul olunan məhlul hazirlanmaq üçün toz(konsentrat) 50 q',
       'r-weiler shot (orange juice flavour), məhlul 60 ml, №20',
       'chela-mag b6 forte shot, məhlul 25ml, ampul №20',
       'aakg 7500 extreme shot (cherry flavor), məhlul 25 ml, ampul №20',
       'l-carnitine 3000 extreme shot, məhlul 25ml, №20',
       'extreme speed shot, 25ml məhlul, №20',
       'sivi propolis, daxilə qəbul üçün məhlul-damci, 30 ml.',
       'fitolaks konsentrat,məhlul 100ml',
       'slipan, daxilə qəbul üçün məhlul 50 ml', 'pix fagi sprey',
       'fipron 2.5 mq/ml sprey', 'epepe aktiv məhlul damci 50 ml',
       'karbemin, daxilə qəbul üçün məhlul-damci, 50 ml',
       'uromidol, daxilə qəbul üçün məhlul-damci, 20ml',
       'hardline nutrition thermo l karnitin green apple flavored, məhlul 1000 ml',
       'ri̇o-d, daxi̇lə qəbul üçün məhlul 15 ml',
       'helprem, daxi̇lə qəbul üçün məhlul, 15 ml №10',
       'nutrigen vitamin d3, daxilə qəbul üçün məhlul - damci sprey formasinda 20 ml',
       "möller's omega-3 tutti frutti aromali, məhlul 250 ml",
       'dormi̇n daxi̇lə qəbul üçün məhlul-damci 30ml',
       'gutenfer, daxi̇lə qəbul üçün məhlul-damci, 15 ml',
       'guten d3, daxi̇lə qəbul üçün məhlul-sprey, 20 ml',
       'demaks,daxilə qəbul üçün məhlul-damcı,10 ml',
       'denora forte,daxilə qəbul üçün  məhlul-damcı,n 2', 'çemi sprey',
       'ascorkap vitamin c, məhlul-damci 30 q', 'çemi-sprey',
       'biocura aronia 330 ml məhlul',
       'aloe vera extra fresh, məhlul 500 ml',
       'farmachol, məhlul-damci 30 ml',
       'tridrop daxilə qəbul üçün məhlul 50 ml',
       'aeromax, daxilə qəbul üçün məhlul, 50 ml',
       'dekol, 5 ml məhlul №4',
       'detribus, daxilə qəbul üçün məhlul damci ,15 ml',
       'epam 96,daxilə qəbul üçün məhlul 30ml',
       'essential sorbents pi̇k/ essential sorbents təbii inulin konsentrati, daxilə qəbul olunan məhlul hazirlamaq üçün toz,75 q',
       'laytvi̇t, 15 ml daxi̇lə qəbul üçün məhlul',
       'lizoston,daxilə qəbul üçün məhlul 600 ml',
       'stonedol , daxilə qəbul üçün məhlul 600 ml',
       'furokan + daxilə qəbul üçün məhlul-damci 15 ml',
       'nemoten kids, daxilə qəbul ücün məhlul - damci 50 ml',
       'reks və musya askarid sprey ir və pişiklər üçün',
       'makay, məhlul 30 ml',
       'epam 31, daxilə qəbul üçün məhlul-damcı, 30 ml',
       'epam 24, daxilə qəbul üçün məhlul, 30 ml',
       'qlükoza məhlulu 40%-li', 'yod məhlulu 5%-li',
       'gst-ki̇d, daxilə qəbul üçün məhlul-damci 30 ml',
       'i̇lozar,məhlul 10 ml',
       'erbolak, daxilə qəbul üçün məhlul, 200 ml',
       'dvit3, daxilə qəbul üçün məhlul-damci, 10 ml',
       'esseessential sorbents i̇ntestinal defense, daxilə qəbul olunan məhlul hazırlamaq üçün toz, 80 qntial sorbents intestinal/essential sorbents bagirsaq fitosorbenti  80 q   r              bents intestinal defence/essential sorbents bagirsaq fitosorbenti',
       'venatura yod, daxilə qəbul üçün məhlul-damci 25 ml',
       'melakid daxilə qəbul  üçün məhlul-damci, 30 ml',
       'alagastro, məhlul 50 ml',
       'b-nives, daxilə qəbul üçün məhlul-damci, 50 ml',
       'sentocal, 150 ml daxilə qəbul üçün məhlul',
       "möller's omega-3 limon aromali, məhlul 250 ml",
       'carne-grow, daxi̇lə qəbul üçün məhlul 10 ml №10',
       'platovit daxilə qəbul üçün məhlul 200 ml',
       'bi̇tog 50 ml daxi̇lə qəbul üçün məhlul-damci',
       'redokan, daxilə qəbul üçün məhlul, 15 ml',
       'sivi propolis, daxilə qəbul üçün məhlul-damci, 30 ml',
       'farmi d3 ,daxilə qəbul üçün məhlul-damci 15 ml',
       'dharma ,daxilə qəbul üçün məhlul-damci 15 ml',
       'kostimol daxilə qəbul üçün məhlul 150 ml',
       'eurofar daxilə qəbul üçün məhlul, 20 ml',
       'artichoke apero daxilə qəbul üçün məhlul,500 ml',
       'immun fit ,daxilə qəbul üçün məhlul 500 ml',
       'kombivit d3 daxilə qəbul üçün məhlul-damci 20 ml',
       'melatson, məhlul 50 ml', 'ocean vitamin d3 400 iu, məhlul 20 ml',
       'kontra hard, məhlul 30 ml', 'evestrong, məhlul 200 ml',
       'elber kids, məhlul 30 ml', 'distan, məhlul 150 ml',
       'breyom, daxilə qəbul üçün məhlul-damci, 50 ml',
       'novakar 10 ml məhlul', 'neym 150 ml daxi̇lə qəbul üçün məhlul №1',
       'd-ferol daxilə qəbul üçün məhlul-damci 10 ml',
       'deltona, daxilə qəbul üçün məhlul - damci 10 ml',
       'dekal+k2, daxi̇lə qəbul üçün məhlul-damci, 10 ml',
       'devrona, daxi̇lə qəbul üçün məhlul-damci, 10 ml',
       'detodel+k2, daxi̇lə qəbul üçün məhlul-damci, 10 ml',
       'oilesen d3, daxilə qəbul üçün məhlul-damci, 10 ml',
       'delabs, daxilə qəbul üçün məhlul-damci, 10ml',
       'kolciferol d3, məhlul 30 ml', 'dekontrol məhlul 15 ml',
       'didrogyl / didrogil, daxilə qəbul üçün məhlul-damci 20 ml',
       'novasan d3, məhlul 15 ml', 'priyas forte, məhlul 120 ml',
       'starobet, məhlul 150 ml', 'pastoral, məhlul 50 ml',
       'colispas, daxilə qəbul üçün məhlul-damci, 30 ml',
       'veti̇xol daxi̇lə qəbul üçün məhlul, 50 ml',
       'epam 1000, daxilə qəbul üçün məhlul, 30 ml',
       'platin q10, daxilə qəbul üçün məhlul, 100 ml',
       'ayased məhlul 50 ml', 'memoginkar-q 100 ml məhlul',
       'delife məhlul (damcı) 30 ml', 'pasifon kids məhlul 150 ml',
       'zekovit daxilə qəbul üçün məhlul damcı 15 ml',
       'rudelak daxilə qəbul üçün məhlul damcı 15 ml',
       'funtin, daxilə qəbul üçün damc, 30 ml', 'cinkobevit 50 ml damcı',
       'pedikar 50 ml damcı',
       'altekid daxilə qəbul üçün məhlul-damcı, 30 ml',
       'd-power, daxilə qəbul üçün məhlul-damcı, 15 ml',
       'defans, daxi̇lə qəbul üçün məhlul-damci, 15 ml',
       'd-force,  daxi̇lə qəbul üçün məhlul-damci, 15 ml',
       'prostilin, daxilə qəbul üçün məhlul-damcı, 20 ml',
       'desun, daxilə qəbul üçün məhlul-damcı 20 ml',
       'tempiron daxilə qəbul üçün məhlul-damcı 30 ml',
       'inocit, 50ml daxilə qəbul üçün məhlul-damcı n1',
       'sundrops, daxilə qəbul üçün məhlul-damci, 30ml',
       'venatura vitamin d3, daxilə qəbul üçün məhlul-damcı, 20 ml',
       'chondrostrong, daxi̇lə qəbul üçün suspension gel, saşe, 10ml x 30',
       'polivit , balzam (məhlul)  100 ml',
       'kardioşit, balzam (məhlul) 100 ml',
       'l-eli̇xi̇r / l-eli̇ksi̇r daxi̇lə qəbul üçün məhlul-damci 30 ml',
       'caldesun daxilə qəbul üçün məhlul 200 ml',
       'altekid daxilə qəbul üçün məhlul-damcı 30 ml',
       'candi b 200 ml daxilə qəbul üçün məhlul n1',
       'ironlink kids (ayronlinkk uşaqlar uçün) məhlul 8 ml no 10',
       'diapeps, daxilə qəbul üçün məhlul, 125 ml',
       'nonstress kids, daxilə qəbul üçün məhlul, 150 ml',
       'colenzim plus, daxilə qəbul üçün məhlul, 125 ml',
       'genodrop d3, daxilə qəbul üçün məhlul-damcı 15 ml',
       'tenat məhlul (damci) 30 ml',
       'extratop urologi̇cal/ekstratop uroloji̇,daxi̇lə qəbul üçün məhlul ,600 ml',
       'depovi̇t, 15 ml daxi̇lə qəbul üçün məhlul-damci , №1',
       'reginat daxilə qəbul üçün suspenziya 200 ml',
       'redovit daxilə qəbul üçün məhlul damci 15 ml',
       'i̇ronlink adult(ayronlink böyüklər üçün) məhlul 8ml, №10',
       'nutricomp standart neutral, məhlul 500ml',
       'maltazon məhlul 50 ml', 'suptali̇fe, məhlul 150 ml',
       'speech ki̇ds məhlul 100 ml',
       'prudens d3,daxi̇lə qəbul üçün məhlul-damci,15 ml',
       'alteki̇d daxi̇lə qəbul üçün mahlul-damci 30ml',
       'l-elixir/l-eliksir daxilə qəbul üçün məhlul-damcı,30ml',
       'zetavit d3  daxilə qəbul üçün məhlul -damcı 15 ml',
       'kislogon daxilə qəbul üçün məhlul 50ml',
       'zekolek daxilə qəbul üçün məhlul damci 50 ml',
       'neoferol, daxi̇lə qəbul üçün məhlul (damci) 5 ml',
       'botamin, məhlul-damcı 30 ml', 'unodrops, məhlul-damcı 50 ml',
       'tempufen daxilə qəbul üçün məhlul 50 ml',
       'vitalink vitamin d3,daxile qebul uchun mehlul (damci) 20 ml',
       'bemory kids/bemori uşaqlar üçün, №10, oral məhlul',
       'desol -d3 ,daxi̇lə qəbul üçün məhlul-damci,15 ml',
       'deluks ,daxi̇lə qəbul üçün məhlul  -damcı 10 ml',
       'full d-vi̇t ,daxi̇lə qəbul üçün məhlul-damcı ,15 ml',
       'emborg nişanlı viped krem sprey',
       'epid junior 15 ml boğaz üçün sprey n1',
       'tonorex calcium d3 / tonoreks kalsium d3 , daxilə qəbul üçün məhlul 120 ml',
       'zeytun yarpağı ekstraktı, daxilə qəbul üçün məhlul 30 ml',
       'çay tikanı yağı məhlul 100 ml', 'faicol, məhlul 50 ml',
       'depronax d3 , məhlul- damcı  15 ml',
       'bio ferol, daxilə qəbul etmək üçün məhlul-damcı 15 ml',
       'layfsan,daxilə qəbul üçün məhlul (damcı) 5 ml, n2',
       'pronem,daxilə qəbul üçün məhlul 10ml, n10',
       'zadeks d3,daxilə qəbul üçün məhlul-damcı,10 ml',
       'vi̇ta d3 30 ml damla daxilə qəbul üçün məhlul № 1',
       'chole sprey məhlul 20 ml', 'botamin 30 ml  məhlul (damcı)',
       'keprolux daxilə qəbul üçün məhlul-damcı 30 ml',
       'voladex, daxilə qəbul üçün məhlul 50 ml',
       'ritmo-son premium, daxilə qəbul üçün məhlul 30 ml',
       'zekami̇n + daxi̇lə qəbl üçün məhlul-damci 30 ml',
       'paradi̇se forte məhlul 150 ml', 'osso məhlul 150 ml',
       'karni̇-q məhlul 100 ml',
       'betamun, daxilə qəbul üçün məhlul, 150 ml', 'b-mi̇x məhlul  ml',
       'farmachol daxilə qəbul üçün məhlul-damcı 30 ml',
       'ocean vitamin d3,daxilə qəbul üçün məhlul (sprey) 20 ml',
       'denora-3, daxilə qəbul üçün məhlul-damcı, 10 ml',
       's-devit, daxilə qəbul üçün  məhlul 30 ml',
       'epam 4, daxilə qəbul üçün məhlul, 30 ml',
       'carmelin, məhlul 10 ml, №10',
       'black seed oil daxilə qəbul üçün yağ 125 ml',
       'debalance forte, daxilə qəbul üçün məhlul-damcı, 30ml',
       'memodrop b6, daxilə qəbul üçün məhlul-damcı, 50ml',
       'tron-d daxilə qəbul üçün məhlul 200 ml',
       'aktiv flora beybi,oral damcı,5 ml',
       'danil daxilə qəbul üçün məhlul-damcı 30 ml',
       'depay, daxilə qəbul üçün məhlul - d3, 15 ml',
       'kokteyl "dəniz məhsullarından məhlulda hazırlanmış" 200qr',
       'forever freedom (orange flavored aloe vera jui̇se)/forever azadliq şi̇rəsi̇ (portağal əti̇rli̇ aloe vera şi̇rəsi̇), məhlul 1000 ml',
       'deful forte, daxi̇lə qəbul üçün məhlul-damci, 15 ml',
       'lentavi̇t forte, daxi̇lə qəbul üçün məhlul-damci, 15 ml',
       'dekas,daxilə qəbul üçün məhlul, 15 ml',
       'bicom-b, məhlul(damci) 50 ml',
       'silverol daxilə qəbul üçün məhlul 15 ml',
# tablet_flakon_kapsul_dermanlar                                  
'bifidumbakterin,flakon,məhlul hazirlamaq üçün toz№10',
        'faqosporin toz və ya məsaməli kütlə flakon №10',
        'qentamisin sulfat 4% flakon 100 ml',
        'qentamisin sulfat 4% flakon 10 ml',
        'trimetil-a, məhlul flakonda, n20',
        'desol d3+ ,daxilə qəbul üçün məhlul-damci, 5 ml flakon № 4',
        'bergwacholder extrakt (plasti̇k flakonda) 470 q',
        'equilink,mehlul (plastik flakonda) 10 ml',
        'propolink,məhlul (plastik flakonda) 30 ml',
        'orthomol vital f / ortomol vital f , maye /kapsul , flakon №7',
        'zinco green, daxilə qəbul üçün məhlul 150 ml şüşə flakonda №1',
        'orthomol i̇mmun liquid / ortomol i̇mmun maye , maye / tablet plastik flakon №7',
        'tablet plastik flakon №7',
        'bifidumbakterin,məhlul hazirlamaq üçün toz və ya məsaməli kütlə (flakonda) № 10',
        'fi̇brami̇x, n60 plasti̇k flakonda,yumşaq jelatin kapsul',
        'nutricomp standart neutral, məhlul (plastik flakonda)500ml',
'cewitan vitamin c 200 mq, tablet №625', 'verazol ,tablet №30',
        'venatura melatonin odt, tablet №90',
        'venatura metilfolat odt, tablet №30',
        'venatura maqnezium malat, tablet №60', 'malidens tablet №30',
        'fitstar c, fişildayan tablet, №15', 'sorbicard, tablet №30',
        'sommelin, tablet №60', 'solid-as, tablet №30',
        'nomi̇nexi̇n, tablet №30', 'foli̇mi̇n-forte, tablet №60',
        'polen, tablet №60', 'kara mürver, tablet №30',
        'inevra tablet № 30', 'salkovit/salcovit , tablet № 30',
        'fitin, tablet №40', 'sunset plus tablet № 30',
        'sedamix pro tablet №30', 'riocal, tablet №30',
        'vi̇tason, tablet №30', 'magnenorm, tablet №60',
        'forever bee propolis, tablet №60', 'forever calcium, tablet №90',
        'rudefar tablet n30', 'maksifol tablet №30', 'metifol tablet n30',
        'folio №120 tablet', 'folio №20 tablet', 'kostimol tablet № 60',
        'nobelux, tablet № 60', 'marelin neo, tablet № 30',
        'restomag, tablet № 30', 'neurotin,tablet № 30',
        'sedalux, tablet № 30', 'bonestone, tablet № 30',
        'elasti-q, tablet № 30', 'magnat, tablet №30',
        'i̇mpulmax, tablet №30', 'd3 aktiv denk,tablet №100',
        'protosfer, tablet №30', 'zintox tablet n30',
        'gammafertil prenatal tablet n30', 'qlisimag b6, tablet №30',
        'metrip tablet №30', 'epindol forte tablet №60',
        'daseron forte tablet №30', 'endofol, №30 tablet',
        'spazmag tablet №30', 'gastro fast tablet №30',
        'zerostress, tablet №60', 'servaks, tablet №30',
        'hondrolex extra, tablet №60', 'asferon c tablet №30',
        'viazim tablet №30', 'plusssz junior d3+calcium, tablet №30',
        'mikronet, tablet №30',
        'venatura maqnezium sitrat və p-5-p (vitamin b6), tablet n60',
        'forever nature-min tablet №180', 'nemober, tablet n30',
        'luvit-r tablet n30', 'sinaliv tablet №30',
        'chondrowise, tablet №30', 'kalsidens, tablet №30',
        'lithos, tablet №100', 'nerogab tablet №30', 'barovit tablet №30',
        'fleksobon / flexobon, tablet № 60', 'aflib, tablet №20',
        'prudens i̇mmuno tablet №20', 'prudens c500 tablet №20',
        'solitra, tablet №30', 'hipoksikar, tablet №30',
        'urosolis, tablet №30', 'serenzim, tablet №30',
        'melosedin tablet №30', 'vitafull tablet №30',
        'sulajin tablet №30', 'venatura selen tablet n90',
        'neurovef , tablet n30',
        'nemoir f, kapsul / dilalti tablet, n30/30',
        'nemoten l, tablet n20', 'fi̇brenal tablet n60',
        'aci̇dekzol tablet n50', 'sedakar tablet №30',
        'flodox forte , tablet №30', 'panakar, tablet № 60',
        'uromira tablet №30', 'urokar tablet № 30', 'reliston tablet №30',
        'pirocal tablet №30', 'letaze forte, tablet №30',
        'legaston plus tablet №30',
        'cefavit d3 1000 bv, örtüklü tablet n 60',
        'cefasel 100 nutri, tablet n20', 'viafemin tablet №30',
        'deprevit bio tablet №30', 'pms 50 tablet n:30',
        'novarest , tablet №30', 'selen, tablet №30', 'status, tablet №30',
        'nevrotram, tablet №30', 'vettrimerazin tabletka',
        'silima-b tablet №30', 'womaxil tablet n30',
        'reneval collagen tablet n30', 'vi̇nkarti̇n tablet n30',
        'menaxi̇l tablet n30', 'butsin tablet №20', 'seksinon tabletka',
        'metronidazol tablet', 'sisturin, tablet №30',
        'prostalux tablet №30', 'solidofol ,tablet №30',
        'prenat tablet № 30', 'normavas tablet № 30',
        'maxiday, tablet №30', 'centramin,tablet №30',
        'normos forte tablet № 30', 'oksitetrasiklin tabletka 250 mq',
        'betaful, tablet №30', 'tubostat" №30 tablet',
        'revigen, tablet №60', 'osteovef, tablet №30',
        'simonel , tablet №30', 'kolagen activ plus, tablet №80',
        'kvinor, tablet №30', 'sulfaprim 480 №100 tabletka',
        'calmax tablet', 'spektrum prenatal, tablet №30',
        'spektrum, tablet №30', 'lebermax, tablet №30',
        '17/497- alfaseptol tablet', 'montana 300 , tablet n 30',
        'lakto-fi̇ltrum eko tablet №60', 'bi̇o calma tabletn30',
        'skantrum tablet n 30', 'rudatek tablet n 30',
        'hepasept, tablet №30', 'seloxid / seloksid, tablet №30',
        'anabolic amino 9000, tablet №300', 'lipakreon, tablet №60',
        'plenux, tablet №30', 'exsovi̇n, tablet №30',
        'dağ kalsiumu d3,tablet no 80',
        'gördəvər-evalar/sabelnik-evalar,tablet no 60',
        'ginkqo biloba evalar,tablet no 40',
        'askorbin turşusu evalar 100 mq qlükoza ilə,tablet no 60',
        'ovesol,tablet no 40', 'damotu ekstraktli evalar,tablet no 50',
        'damotu forte evalar,tablet no 40',
        'ovesol gücləndirilmiş formul,tablet no 20',
        'qızıl mumiyo evalar,tablet no 20', 'taym ekspert,tablet no 20',
        'laminariya,tablet no 100', 'alaqanqal ekstrakti,tablet no 20',
        'qaragilə-forte vitaminlərlə və sink ilə,tablet no 50',
        'xitozan evalar,tablet no 100', 'ruscus, tablet №30',
        'caldep, tablet №30', 'terris active ,tablet n 90',
        'metridiazin oral tablet', 'broncho natural, tablet №30',
        'calmen,tablet,№30', 'sambovin,tablet,№30', 'karsofen, tablet №30',
        'sodep forte, tablet №30', 'sorimaks, tablet №30',
        'nebufer, tablet n30', 'klimastop, tablet n30',
        'uromed tablet № 60', 'vestli̇ver tablet № 60',
        'zi̇nefol tablet № 50', 'ferratonal №30 tablet',
        'ferrofolvit n 30 tablet', 'saframag tablet №60',
        'unomaq-alisa, tablet №30', 'avers, tablet №30',
        'vita balance, tablet №30', 'i̇munovel, tablet №45',
        'afleksin, tablet №60', 'fernicsil,tablet,№30',
        'mi̇grend plus n30 tablet',
        'qaragilə-forte vitaminlərlə və sink ilə,tablet no 150',
        'fitolaks,çeynəmə tablet no 20', 'forever calcium, tablet№90',
        'qaragilə-forte lüteinlə,tablet no 100',
        'cosakondrin op tablet n30', 'sondomag, tablet №30',
        'qizil mumiyo evalar tablet №20',
        'qaragilə forte lüteinlə tablet №50',
        'askorbin turşusu evalar 100mq qlükoza ilə tablet №60',
        'samerin tablet,№24', 'fitolaks çeynəmə tablet №20',
        'ginkqo biloba evalar tablet №40', 'euromag b6, tablet №30',
        'vipanzim, tablet №30', 'enzimaza forte, tablet №50',
        'oximag, tablet №30', 'deluxevit tablet №30',
        'livolen plus, tablet №30', 'solidoste,tablet №30',
        'dostanol forte tablet № 60', 'ramon plus № 30 tablet',
        'sulasten forte № 60 tablet', 'vagoslan tablet №45',
        'tanalgin tablet №30', 'soneks №30 tablet', 'ovesol tablet №40',
        'damotu forte evalar tablet №40',
        'adaçayi/şalfey sorma tablet №20',
        'santro vit c, fişildayan tablet, №15', 'ferromax ,tablet № 30',
        'laryn family mix, tablet № 36', 'anfomin tablet №30',
        'nervocol complex, tablet №30', 'sistinorm, tablet №20',
        'nemoliv, tablet n30', 'dnk-vit tablet № 40',
        'agrix plus tablet №60',
        'siberian super natural sport mega essentials / siberian super natural sport meqavitaminlər, tablet №120',
        'encegam tablet №30',
        'vitalife vitamins & minerals №40 tablet+20 kapsul',
        'siberian super natural sport glucosamine chondroitin/ siberian super natural sport glukozamin və xondroitin tablet n 120',
        'power zn tablet №30', 'elurox, tablet №30',
        'dayfull tablet , n30', 'megvitab №60 tablet',
        'forever absorbent c, tablet n100', 'pervenol, tablet №30',
        'calcium/kalsium tablet №60', 'cerebrovin , tablet №30',
        'proqram b6, tablet №20', 'santro vit c,fişildayan tablet, n15',
        'ati̇tromb n60 tablet', 'epas, tablet № 60',
        'calcium d3 forte, tablet, №60', 'epokal forte, tablet №30',
        'osteohard, tablet, №30', 'super ala 800 tablet № 20',
        'mumiyo" №30 tablet', 'zural, tablet №30',
        'swiss energy magnesium + b complex, tablet №20',
        'biotri, tablet №30', 'ferison duodenal , tablet № 30',
        'zincson , tablet № 30', 'desol vita complex ,tablet № 30',
        'i̇munovin ,tablet № 30', 'ultrain , tablet № 30',
        'i̇mmon ,tablet № 30', 'hu-metil ,tablet № 60',
        'chondra era, tablet № 30', 'sedoart , tablet № 30',
        'foli̇nferi̇k №30 tablet', 'balansfol tablet №50',
        'mestovan, tablet n20', 'astera tablet №30',
        'acerola chewable tablets with vitamin c, tablet №60',
        'activline vitamin c, fışıldayan tablet №20',
        'fosfazon tablet n30', 'sakura, tablet №45', 'rev-500, tablet №40',
        'novabina tablet n30', 'obimag tablet n45',
        'swiss energy collagen,  tablet №20',
        'swiss energy calcium + vitamin d3, tablet №20',
        'vites, tablet №20',
        'siberian super natural sport meqavitaminlər, tablet №120',
        'venatura magnesium bisglycinate, tablet №60',
        'megatron tablet №30', 'portaze, tablet №30',
        'i̇namiks forte, tablet №30', 'forever bee pollen (tablet №100)',
        'orthomol i̇mmun / ortomol i̇mmun tablet/kapsul , paket n30',
        'orthomol i̇mmun / ortomol i̇mmun tablet/kapsul , paket n15',
        'orthomol vital m / ortomol vital m toz/tablet/kapsul №15',
        'orthomol vital f / ortomol vital f toz/tablet/kapsul №15',
        'forever nature-mi̇n/forever natur-mi̇n, tablet n180',
        'balanced, tablet №30', 'siberin tablet n30',
        'ornicsil ,tablet,№30',
        'selenium plus zinc/selen plus sink,tablet no 90',
        'prostagold, tablet №30', 'alflexil, tablet №30',
        'nonstress, tablet №30', 'medenza, tablet №30',
        'nefrolit, tablet №30', 'androstan ultra tablet №60',
        'sunset plus, tablet №30', 'zinkomax, tablet №30',
        'salutem forte, tablet №20', 'novanefron, tablet №30',
        'ocean microfer tablet №30', 'ulkasel-sm tablet n 30',
        'piremax tablet n 30', 'osteoton tablet n 30',
        'vi̇beran, tablet № 30', 'juni̇-c, sorma tablet №45',
        'vestabron,tablet №30', 'restofer tablet № 30',
        'vitamin c 500 mq, tablet №10', 'vitamin c 500 mq, tablet №20',
        'hepatosil tablet №30', 'hairmax, tablet №30',
        'riobeta, tablet №30', 'qayten, tablet №30',
        'balen fonsept licorice extract+vitamin c & zinc with orange aroma \\ balen fonsept biyan kökü+vitamin c  və sink, portağal dadlı tablet №24',
        'balen fonsept echinacea+honey&propolis with lemon aroma \\ fonsept exinaseya+bal və propolis, limon dadlı tablet №24',
        'balen fonsept eucalyptus+menthol&mint with mint aroma \\ balen fonsept evkalipt+mentol və nanə, nanə dadlı tablet №24',
        'balen fonsept ginger+turmeric&juniper with orange mandarin \\ fonsept zəncəfil+zərdəçal və ardıc, naringi dadlı tablet №24',
        'vitamin c şəkər və portağal dadi ilə, tablet №2',
        'orthomol i̇mmun liquid / ortomol i̇mmun maye , maye / tablet plastik flakon №7',
        'spektrum tablet №30', 'spektrum prenatal tablet № 30',
        'tablet plastik flakon №7', 'referans plus n30 tablet',
        'kalmafort №30 tablet', 'vi̇tami̇n c ulutan fişildayan tablet №20',
        'fat metaboliser/ yağ metaboli̇zatoru tablet №56',
        'lutemax tablet №30', 'ultra vitamin d (d3) 10000 i̇u, tablet №96',
        'glinat tablet, №60', 'tuvia №30 tablet',
        'magnised forte №30  tablet', 'vitacold ester-c, tablet №20',
        'muttarin tablet n30', 'bi̇ovi̇t complex, tablet №30',
        'antidep,  örtüklü tablet №30', 'antidep, örtüklü tablet №30',
        'sunrise c-vit 999, tablet n20', 'benefit-bs , tablet № 30',
        'pfefferminz gruentee drops. tablet n120',
        'kalsemid,  tablet  №50', 'magnefol tablet №50',
        'kalsilong d3 tablet №30', 'swiss energy vitamin c tablet №20',
        'forever royal jelly, tablet, n60',
        'centrum silver multivitamin complex from a to zinc (sentrum silver multivitamin kompleksi a-sinkə qədər) tablet №30',
        'vitamin с 900 mq,fışıldayan tablet no20', 'carniton tablet n30',
        'vitamin c okster tablet №30', 'calci̇far ,tablet №30',
        'vi̇tarest i̇mmuno , tablet №30', 'ni̇plex ,tablet №30',
        'neuroti̇n ,tablet №30', 'elasti̇-q,tablet №30',
        'premax ,tablet №40', 'maxi̇day,tablet №30',
        'non-asi̇doz,tablet №50', 'selavit, tablet n30',
        'vitagamma d3 1000 i.u tablet №100',
        'marine collagen 1000mg with vitamin c tablets',
        'gastrimax tablet №12', 'cardimax tablet n50',
        'gastrobest tablet №12',
        'ultra vitamin c fizz,fışıldayan tablet no 20',
        'calzin-d, tablet №30', 'arganorm tablet №30',
        'opti̇gold №30 tablet', 'magnaton tablet n 50',
        'riaton tablet n 30', 'riacor tablet n 90', 'neutron tablet №60',
        'i̇mmuni̇tex , tablet №30', 'pruselin , tablet №30',
        'denskard,tablet №30', 'grendecal, tablet n30',
        'neurovef, tablet №30', 'pikovit 4+ çeynəmə tablet (pastil) №30',
        'd3 juni̇or denk / d3 uşaqlar üçün denk tablet №100',
        'd3 active denk / d3 aktiv denk tablet №100',
        'selen 100 mkr vi̇tami̇r tablet №30', 'mc-vit 60 tablet',
        'unimaq, tablet №30', 'sentry tablet №30', 'roicard, tablet n30',
        'miomag №30 tablet', 'tireozob №60 tablet',
        'vitagamma d3 1000 i.u,tablet no100', 'fitin,tablet №40',
        'osteokea fizz, fışıldayan tablet, № 20',
        'i̇munofull , tablet №30', 'soli̇do forte , tablet №30',
        'melimax, tablet №30', 'i̇mmuno-vaq forte , tablet №30',
        'salkovi̇t/salcovi̇t ,tablet №30', 'arti̇kal ,tablet №30',
        'calci̇best ,tablet №30', 'deluks-1000 ,tablet №60',
        'remenol,tablet №60', 'kalness tablet №30',
        'ari südü, tablet  №30', 'singavit tablet n30',
        'gammaferti̇l tablet №60', 'hepas tablet № 60',
        'zazgi̇n tablet №30', 'fomi̇gan tablet № 30',
        'faroenzym hb tablet №20', 'forever bee propolis, tablet№60',
        'novacal-d,fışıldayan tablet n60', 'vitix tablet №30',
        'fitstar c, fışıldayan tablet, n15',
        'umbrella, fışıldayan tablet №15',
        'santro vit c,fışıldayan tablet, n15', 'vestliver tablet №60',
        'aspermed - l , tablet №60', 'sitratin tablet №60',
        'gettafer tablet n30', 'novasan,tablet №30', 'sambovin,tablet №30',
        'hemomag tablet №30', 'reful tablet  №30',
        'spektrum prenatal,tablet no30',
        '"centrum multivitamin complex from a to zinc (sentrum multivitamin kompleksi a-sinkə qədər) tablet №30',
        'vitason tablet №30', 'kapitos tablet №30', 'verazol, tablet n30',
        'vestalon, tablet n30', 'si̇mfer tablet №30', 'lanahep tablet №30',
        'orthomol juni̇or c plus chewable tablets ( wild berry) / ortomol junior c plus çeynəmə tabletlər ( meşə gi̇ləmeyvəsi̇) çeynəmə tablet (paketdə) №30',
        'orthomol juni̇or c plus chewable tablets (mandarin , orange & wild berry) / ortomol junior c plus çeynəmə tabletlər (mandari̇n , portağal və meşə gi̇ləmeyvəsi̇) , çeynəmə tablet (paketdə) № 14',
        'citracal,tablet n30', 'kostatin,tablet №30',
        'duovit women / duovit qadinlar üçün tablet 30',
        'duovit men / duovit kişilər üçün tablet №30', 'i̇ntes tablet №30',
        'nevrolex №30 tablet', 'forever royal jelly, tablet №60',
        'lenomax, tablet n30', 'verkali̇n, tablet n30',
        'l-carnitine/l-karnitin,tablet no30', 'zerax forte, tablet №30',
        'panakar tablet №60', 'reloflex, tablet №60',
        'sulasten forte tablet №60', 'dostanol forte, tablet №60',
        'maximum strength zinc/maksimum güclü sink,tablet no100',
        'malidens, tablet №30', 'selevef, tablet №30',
        'fitstar c fişildayan tablet n15', 'sedalux,tablet №30',
        'chonzami̇n ,tablet №30', 'neocti̇zi̇n №30  tablet',
        'vitus forte tablet №30', 'forever keys, tablet,№120',
        'restvi̇t forte n 30 tablet', 'furagab tablet n30',
 'plamin, kapsul №50', 'etomadin, kapsul n30',
        'tiogren, kapsul n40', 'pari-flo,kapsul №30', 'donsir, kapsul №30',
        'komfovita, kapsul №30', 'menolytin / menolitin kapsul №30',
        'treataprost, kapsul №30', 'ginjal urit 60 kapsul',
        'venatura xlorella, kapsul №60', 'venatura bromelain, kapsul №60',
        'venatura 5-htp kapsul №30', 'evadol, kapsul №60',
        'miamor, kapsul, №30', 'miofer, kapsul, №30',
        'gasduram kapsul n30', 'afrika şi̇ri̇ kapsul n10',
        'erex sper n60 kapsul', 'erex sper kapsul №60',
        'neurovin, kapsul №30', 'omemag cardio, kapsul №30',
        'bio azf, kapsul №60', 'renadil, kapsul №30',
        'stresonat, kapsul №30', 'glucosamine chondroitin msm,kapsul №90',
        'kalsium, magnezium, sink, d vitamini kapsul №90',
        'cüce palmiye, kapsul №60', 'akgünlük, kapsul №60',
        'magnimax №60 kapsul',
        'forever active ha/forever aktiv ha, kapsul №60',
        'elis forte, kapsul №60', 'smartex, kapsul №30',
        'flanzen kapsul №30', 'si̇msed 30 kapsul', 'protesol, kapsul №30',
        'dyukast, daxilə qəbul üçün məhlul ilə birgə kapsul, 5 ml №10',
        'menolitin, kapsul №30', 'makafertil, kapsul №90',
        'urobest, kapsul №30', 'qlisimag b6, kapsul №30',
        'grenzi, kapsul n30', 'lutein omega 3 , kapsul n 30',
        'hepolaf kapsul №30', 'hepabest forte, kapsul №30',
        'biozyme,kapsul №20',
        'essential fatty acids siberian li̇nseed oil & omega-3, kapsul №30',
        'siberian super natural sport ultra omega-3/siberian super natural sport ultra omeqa-3, kapsul №120.',
        'elbifid, kapsul №15.', 'novomin, kapsul n 120',
        'si̇beri̇an super natural nutri̇ti̇on neurovi̇si̇on paket/kapsul(20x3)',
        'essential minerals selenium/ essential minerals orqanik selen kapsul n 60',
        'essential minerals calcium/essential minerals orqanik kalsium, kapsul n 60',
        'forever garlic-thyme, kapsul №100', 'maksinit forte kapsul n30',
        'eduser kapsul №30', 'kanoprevit kapsul №30',
        'makafertil kapsul № 90',
        'venatura vitamin a palmitat, kapsul №60',
        'venatura coq-10 koenzim q10 200 mq, kapsul №30',
        'leen (woman) / leen (qadinlar üçün), kapsul №45',
        'sedokalm, kapsul № 30', 'teokard, kapsul №30',
        'tepdan forte, kapsul № 30', 'promental, kapsul № 30',
        'pepdida ,kapsul № 30', 'serolan kapsul № 30',
        'peronop, kapsul № 30', 'nomifort , kapsul № 30',
        'star slim, kapsul n°20',
        'essential vitamins glucosamine and chondroitin, kapsul №60',
        'sedaflor b, kapsul №30', 'ritmi zdorovya, kapsul №60',
        'i̇stoki çistoti, kapsul №180.', 'tianshi spirulina kapsul № 100',
        'tianshi zinc capsules, kapsul №60',
        'tianshi chitosan kapsul № 100',
        'forever active pro-b/forever aktiv pro-b, kapsul №30',
        'ri̇nafos forte, №30 kapsul', 'bigflor, kapsul n 10',
        'forven forte kapsul №30',
        'venatura kids mikroinkapsulyasiya olunmuş dəmir və c vitamini, daxilə qəbul üçün məhlul 20 ml',
        'krominat, kapsul №90', 'lizaflor-c, kapsul №15',
        'linkosed, kapsul № 30', 'fiziogink,kapsul n 30',
        'bvitas kapsul №30', 'munotek kapsul №20',
        'agnuvit bio, kapsul №30', 'forever arctic sea, kapsul №120',
        'sedaton plus kapsul №30', 'lonpasin, kapsul №30',
        'oxybreyn n30 kapsul', 'relaxo max №30 kapsul',
        'ci̇nkofer, № 30 kapsul', 'kofe kapsulu', 'kandidor, kapsul №30',
        'benesist, kapsul №30', 'swiss energy neuroforce kapsul №30',
        'redom, kapsul №30', 'crannoza, kapsul n30',
        'ferrovef, kapsul n60',
        'venatura koenzim q10 və alfa lipoy turşusu, kapsul n60',
        'venatura kurkumin, kapsul n30', 'vivasil kapsul n30',
        'enes forte n30 kapsul', 'materifol n30 kapsul',
        'nemoir f, kapsul / dilalti tablet, n30/30',
        'selifort , kapsul № 30', 'lipoxtor kapsul n30',
        'ritmoson premium kapsul n30', 'sedabest kapsul n30',
        'nefrobest kapsul №30', 'swiss energy calcivit kapsul №30',
        'bilivent kapsul №30', 'hepaşok kapsul n30',
        'defromax, kapsul №30', 'evolyx, kapsul №30',
        'nokamen forte, kapsul №30', 'depresol, kapsul №30',
        'sulfafer, kapsul №30', 'i̇mmunogloba, kapsul №30',
        'caliperon, kapsul №30', 'remega, kapsul №30',
        'visan kompleks kapsul №30', 'hepacar forte №50 kapsul',
        'levolat, kapsul n30', 'repastab forte kapsul №40',
        'renoles forte kapsul №40', 'enzywise,kapsul №30',
        'prudenzim,kapsul №30', 'perunal, kapsul №30',
        'neyrosetin, kapsul №30', 'paumax, kapsul №30',
        'erospin, kapsul №10', 'fondosed kapsul №30',
        'helicoter, kapsul №30', 'anabolic amino 5500, kapsul №400',
        'gold omega 3, kapsul, №60', 'biosed kapsul', 'urbanol kapsul №30',
        'prolagnus kapsul n60', 'nurkasan, kapsul №20',
        'best berry kapsul № 20',
        'trigelm paracleanse triple set, toz 200q/kapsul №90.',
        'essenti̇al mi̇nerals magnesi̇um / essenti̇al mi̇nerals orqani̇k maqnezi̇um, kapsul №60',
        'uranef, kapsul №60', 'zinofen fm ,kapsul n 30',
        'dortil complex , kapsul n 30', 'enzilife , kapsul n 30',
        'eqloforin kapsul №30', 'maqnisan kapsul n40',
        'maqnisan, kapsul №40', 'femitrop kapsul №60',
        'synchrovitals iv / sinxrovital iv, kapsul №60',
        'bcaa 1100 mega caps, kapsul, №120',
        'glucosamine plus, kapsul №60', 'omecaps ,kapsul № 60',
        'çörəkotu yaği kapsul №80', 'afrika pələngi, kapsul №10',
        'triptodep, kapsul n30', 'fuqardin, kapsul n30',
        'adefosin, kapsul n60', 'sedofaq, kapsul №30',
        'memori şarp, kapsul n30', 'kartil - ms super forte, kapsul n30',
        'rowalespril n30 kapsul', 'selekte kapsul №30',
        'best-rest, kapsul №30', 'neuro gamma kapsul n30',
        'turboslim gündüz gücləndirilmiş formul,kapsul no 30',
        'troyçatka evalar,kapsul no 40', 'hepatrin,kapsul no 30',
        'hepatrin,kapsul no 60', 'yoximbe forte,kapsul no 30',
        'möller s omega-3, kapsul №60', 'nodep, kapsul №30',
        'selvidol, kapsul №30',
        'synchrovitals iv / sinxrovital iv, kapsul no 6',
        'essential fatty acids siberian borage and amaranth oil kapsul №30',
        'essential minerals zinc / essential minerals orqanik sink n 60 kapsul',
        'chronolong/xronolonq kapsul n 30',
        'essential botanics valerian & melissa / essential botaniks valerian və melissa , kapsul n 30',
        'all - natural beta-carotene in sea buckthorn oil/ təbii beta karotin və çaytikani yaği, kapsul n 30',
        'synchrovitals ||| /sinxrovital ||| ,kapsul №100',
        'lesistan, kapsul n-30', 'osidoks, kapsul №30',
        'sedon forte, kapsul №30', 'sirido, kapsul №30',
        'visokal, kapsul №30', 'neurocortin / neyrokortin, kapsul №30',
        'forselen №60 kapsul', 'sedasan kapsul n30', 'folzed kapsul n30',
        'memo day, kapsul №30', 'oftavit n 30 kapsul',
        'smart hit ferrum, kapsul №30', 'ecoflakton, kapsul №20',
        'sedalin forte,kapsul №20', 'xonda,kapsul no 30',
        'grengam plus, kapsul n30',
        'turboslim gece gücləndirilmiş formul,kapsul no 30',
        'flexiforce plus kapsul №60', 'passicura №30 kapsul',
        'tokofer, kapsul №30', 'probilac / probilak, kapsul №10',
        'ginkoum bio kapsul №30', 'denomaks, kapsul №30',
        'solefarin i̇muno, kapsul №15', 'presnora, kapsul №30',
        'i̇muno-sitoz, kapsul №30', 'kombilak, kapsul №30',
        'laksoren, kapsul №30', 'serranat forte, kapsul №30',
        'manuflor, kapsul №30', 'novospal, kapsul №30',
        'venatura sink & mis, kapsul №60', 'levalon sr kapsul n30',
        'synchrovitals vi/sinxrovital vi ,kapsul №60',
        'hepatrin kapsul №30', 'essential omega-3, kapsul №90',
        'beta zim kapsul n30', 'prostam kapsul n30', 'karbin ,kapsul № 30',
        'nominel forte ,kapsul № 30', 'bestom, kapsul № 60',
        'neo-enzi̇m kapsul №30', 'nefrenol ,kapsul № 30',
        'omeqa lux kapsul №30', 'butifar kapsul №30',
        'deprestin plus , kapsul № 30', 'symbiotic+, kapsul №10',
        'obatin, kapsul № 30', 'denogab, kapsul № 30',
        'grenforte, kapsul n30', 'urogold, kapsul n30',
        'elfolat-m, kapsul n30', 'duoagnus, kapsul n30',
        'relaxin, kapsul n30', 'brainmax, kapsul n30',
        'beneflor, kapsul n30', 'max ultralac, kapsul №10',
        'swiss energy omega-3 multivit, kapsul №30',
        'i̇munoglukan p4h, kapsul №40', 'mersil neuro kapsul №60',
        'ferogren, kapsul n30', 'zofish, kapsul №30',
        'serrokar er, kapsul №30',
        'neyroeyd ii mlc 901 /neuroaid ii mlc 901,kapsul №180',
        'bekol № 60 kapsul', 'ferfolin kapsul № 30',
        'vitalife vitamins & minerals №40 tablet+20 kapsul',
        'synchrovitals || ( brain support) / sinxrovital || , kapsul n 60',
        'essential fatty acids nordic omega -3/ essential fatty acids şimali omega-3, kapsul n 60',
        'si̇yah boğa/black bull №10 kapsul', 'suvastin , kapsul №30',
        'i̇ndolex n60 kapsul', 'pri̇mon forte n30 kapsul',
        'grenlas, kapsul n30', 'phospho i̇ntellect, kapsul n30',
        'pelagon i̇mmuno, kapsul №30', 'menomax, kapsul №30',
        'luzaktol, kapsul №30', 'estrolife, kapsul №30',
        'enformin, kapsul №30', 'hepanur kapsul №60',
        'omega 3-6-9 kapsul №60', 'vi̇konel n20 kapsul',
        'gammabrai̇n n30 kapsul', 'blokalin forte, kapsul №20',
        'polvest n30 kapsul', 'nuclaton n30 kapsul', 'pramed-a n30 kapsul',
        'ati̇laza n30 kapsul',
        'leen (man) / leen (kişilər üçün), kapsul №45',
        'perfect skin hydro-complex, kapsul №30', 'chela-zinc kapsul №30',
        'platinum ginseng 550 sport edition, kapsul №60',
        'novastar kapsul №50', 'senaval, kapsul №30',
        'oksimega, kapsul №30', 'umuno vita, kapsul, №30',
        'seraflazi̇m, №30 kapsul', 'ferraktin, kapsul №30',
        'disurinorm,kapsul №30', 'exbona, kapsul ns30',
        'mauglin, kapsul,№30', 'disurinorm, kapsul №30',
        'normix, kapsul №30', 'evalook, kapsul №30',
        'elevit əmizdirmə, kapsul №30', 'spi̇ruli̇na kapsul №90',
        'rei̇şi̇ göbələyi̇, kapsul №60', 'pro 360, kapsul №80',
        'arginon-l, kapsul №50', 'semiful, kapsul №30',
        'deasedin forte, kapsul № 30', 'karinor, kapsul № 30',
        'mardina,kapsul № 30', 'raserra , kapsul № 30',
        'narased , kapsul № 30', 'denoserra,kapsul № 30',
        'levokard , kapsul № 60', 'disoben, kapsul n30',
        'agliocrat, kapsul, no 60', '"fito 40, kapsul no 60"',
        '"nigenol, kapsul no 60"',
        'leberschutz ultra plus vitamin e (kapsul, no 60)',
        'vital plus” lachsöl , kapsul n60',
        '"rosskastanie weinlaub, kapsul no 60"', 'bigmag kapsul №30',
        'ferractin kapsul №30', 'meflor, kapsul №20',
        'venatura xrom №90 kapsul',
        'venatura fosfatidilserin və omega 3, kapsul №30',
        'mefolic/mefolik, kapsul №30', 'melaz, kapsul №30',
        'trio sinergy, jelatin kapsullar №60',
        'swiss energy activelife kapsul №30', 'prozim, kapsul №30',
        'vitacal / vitakal, kapsul №30', 'strepaksin, kapsul №30',
        'hiposel selen, kapsul №30', 'trioactive, kapsul №30',
        'sanuslak, kapsul №30',
        'orthomol i̇mmun / ortomol i̇mmun tablet/kapsul , paket n30',
        'orthomol i̇mmun / ortomol i̇mmun tablet/kapsul , paket n15',
        'orthomol vital m / ortomol vital m toz/tablet/kapsul №15',
        'orthomol vital m / ortomol vital m maye/kapsul №7',
        'orthomol vital f / ortomol vital f toz/tablet/kapsul №15',
        'prince & princessa, kapsul №10', 'hepolip, kapsul №30',
        'urosit n30 kapsula', 'foldiferin , kapsul № 30',
        'androlife , kapsul № 60', 'forever arctic sea, kapsul n120',
        'alhadaya, kapsul №120', 'good sleep, kapsul №30',
        'pasifon kapsul n 30', 'cerbi̇kor, kapsul №30',
        'gold ray, kapsul №10', 'slezavit, kapsul №30',
        'forever garlic-thyme, kapsul№100', 'tripgold, kapsul №30',
        'urolizin, kapsul №30', 'bioptol kapsuln30', 't-dol n30 kapsul',
        'herbalifeline max, kapsul №30', 'li̇verkaps kapsul №50',
        'atmosil kapsul №15', 'memofit n30 kapsul', 'radi̇max, kapsul №60',
        'venatura silimarin, kapsul №60',
        'orthomol vital f / ortomol vital f , maye /kapsul , flakon №7',
        'antiflam , kapsul № 30', 'aselon kapsul n60',
        'i̇q box kapsul №60', 'primera n30 kapsul', 'fosfodep n60 kapsul',
        'extreme pac, kapsul №30', 'kardi̇prev №30 kapsul',
        'urofit kapsul №30', 'kanomaq kapsul №50', 'liafos, kapsul n10',
        'sagren - sh, kapsul n30', 'kn - plus kapsul №90',
        'ultra slux, kapsul №30', 'resed, kapsul №40',
        'booster, kapsul №60', 'olvinax, kapsul №40',
        'marvelos, kapsul №30', 'pnevmanol, kapsul №40',
        'panto androshan, kapsul №60', 'seuril, kapsul №40',
        'verfen, kapsul №40', 'urodoks, kapsul №30', 'eldox, kapsul №60',
        'adelton, kapsul №30', 'ero day, kapsul №10',
        'l-arginine/l-arginin,kapsul no50', 'himofin plus № 20 kapsul',
        'xromin, kapsul №30', 'megaglukan synbio №30 kapsul',
        'folzed a kapsul  №30',
        'orthomol cholin plus / ortomol xolin plus kapsul №60',
        'ortomol mental toz/kapsul , saşe №30', 'chondorocol kapsul №60',
        'ferracti̇n kapsul №30', 'evadol kapsul №60',
        'maximuno, kapsul №30', 'plexeral /plekseral  kapsul n30',
        'sistonef , kapsul № 30', 'epadez omega 3 1000 mq, kapsul №60',
        'agemo, kapsul №50', 'fondosed, kapsul №30', 'biotenal №30 kapsul',
        'novastar № 50 kapsul', 'senoflam №30 kapsul',
        'venatura sink, kapsul №60', 'vitap gs, kapsul №30',
        'pi̇konat, kapsul №90', 'fukus  kapsul № 60',
        'alqa xeliantus kapsul № 50', 'alqa petra, kapsul №50',
        'alqa validis  kapsul № 50', 'laminarina kapsul №60',
        'vitanorma tonus kapsul № 60', 'lenoks, kapsul n60',
        'natural nest bio glucan, kapsul  n30', 'i̇novi̇t duo, kapsul №30',
        'trimeqavitol lutein and zeaxanthin superconcentrate kapsul 450 mg n 30',
        'essentials by siberian health bearberry and lingonberry n 30 kapsul',
        'essentials by siberian health ginkgo and baikal skullcap n30 kapsul',
        'biozyme kapsul №20', 'hepabest forte kapsul №30',
        'simflor, kapsul №20', 'alfa-omega 369 №30 kapsul',
        'i̇mmunoksid, kapsul №30', 'aktigin, kapsul №30',
        'bekali̇n , kapsul №30', 'bio selen zn  kapsul №30',
        'zink godarov kapsul  №30', 'enser forte kapsul №30',
        'zenavit kapsul №60', 'serrazin kapsul n30',
        'menomax , kapsul n 30', 'luzaktol ,kapsul n30',
        'eurofar kapsul №60',
        'high potency vitamin d3 1000 iu,kapsul №180',
        'cats claw 500 mg\\pişik pəncəsi 500 mq kapsul №100',
        'mis.ar.100% espresso latta 250 ct12 - kofe kapsula',
        'demor n30 kapsul', 'imunol №40 kapsul', 'grenliv, kapsul n30',
        'venatura l-qlütation 250 mq, kapsul №60', 'greencaps 30 kapsul',
        'metabolin kapsul n30', 'hepakti̇v kapsul № 60',
        'eye look  kapsul №30', 'deponur, kapsul n30',
        '"nutrolin, kapsul №20"', 'prosta-ur-f (9) №30 kapsul',
        'hepaktiv kapsul №60', 'feroglobin kapsullar,kapsul no 30',
        'rakinaza kapsul n30', 'serramaks forte , kapsul №20',
        'imuno star kapsul № 30', 'metafoli̇n,kapsul №30',
        'meqranol forte, kapsul №30', 'sperokard, kapsul №30',
        'cereox kapsul №30',
        'fi̇brami̇x, n60 plasti̇k flakonda,yumşaq jelatin kapsul',
        'swiss energy multivit kapsul №30', 'adetolin, kapsul n30',
        'lofal, kapsul n30', 'fortede, kapsul n30', 'dios-ven ,kapsul n30',
        'sisticap, kapsul n30', 'ezomega, kapsul n30',
        'eyebest, kapsul n30', 'vizinorm, kapsul n20',
        'bonlak, kapsul n30', 'fogenalin, kapsul n30',
        'bestkalin, kapsul n30', 'periksen, kapsul n 20',
        'digeszym, kapsul n30',
        'essential vitamins with calcium /kalsiumlu vitaminlər,kapsul 550mg №60',
        'essential minerals orqanik dəmir, kapsul №60',
        'essential minerals orqanik sink,kapsul №60',
        'xronolonq,kapsul №30',
        'essential vitamins beauty vitals/ gözəllik vitaminləri ,kapsul 550mg №30',
        'sinxrovital vii ,kapsul №60', 'elbifid kapsul № 15',
        'trimeqavital lutein and zeaxanthin superconcentrate, kapsul 450mq №30',
        'çörəkotu yaği kapsul № 80', 'c vitamini kapsul №60',
        'pro-360, kapsul  №80', 'karnizonat n30 kapsul',
        '"enkamega , kapsul № 60"', '"insugen , kapsul № 30"',
        '"spermabest , kapsul № 60"', '"livaton , kapsul № 30"',
        '"orsafer , kapsul № 30"',
        'forever active pro-b / forever aktiv pro-b, kapsul №30',
        'lipiscor kapsul №60', 'imunoglukan p4h kapsul №40',
        'epifizin plus kapsula n60', 'nuclaton kapsula n30',
        'desita kapsul n50', 'osteosan kapsula n30',
        'androlife, kapsul №60', '3-x, kapsul №30', 'smartex kapsul n30',
        'cremesso kofe kapsullar', 'biozyme,kapsul no20',
        'atmosi̇l kapsul №15', 'chondrocol kapsul №60',
        'hepabest forte,kapsul no 30', 'extrased, kapsul №30',
        'rs-vera kapsul №30', 'venatura vitamin c, kapsul №60',
        'venatura sambu-c, kapsul №20', 'prostakaps, kapsul №30',
        'ferrovef, kapsul №60', 'merfer, kapsul №60',
        'super volos activ/super hair aktiv,kapsul №50',
        'radimax, kapsul n60', 'forselen ,kapsul n60',
        'selenorm,kapsul№30', 'fermental kapsul n15',
        'omega 3 natura,kapsul № 60', 'nevali̇son, kapsul №30',
        'stressnol, kapsul № 15', 'soluro active, kapsul, №30',
        'demaks caps, kapsul, №30',
        'forever acti̇ve ha / forever akti̇v ha, kapsul №60',
        'orthomol i̇ care / ortomol i̇-care toz/kapsul №30',
        'orthomol immun pro/ortomol i̇mmun pro , toz/kapsul №30',
        'melaxamin l/melaksamin l,kapsul no30', 'lavien-l,kapsul no60',
        'enterikos  kapsul №20', 'epafor kapsul n60',
        'memo-gink kapsul n30', 'protesol kapsul n30',
        'lutein omega 3 kapsul n30', 'eqloranon valeriana n45 kapsul',
        'essential fatty acids şimali omeqa-3,kapsul №60',
        'ritmi zdorovya,kapsul №60',
        'essensials by siberian health ginkgo and baikal skullcap (ginko biloba və baykal başlıqotu,kapsul №30',
        'sinxrovital iv ,kapsul №60', 'ergaten, kapsul №30',
        'siberian super natural sport ultra omeqa-3, kapsul №120',
        'trigelm paracleanse triple set,toz 200q/kapsul №90',
        'essentials by siberian health  bearberry and lingonberry (ayıqulaği və mərcangilə),kapsul №30',
        'essential minerals orqanik selen,kapsul №60',
        'essential minerals orqanik kalsium,kapsul №60',
        'i̇stoki çistoti, kapsul №180',
        'essential botaniks valerian və melissa,kapsul №30',
        'farest kapsul n30', 'esti̇mul, kapsul n30',
        'karas yumşaq kapsul n30', 'slim stay n30 kapsul',
        'novastar, kapsul №50', 'elevit əmi̇zdi̇rmə,kapsul no 30',
        'vitamin e-200 i.u/vitamin e-200 bv,kapsul no100',
        'fitatop kapsul №30', 'nemelin, kapsul №30', 'urodep, kapsul №30',
        'orthomol flavon m / ortomol flavon m kapsul № 30',
        'prossimol gold №30 kapsul', 'ti̇ogren, kapsul n40',
        'forever ivi̇si̇on, kapsul n60', 'omeganorm n30 softgel kapsul',
        'artrofort kapsul n60', 'atinex kapsul n30'
]
tablet_flakon_kapsul_dermanlar = list(set(tablet_flakon_kapsul_mehlul_damci_dermanlar))

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Kapsul, flakon, damcı məhlul və tablet formasında olan dərmanlar" if x in tablet_flakon_kapsul_mehlul_damci_dermanlar else x)

### Derman olmayanlar

In [ ]:
mese_materiali_taxta = ['meşə materiali (taxta)', 'meşə materiali(taxta)',
                        'ağac məmulatları', # Bezek dekorativ agaclari
        'palıddan meşə materialı (palıd bruslar qabığı təmizlənmiş)',
        'kəsilmiş budaqlar', 'bəzək (dekorativ) ağacları və kolları']

hazir_yemekler = [
"hazir yemekler", 
'hazır yemək', 'hazir yemək - düyü',
'hazır səhər yeməkləri start "cocoa pics"',
'hazir səhər yeməkləri̇',
'hazir quru səhər yeməkləri̇ krokodi̇lçi̇ki̇',
'tez hazırlan yeməklər ( makaron ) 50 qr', 'yarımfabrikat']

kesnis_ve_toxumlari = ['keşniş toxumu', 'keşniş meyvələri', 'keşniş', "keşni̇ş"]
uzum_ve_uzumden_hazırlanan_mehsullar_ve_uzume_aid_numuneler = [
'dorsa uzum sirkesi', 'qurudulmuş uzum',
'üzüm', 'üzüm qurusu', 'üzüm yarpağı', 'üzüm tingi',
'meyvə şirələri (üzüm şirəsi daxil olmaqla) və tərəvəz şirələri, qıcqırdılmamış və tərkibində spirt əlavəsi olmayan, şəkər və ya digər şirinləşdirici maddələr əlavə edilməklə və ya edilməməklə:',
'nutrigen üzüm, şərbət 200ml', 'üzüm çəyirdəyi yaği',
'üzüm qurudulmuş sultan', 'üzüm çubuqları', 'üzüm sirkəsi',
'quş üzümü', 'baxçdan quru üzüm 150g', 'qurudulmuş üzüm',
'1300 gr bidon üzüm bəhməzi̇ *6', 'üzüm şirəsi',
'qara qurudulmuş üzüm 2 sort', 'quru üzüm', 'doyca üzüm bəhməzi',
'zakaro li̇monadlari(armud,terxum,üzüm) 0.5lt şüşə', 'üzüm doşabi',
'qara üzüm qurusu 1 sort', 'sari üzüm qurusu 1 sort', 'üzüm  bəkməzi',
'qıcqırdılmamış və tərkibində spirt əlavəsi olmayan, şəkər və ya digər şirinləşdirici maddələr əlavə edilmiş və ya edilməmiş meyvə şirələri (üzüm şirəsi də daxil olmaqla) və tərəvəz şirələri:',
'üzüm bəkməzi̇'                                        
]

patoloji_material = ['quş',
                     'toyuq dərisi',
'hi̇nd quşu si̇nəsi̇ni̇n dəri̇li̇ tri̇mi̇ngi̇',
       'hi̇nd quşu ayağinin dəri̇si̇z tri̇mi̇ngi̇',
       'dondurulmuş toyuq dəri̇si̇ məhsulu',
       'dondurulmuş toyuq əti̇ məhsullari- dəri̇',
       'dondurulmuş toyuq əti məhsullari - dəri̇',
       'dondurulmuş toyuq dəri̇si̇'
    'toyuq dərisi', 'hi̇nd quşu si̇nəsi̇ni̇n dəri̇li̇ tri̇mi̇ngi̇',
'i̇bh dərisi', 'dəri', 'dondurulmuş toyuq dərisi',
'xirda buynuzlu dəri', 'hi̇nd quşu ayağinin dəri̇si̇z tri̇mi̇ngi̇',
'skin,hair and nails formula/dəri,dırnaq və saç formulu,tablet no 60', 
'dondurulmuş toyuq dəri̇si̇ məhsulu',
'dondurulmuş toyuq əti̇ məhsullari- dəri̇',
'dondurulmuş toyuq əti məhsullari - dəri̇', 'gön-dəri',
'dondurulmuş toyuq dəri̇si̇',
    'kal',
'patoloji material', 'i̇bh patoloji material', 'patoka',
'çöl quşu patoloji materiali', 'donuzdan patoloji material',
'camişin patoloji materiali', 'i̇bh patoloji material', 'i̇bh dərisi', 'i̇bh kəllə',
'donmuş iri buynuzlu heyvanlarin i̇ç orqanlari', 'daxili orqan',
'heyvan orqani','i̇ri̇buynuzlu heyvanlarin i̇ç orqanlari donmuş',
'iri buynuzlu heyvanlarin i̇ç orqanlari donmuş', 'quş cəsədləri', "qulaq",]

su_ve_mineral_sular = ["su",'akmi̇na li̇mon + c 200 ml 6*4', 'qazlı su', # qazli
       'akmina qirmizi meyvələr 200 ml 6*4',
       'akmina mandalin + c 200 ml 6*4', 'akmina limon 200 ml 6*4',
'dəngə mədən suyu 0,2 lt limonlu  1x24',
'dəngə mədən suyu 0,2 lt sadə 1x24',
'sari̇ki̇z ni̇şanli maden suyu sadə soda',
'mineral sular', 'esentuki suları (və ya mineral sular)',
'qazli mi̇neral sular essenskaya və retsep ot pri̇rodi̇',
'mineral sular alleya istoçnikov №4,17',
'mi̇neral qazli sular narzan',
'limon aromalı sarıkız nişanlı  qazli sular',
'multivitamin suları',
'borjomi̇ ni̇şanli təbi̇i̇ mi̇neral qazli sular',
'c vitamin aromali mineral sular',
'volvic 50lp ex2 pk6cs4 vr96 beautys ni̇şanli mi̇neral sular',
'essentuki sulari', 'qazlı sular', 'sarikiz mineral sulari',
'eri̇kli̇ su 5 l|tre рет ni̇sanli̇ mi̇neral sular',
'sarıkız nişanlı sadə qazlı sular',
'mineral qazlı sular dr essent №4,17 export',
'təbii mineral müalicəvi- süfrə suyu "yessentuki no. 17" qazlı 0.45l şüşə',
'təbii mineral müalicəvi- süfrə suyu "yessentuki no. 4" qazli. 0.45l şüşə',               
]
qarpiz_ve_qarpiz_toxumlari = ['qarpız toxumu', 'qarpiz', 'qarpız']
gavali_ve_gavalidan_hazirlanan_mehsullar = ['gavalı', 'qara gavalı qurusu', 'gavalı sousu', 'gavali qurusu',
        'gavali', 'qurudulmuş qara gavalı (tumsuz)', 'gavalı tingi',
        'gavalı qurusu', 'qara gavali',
        'qara gavalı qurusu 500 gramlıq qablarda', 'qara gavalı']
sogan_ve_sogandan_hazirlanan_mehsullar = ['sogan kever (prasa)', 'bas sogan'
'yaşıl soğan', 'soğan', 'gül soğanaqları', 'soğan halqası',
        'soğan toxumu', 'soğan şalot', 'doğranılmış və qurudulmuş soğan',
        'qurudulmuş soğan halqası', 'təzə soğan', 'ərzaqliq soğan',
        'baş soğan', 'gül soğanağı', 'qurudulmuş soğan']
arpa_ve_arpa_mehsullari = ["arpa",
           'arpa yemlik', 'arpa yarması', 'arpa toxumu', 'arpa səməni̇si̇',
        'arpa lopası', 'arpa səməni̇si̇ (solod)', 'arpa bitkisi',
        'müxtəlif növ arpa lopaları', 'arpa səmənisi (solod)',
        'ərzaqlıq arpa', 'müxtəlif növ arpa lopaları', 'ərzaqliq arpa'

]

bfmqə = ['bioloji fəallığa malik qida əlavəsi',
        'biolojifəalliğa malik qidaəlavələri'
        ]
saftali_saftalidan_hazirlanan_mehsullar = [
'forever aloe bits n peaches/ şaftalili aloe şirəsi, məhlul 1000ml',
'şaftalı', 'şaftalı (yastı)', 'şaftalı nektarin', 
#'şaftalı çiligi',
'şaftalı şirəsi', 
#'şaftalı tingi', 
'şaftalı püresi konsentratı',
'desert siropu şaftalı', 'şaftalı qurusu', 'şaftalı püresi',
'forever aloe bi̇ts n peaches/şaftali aloe şi̇rəsi̇, məhlul 1000ml'  
]
armud_ve_armuddan_hazirlanan_mehsullar = [
        'armud', 'qurudulmuş armud',
        'armudlu limomad 0.5l 1x12',
        # 'zakaro li̇monadlari(armud,terxum,üzüm) 0.5lt şüşə',
        'armud dilimləri qurusu',
]
findiq_ve_findiq_mehsullari = [
        "findiq",
        'findiq ləpəsi', 'findiq qabiqli',
        'findiq ləpəsi qovrulmuş doğranmiş',
        'qovrulmuş ağardilmiş findiq ləpəsi', 'findiq əzməsi',
        'ağardilmiş bölünmüş findiq ləpəsi', 'findiq ləpəsi qovrulmuş',
        'şokoladli və findiqli krem', 'findiq kremi',
        'kakaolu findiq əzməsi', '5kq kakaolu findiq kremi 2li̇',
        'findiqli krem',
        'qabıqlı fındıq', 'qovrulmuş fındıq', 'fındıq unu',
        'fındıq ləpəsi qovrulmuş, ağardılmış', 'bölünmüş fındıq ləpəsi',
        'fındıq', 'fındıq pastası', 'ağardılmış fındıq ləpəsi',
        #'fındıq tingi', 
        'şokolad fındıqlı', 'fındıq ləpəsi',
        'fındıq ləpəsi qovrulmuş', 'i̇nkişaf etməmiş fındıq',
        'd+w la comt erdnusscreme crunchy nişanlı fındıq əzməs',
        'edeka wasabi asia nuts nişanlı fındıq',
        'südlü şokolad roşen fındıqlı nuqa',
        'qarışıq püstəli,badamlı,şokoladlı fındıqlı lokumlar',
        'doğranmış fındıq', 'kakaolu fındıq yağı'
]
nar_ve_nardan_hazirlanan_mehsullar = ['nar', 'nar sousu', 'nar şirəsi', 
       # 'nar tingi', 
       'nar tumu', 'nar sousu (şəkərsiz)', 'nar suyu']

gilas_ve_ondan_hazirlanan_mehsullar = ['mürəbbə qoz ilə birlikdə ağ gilas', 'gilas',
        'qozlu ağ gilas mürəbbəsi', 'gilas çilingləri', 'gilas şirəsi',
        # 'gilas tingi', 
         'konservləşdirilmiş meyvə (gilas)']
yaxma = ['yaxma', 'boğaz yaxmasi', 'burun yaxmasi']
nektarin = ['nektarin','nektarin (yastı)']
pambiq_lifi_ve_pambiga_aid_numuneler = ['pambıq lifi', 'pambiq çiyidinin cecəsi', 'pambiq çiyidi',
        'pambiq yağı', 'pambiq linti',
        'pambıq liflərinin tullantıları (əyirmə tullantıları və didilmiş xammal daxil olmaqla',
        'pambıq ipliyi', 'pambiq tiftiyi', 'pambiq lifinin tullantisi',
        'pambıq toxumu', 'pambıq uluxu', 'pambıq lifinin tullantısı']

qan = ['qan', 'quş qanı', 'qan yaxmasi']
bugda_unu_mehsullari = ['4-cü sinif buğda', 'buğda yarması', '4 cu sinif erzaqliq bugda', '4-cü si̇ni̇f  ərzaqlıq buğda', 'ərzaqliq buğda 4-cü si̇ni̇f', 
         'bugda 4-cü sinif', '4-cü si̇ni̇f buğda', '4 sınıf ərzaqlı buğda', 'buğda irmiği', 'buğda kəpəyi', 'ərzaqlıq buğda 4 cü sinif', 
         'ərzaqlıq buğda 4-cü sinif', '4-cü si̇ni̇f ərzaqliq buğda', 'ərzaqlıq buğda 3 növ', 'ərzaqlıq buğda 3 sinif', 'buğday unu', 
         'ərzaqlıq yumşaq buğda 3-cü sinif', 'koelln bluetenz flocken nişanlı buğda yulafı', '3-cü si̇ni̇f ərzaqliq buğda', '4 cu sinif ərzaqlı buğda', 
         'ərzaqliq bugda 3 cü sinif', 'ərzaqliq buğda 3 sinif', 'buğda ni̇şastasi 200 qr', 'buğda 3-cü sinif', '3-cü sinif ərzaqliq buğda', 
         'ərzaqliq buğda 4-cü sinif', '4 sinif ərzaqlıq buğda', 'buğda nişatası', 'ərzaqlıq buğda 3-cü sinif', 'buğda bitkisi', 'buğda nişastası', 
         'ərzaqliq buğda 3-cü si̇ni̇f', 'buğda lopası', 'diamant mehl extra t405 nişanlı buğda unu', 'ərzaqliq buğda 4 ci sinif', 'ərzaqliq buğda 4 si̇ni̇f', 
         'buğda', 'ərzaqliq buğda 3 cü sinif', 'ərzaqlıq bugda 4 sinif', 'ərzaqliq buğda 4 sinif', 'buğda unu', 'ərzaqlıq buğda', 'ərzaqliq bugda 4 ci sinif',
         '1 kg-lıq paketlərdə buğda unu', 'buğda yarmasi', 'ərzaqliq buğda 3-cü sinif', '3-cü sinif ərzaqlıq buğda', 'ərzaqlıq yumşaq buğda 4-cü sinif', 
         'buğda toxumu', '3-cu sinif ərzaqliq buğda', 'bugda 4-cu sinif', 'bugda 3-cu sinif', 'əla növ buğda unu', 'buğa toxumu', '3-cü sinif buğda', 
         'yumşaq buğda və meslin', 'ərzaqliq buğda', 'ərzaqlıq buğda 4 sinif', 'ərzaqlı buğda 3 sinif', 'ərzaqlı buğda 4 sinif', 'ərzaqliq buğda 4ci sinif', 
         'ərzaqlıq buğda 3 cü sinif', '4-cü sinif ərzaqlıq buğda', 'buğda rüşeymi']
pomidor_ve_pomidor_mehsullari_pomidorculuga_aid_mehsullar = ['pomidor','pomidor qurusu', 'turşu (pomidor)', 'konservləşdirilmiş pomidor',
           'təzə pomidor', 'soyulmuş pomidor','təzə pomidor', 'soyulmuş pomidor', 'pomidor şirəsi', "pomi̇dor sousu"
           'pomidor tomatı', 'qabıqlı pomidor', 'pomidor pastasi', 'doyca  doğranmış pomidor', "pomidor bitkisi yarpaqları",
           'pomidor şitili', 'pomidor toxumu', 'alnatura bio passata nişanlı konservləşdirilmiş tomat',
           
           'tomat pastası',
       
        'tomat  sousu', 'tomat', 'tomat ( marinə edilmiş)', 'pomidor tomatı',
        'təzə və ya soyudulmuş tomat', 'şirinbibər tomatı',
        'tomat pastasi̇', 'tomat pastasi', 'tomat siresi', 'tomat sousu',
        'alnatura bio passata nişanlı konservləşdirilmiş tomat',
        'tomat pastası 7 qradok 270 qr',
        'bioz. bio kids tomatenketchup nişanlı ketçup',
        'tomat sousu "mexico"',
        'wellness quru zülal şorbasi tomat və bazi̇li̇k',
        'doyca tomat pastası',
        'akfa tomat pastasi', 'tomat pastası 480 qr'

           ]
alma_ve_sire_pure_sok = ['alma', 'alma şirəsi', 'alma püresi', 'alma sirkəsi 220 qr', 'albalı alma nektarı',
                         'pfanner yaşil alma 40%',
        'swell  alma  0.25 l', 'g+g apfelmus nişanlı alma əzməsi',
        'birinci növ 100% alma və kök şirəsi',
        'birinci növ 100% alma şirəsi', '4,7 kg apple jam - alma cemi',
        'püre alma qaymaqli','happylife organik alma sirkəsi', 'alma şi̇rəsi̇',
        'alma kiseli tu 200qr 1*32']
kartof_ve_mehsullari = ['ərzaqlıq kartof', 'dondurulmuş kartof fri', 'toxumluq kartof',
        'təzə kartof', 'kartof çipsisi', 'emal olunmuş kartof fri',
        'i̇sti emal olunmuş hər hansısa bir üsulla hazırlanmamış kartof',
        'şirin kartof', 'kartof nişastası', 'dondurulmuş kartof fri̇',
        'dondurulmuş kartof', 'kartof',
        '"arvan" nişanlı dondurulmuş kartof', 'ərzaqliq kartof',
        'kartof çipsi', 'kartofdan hazirlanan ni̇şasta 1 ci̇ sort',
        'kartof ni̇şastasi', 'aci biber dadli kartof cipsi 18 qr',
        'kartof çipsi paprika dadi ile', 'kartof nişastasi',
        'dondurulmu kartofel', 'əzaqlıq kartof', 'qurudulmuş kartof']
xurma_mehsullari = ['xurma', 'qurudulmuş xurma dilimləri', 'qurudulmuş xurma',
        'medjool xurması', 'i̇ran xurmasi', 'iran xurması', 'xurma  tingi',
        'xurma dusabi', 'yarımquru xurma', 'quru xurma (korolyok)',
        'xurma doşabi', 'kluth datteln getrocknet entsteint nişanlı xurma',
        'xurma qurusu', 'i̇ki̇nci̇ növ xurma doşabi']


### Apply metodu

In [ ]:

df['mehsul'] = df['mehsul'].apply(lambda x: "Meşə materialı (taxta) və bəzək dekorativ ağacları" if x in mese_materiali_taxta else x)
# df['mehsul'] = df['mehsul'].apply(lambda x: "Kərə yağı" if x in kere_yagi else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Hazır yeməklər" if x in hazir_yemekler else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Keşniş və toxumları" if x in kesnis_ve_toxumlari else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Üzüm, üzümdən hazırlanan məhsullar və üzüməçülüyə aid nümunələr" if x in uzum_ve_uzumden_hazırlanan_mehsullar_ve_uzume_aid_numuneler else x)

df['mehsul'] = df['mehsul'].apply(lambda x: "Patoloji material" if x in patoloji_material else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Sular (adi, qazlı, müalicəvi və mineral sular)" if x in su_ve_mineral_sular else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Qarpız və toxumları" if x in qarpiz_ve_qarpiz_toxumlari else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Gavalı və gavalıdan hazırlanan məhsullar" if x in gavali_ve_gavalidan_hazirlanan_mehsullar else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Soğan və soğandan hazırlanan məhsullar" if x in sogan_ve_sogandan_hazirlanan_mehsullar else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Arpa və arpa məhsulları" if x in arpa_ve_arpa_mehsullari else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "BFMQƏ" if x in bfmqə else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Şaftalı və şaftalıdan hazırlanan məhsullar" if x in saftali_saftalidan_hazirlanan_mehsullar else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Armud və armuddan hazırlanan məhsullar" if x in armud_ve_armuddan_hazirlanan_mehsullar else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Fındıq və fındıqdan hazırlanan məhsullar" if x in findiq_ve_findiq_mehsullari else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Nar və nardan hazırlanan məhsullar" if x in nar_ve_nardan_hazirlanan_mehsullar else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Gilas və gilasdan hazırlan məhsullar" if x in gilas_ve_ondan_hazirlanan_mehsullar else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Yaxma" if x in yaxma else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Nektarin" if x in nektarin else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Pambıq lifi və pambıqçılığa aid nümunələr" if x in pambiq_lifi_ve_pambiga_aid_numuneler else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Qan" if x in qan else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Buğda, buğda unu və digər məhsullar" if x in bugda_unu_mehsullari else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Pomidor, pomidor məhsulları və pomidorçuluğa aid nümunələr" if x in pomidor_ve_pomidor_mehsullari_pomidorculuga_aid_mehsullar else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Alma və alma məhsulları" if x in alma_ve_sire_pure_sok else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Kartof və ondan hazırlanmış məhsullar" if x in kartof_ve_mehsullari else x)
df['mehsul'] = df['mehsul'].apply(lambda x: "Xurma və ondan hazırlanan məhsullar" if x in xurma_mehsullari else x)


### Bal

In [ ]:
# see_similar("bal")
df[
    (pd.notna(df['mehsul'])) &
   #(df['mehsul'].str.contains("bal")==False) &
   (df['mehsul'].str.contains("şab")==False) &
   (df['mehsul'].str.contains("sab")==False) &
   (df['mehsul'].str.contains("alb")==False) &
   (df['mehsul'].str.contains("aq")==False) &
    (df['mehsul'].str.contains("bal")) #|
  #   (df['mehsul'].str.contains("məhlul")) |
  #  (df['mehsul'].str.contains("damcı")) |
  #  (df['mehsul'].str.contains("sprey")) |
  #  (df['mehsul'].str.contains("qəbul üçün")) |
  #  (df['mehsul'].str.contains("damci")) |
  #  (df['mehsul'].str.contains("damci")) 
  ]['mehsul'].unique()



array(['Balıq, balıq məhsulları və balıqlara aid nümunələr',
       'atılmış bala', 'təbii bal',
       'friskies (sousda quzu, qizilbaliq, ördək, dil, qaraciyər, toyuq və hinduşka ətli)',
       '“felix” iştahaçan tikələr (jeledə toyuq, qizilbaliq, quzu, alabaliq və dovşan ətli)',
       'felix sensations yaşli pişiklər üçün pomidorlu sousda (mal, ördək, baliq ətli) konsevləşdirilmiş tam rasionlu yem',
       '“friskies” (sousda quzu, qizilbaliq, ördək, dil, qaraciyər, toyuq və hinduşka ətli)',
       'feli̇x i̇ştahaçan tikələr(jeledə toyuq, qizilbaliq, quzu, alabaliq və dovşan ətli)',
       'bal şirəsi',
       'aloe blossom herbal tea / aloe çiçəklərindən çay, filtr-paket №25',
       'kitekat baliqcinin ovu,i̇ştahacan dana əti̇,ət bayrami və i̇ştahacan toyuq əti̇ keşi̇dli̇ yetki̇n pi̇şi̇klər üçün yem',
       'çistotel maksimum xalta balaca it və pişik üçün',
       'çistotel qlistoqon suspenziya balaca it üçün',
       'çistotel qlistoqon suspenziya balaca it və pişik üçün',
    

In [ ]:
bal = [
 'təbi̇i̇ bal', "təbii bal",'suni̇ bal shi̇rasi̇', 'suni bal', 'suni carazli bal','alnatura bio waldhonig nişanlı bal',
  'langn ausl sommerblueten honig nişanlı bal' , 'bal şirəsi', "bal", 
]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Bal" if x in bal else x)

### Sud ve sud mehsullari

In [ ]:
# see_similar("bal")
df[
    (pd.notna(df['mehsul'])) &
   #(df['mehsul'].str.contains("bal")==False) &
   (df['mehsul'].str.contains("şab")==False) &
   (df['mehsul'].str.contains("sab")==False) &
   (df['mehsul'].str.contains("alb")==False) &
   (df['mehsul'].str.contains("aq")==False) &
    (df['mehsul'].str.contains("bal")) #|
  #   (df['mehsul'].str.contains("məhlul")) |
  #  (df['mehsul'].str.contains("damcı")) |
  #  (df['mehsul'].str.contains("sprey")) |
  #  (df['mehsul'].str.contains("qəbul üçün")) |
  #  (df['mehsul'].str.contains("damci")) |
  #  (df['mehsul'].str.contains("damci")) 
  ]['mehsul'].unique()



array(['Balıq, balıq məhsulları və balıqlara aid nümunələr',
       'atılmış bala',
       'friskies (sousda quzu, qizilbaliq, ördək, dil, qaraciyər, toyuq və hinduşka ətli)',
       '“felix” iştahaçan tikələr (jeledə toyuq, qizilbaliq, quzu, alabaliq və dovşan ətli)',
       'felix sensations yaşli pişiklər üçün pomidorlu sousda (mal, ördək, baliq ətli) konsevləşdirilmiş tam rasionlu yem',
       '“friskies” (sousda quzu, qizilbaliq, ördək, dil, qaraciyər, toyuq və hinduşka ətli)',
       'feli̇x i̇ştahaçan tikələr(jeledə toyuq, qizilbaliq, quzu, alabaliq və dovşan ətli)',
       'aloe blossom herbal tea / aloe çiçəklərindən çay, filtr-paket №25',
       'kitekat baliqcinin ovu,i̇ştahacan dana əti̇,ət bayrami və i̇ştahacan toyuq əti̇ keşi̇dli̇ yetki̇n pi̇şi̇klər üçün yem',
       'çistotel maksimum xalta balaca it və pişik üçün',
       'çistotel qlistoqon suspenziya balaca it üçün',
       'çistotel qlistoqon suspenziya balaca it və pişik üçün',
       'balqabağ toxumu', 'otaci herba

In [ ]:
see_similar('pudinq')

(6, array(['pudinq', 'çudo nişanlı pudinq', 'südlü şokoladlı pudinqlər',
        'kakaolu pudinq', 'vani̇lli̇ pudinq 125 qr',
        'pudinqlər,südlü kokteylər,südlü içkilər grand desert, grand duet, grand cafe,grand cocktail'],
       dtype=object))

In [ ]:
sud = [
       
       'kere yagi', 'kərə yağı', 'kərə yağı korovuşka 500 qr',
        'kərə yağı gerda 250 qr', 'kərə yağı french cow 1 kq',
        'svalya nişanlı kərə yağı 82%',
        'exporta  butter 82,5 % - kərə yağı', 'kərə yağı 1000 gr',
        '“krestyanskoe" nişanlı kərə yağı', 'kərə yağı 82.5%',
        'şirin kərə yağı', 'kərə yağı naş moloçnik',
        'kərə yağı gerda 2.5 kq',

        'kərə yağı', 'kərə bitki yaği',
        'kərə yaği və yaxilabilən südməhsullari', 
        'kərə yaği 1 kq', '“belaya korova” ni̇şanli kərə yaği',
 'kərə yağı korovuşka 500 qr',
        'süd məhsulları (kərə yaği)', 'kərə yağı gerda 250 qr',
        'kərə yağı french cow 1 kq', 'kərə yaği',
        'emborg ni̇şanli 82% kərə yağ', 'svalya nişanlı kərə yağı 82%',
        '” tulçi̇nskaya” adli  bi̇tki̇-kərə tərkibli yağ(smes)',
        'fi̇n kərə yağ 79%, 500qr',
        'vali̇o kərə  yaği 82%, 500 q lakti̇k duzuz',
        'exporta  butter 82,5 % - kərə yağı',
        'kərə yaği golden crown 180 qr', 'neotame şəkərəvəzedici',
        'vestalia kərə dadlı peçenye 0.250 kq',
        'süd məhsullari (kərə yağ)', 'kərəyağı', 'kərə yağı 1000 gr',
 '“krestyanskoe" nişanlı kərə yağı', 'kərə yağı 82.5%',
        'şirin kərə yağı', 'kərə yağı naş moloçnik',
        '"brest-li̇tovsk" duzsuz kərə yaği əla növ 450 qr qablaşdirmada quru maddədə yağin kütlə payi 82.5%',
        'digər kərəviz, kök kərəvizdən başqa', 'kərə yağı gerda 2.5 kq',
        'kərə yaği west gold 400 qr', 'əri̇di̇lmi̇ş kərə yaği 99%',
       
       'pendir', 'ərinmiş pendir', 'bərk pendir', 'şokoladlı pendir',
        'saçaqlı pendir', 'krem pendir', 'pendir sousu',
        'emmental pendir',
        'ceddar pendir', 
        'muratbey surmeli pendir 400 gr 1x16',
        'ağ pendirlər arla üç i̇nək',
        'pendir "nemuno lanku" 45% yağlılıq çəki',
        'süd məhsulları ( pendir )', 'pendir mozzarella blok 200 gr',
        'pendirlər', 'friendship nişanlı qauda pendiri',
        'friendship nishanli çedar pendiri 1033', 'süd məhsulları  pendir',
        '"svalya" nişanli hisə verilmiş pendirlər',
        '"svalya" nişanli pendirlər',
        'fetahi white cheese -fetahi yunan salatı ücün pendir',
        'yarim bərk pendir "brest-litovsk qollandskiy" quru maddədə yağin kütlə payi   45%    paketlənmiş (doğranmiş- kubik) 200 q',
        'hellim pendiri', 'tulum pendiri', 'royal klassik ağ pendir',
        'mozarella pendiri',
        'bərk pendir "dziugas"12/18/24 ay dəymə müddəti',
        'oldenburger pendir gouda 48%', 'oldenburger pendir tilzit 45%',
        'emmental pendiri',
        '" rokiskio slivoçnıy " yari bərk pendir 48 %yağliliq',
        'pendir (kaşar)', 'pendir (tulum)', 'berk cavan pendir',
        'pendir bərk', 'alima nişanlı krem pendir', 'feta pendiri 4kq',
        'fransız pendiri salat üçün bitki yağı ilə',
        'sarımsaq və zeytun dadlı pendir tənəkə yarpağına bükülmüş',
        'pendir rikotta dınya 125qr',
        'yarim bərk pendir "rossiyskiy molodoy" quru maddədə yağin kütlə payi 50%',
        'favita pendir 18%', 'ağ pendirlər üç inək', 'yetişməmiş pendir',
        'viola 200 q cantharelle göbələkli ərgin pendir',
        'əridilmiş pendirlər hochland', '"çerniy prints" adli pendir',
        'pizza pendiri',
       
 'pudinq', 'çudo nişanlı pudinq', 'kakaolu pudinq',
        'vani̇lli̇ pudinq 125 qr',
'yoqurt', 'ehrmann yoqurtlar alpenland, mama lama,ermiqurt',
        'içməli yoqurt mama lama', 'ehrmann yoqurtlar alpenland,',
        '"svalya" nişanlı yunan yoqurtu, meyvəli 150gr',
        'baltais eko organic yogurt without additives - eko yoqurt sadə',
        'çudo nişanlı yoqurt', 'yoqurt "zemaityos"',
        'ehrmann yoqurtlar mama lama,alpenland',
        'ehrmann yoqurtlar alpenland', 'yoqurtlar alpenland',
        'yoqurtlar alpenland və ermiqurt',
        'yoqurtlar alpenland  mam lama ,ermiqurt',

  'ayran',

  'kəsmi̇k', 'çudo ni̇şanli kəsmi̇k', 'kəsmi̇k 5%',
        'domi̇k v derevne ni̇şanli kəsmi̇k',
       'kəsmik şirin',
        'kəsmik', 'kəsmikli desert', 'şirin kəsmik',
        'yoqurt,yumuşaq kəsmik', 'kəsmik,kəsmi̇kli̇ desert',
        '"svalya" nişanlı kəsmik və kəsmik tərkibli məhsullar',
        'exporta skimmed milk curd 0,5%  - kəsmik yağsız',
        'kəsmik "savuskin xutarok" 9% yağliliqda multivak klinok 220 qr',
        'kəsmik 9% yağ./yağsız kəsmik 0,5% yağ',
        '"belaya korova" nişanlı kəsmik',
        '“çiyələk“ meyvə əlavəli kəsmik məmulatı məktəbə qədər və məktəb yaşlı uşaqların qidalanması üçün, kalsium və d vitamini ilə  zənginləşdirilmiş, 3,5% yağlılıqda  p / st 110 q',
        'kəsmik "savuskin xutarok" 9% yağliliqda multivak silindr 300 qr',
       
       'xama', 'xama 20 %-li', 'xama 15 %-li', 'xama 25 %-li',
        '“babulinı produktı” nişanlı xama', 'ehrmann smetanovna xama',
        'domik v derevne nişanli xama', '"svalya" nişanli xama',
        'exporta sour cream 25% - xama',
        'xama "brest- litovsk" 20% yağliliqda 315 qr', 'xama məhsulu',
        'xama 30% yağ.', 'xama 26%', 'xama 18%', 'mlekovita xama 18%',
        'xama ehrman smetanovna',
       
       'sırok', '"svalya" nişanlı sırok (kəsmikli desert) 40gr',
        '23 % yağlılıqda şirələnmiş kəsmikli sırok “trufel” əlavəli .',
       '23 % yağliliqda şirələnmiş kəsmikli sirok “korovka” əlavəli .',
        'çudo ni̇şanli sirok',
'kefir', 'exporta kefir 3,8 % - 4,3 % - kefir',
        'laktuloza və probiotiklə zənginləşdirilmiş 1.5 % yağlılıqda kefir pet 415 qr',
        'kefir 0.5%/ 2,5% yağ.',
  'şor',
  'krem qaymaq', 'pasterizə edilmiş qaymaq', 'qaymaq',
        'pasterizə qaymaq', 'sterilizə edilmiş qaymaq',
        'xama və qaymaq istehsalında istifadə olunan duz', 'qaymaq tozu',
        'qaymaqli̇ krem', 'i̇çməli̇ qaymaq',
        'domik v derevne nişanli qaymaq', 'qaymaq 15%',
        'baerenmarke die ergiebige 10% nişanlı qaymaq', 'duru qaymaq',
        'domi̇k v derevne ni̇şanli qaymaq', 'qaymaqlar', 'kokoslu qaymaq',
  'qatıq',
'süd', 'qatılaşdırılmış süd', 'pasterizə edilmiş süd', 'çiy süd',
        'quru süd zərdabı',
        'yağsiz süd konsatresi̇', 'bişmiş qatlaşdırılmış süd',
        'quru süd (yağlı)', 'südlü sosiska', 'soya südü', 'südlü içki',
        'quru süd', 'südlü kokteyl', 'pasterizə süd', 'kokos südü',
        'aromalı süd', 'süd qarişiği',
        'qatilaşdirilmiş südvə süd məhsullari',
        'quru süd (yağsız)', 'süd piyi əvəzedicisi', 'süd zərdabı',
        'sterilizə edilmiş süd', 
        'qatlaşdirilmiş süd', 'bambey səhər yeməyi, süd aromalı',
        'quru süd qarişiği (uşaq qi̇dasi)',
        '"standart" sortlu yağsiz quru süd', 'südlü draje şokoladlar',
        'nutrilak premium +3; quru süd; süd içkisi',
        'nutrilak 3; quru süd; süd içkisi',
        'nutrilak premium +1; quru süd; uyğunlaşdırılmış; ilkin süd;',
        'nenni 1 uyğunlaşdırılmış  quru süd içkisi, keçi südü əsasında;  yenidoğulmuş uşaqlar üçün; 800 gr',
        'similak gold 1; 0-6 aylıq uşaqlar üçün süd',
        'qatilaşdirilmiş süd', 'covap cabra  qoyu südü 1.6% yağliliq',
        'südlü şokolad roşen badam',
        '"svalya" nişanli südlü içkilər  ciyələk',
        '"i̇çnya" nişanli qatilaşdirilmiş süd şəkər i̇lə',
        'qatılaşıdırılmış süd', 'süd denge laktozsuz 1 lt 1x12',
        'şokolad südlü lacmi', 'şokolad südlü poristiy roşen',
       
        'südlü içki kefir', 'badam südü', 'südlü şokoladlı pudinqlər',
        'süd yağının əvəzedicisi', 'qida əlavəsi süd istehsalında', 'südlü içki ryajenka',
        'süd məhsulları ( yağ )', 'süd məhsulları  xamalı məhsul',
       
        'vinni uşaq sıyığı; südsüz; yeddi dənli; prebiotikli',
        'nutrilak 1; quru süd; uyğunlaşdırılmış; ilkin süd',
        'nutrilak 2; quru süd; uyğunlaşdırılmış; davam südü',
        '”i̇çnya” nişanlı qatılaşdırılmış süd şəkər ilə',
        '”i̇çnya” nişanli qatilaşdirilmiş süd şəkər ilə',
        'aquşa nişanlı süd', 'domi̇k v derevne ni̇sanli süd',
        'süd məhsullari (spred yaği)', 
       
        'süd məhsullari ( yağ)', 'südlü düyü siyiği',
        '"riyajinka" 3.2% yağlılıqda turş süd məhsulu pet 420',
        'ultra pasterizə edilmiş 1.5% yağliliqda süd tba 1ll',
        'arla südlü kokteylər', 'şokolad südlü lacmi sezam',
        'qatılaşdırılmış süd sssr və squşonka',
        'krem-desert qatilaşdirilmiş süd "fitelle" 100qr',
        'similak gold 2; 6-12 ayliq uşaqlar üçün süd',
        'similak gold 3; 12 aydan sonra uşaqlar üçün süd',
        'similak gold 4; 18 aydan yuxari uşaqlar üçün süd',
       
        'quru süd; uyğunlaşdırılmış; keçi südü əsasında;12 aydan yuxarı uşaqlar üçün; mamako-3 premium',
        'sıyığ; 7 dənli; giləmeyvəli; keçi südü əsasında',
        'quru süd; uyğunlaşdırılmış; keçi südü əsasında; 0-6 aylıq uşaqlar üçün; mamako-1 premium',
        'sıyığ; buğda armud və banan ilə; prebiotikli; keçi südü əsasında',
        'similak aimentum; süd', 'similak komfort 2; süd',
        'similak azlaktozalı; süd', 'similak hipoallergen 2; süd',
        'similak  hipoallergen 1; süd; 0-6 aylıq uşaqlar üçün süd',
        'süd yumurta qarişığı', 'landl h-landmilch 3,8% nişanlı süd',
        'fi̇todar i̇sti südlü şokolad',
        'qənnadi məmulatlarinin istehsalı üçün süd yağının əvəzedicisı-ekolakt 1403-35 ko',
        'südlü şokolad roşen qaragilə nuqa',
        'tərkibində süd yağları, saxaroza, izoqlükoza, qlükoza və ya nişasta olmayan və ya tərkibində 1,5 kütlə %-dən az süd yağı, 5 kütlə %-dən az süd yağı, 5 kütlə % -dən az saxaroza və ya izoqlükoza, 5 kütlə %-dən az qlükoza və ya nişasta olan',
        'süd 3.2%', 'südlü siyiq',
       
        'qatılaşdırılmış süd slobajanskaya və ekstra',
        'südlü içki prostokvaşa', 'uht süd 1,5% laktozsuz süd',
        'pudinqlər,südlü kokteylər,südlü içkilər grand desert, grand duet, grand cafe,grand cocktail',
        'krem desert  qatılaşdırılmış süd', 'süd tam yağli 500 ml',
        'keçi̇ südündən ağ pendi̇r',
        'nutrilak premium ; anireflüks; quru süd', 'yağsız süd tozu',
        'yağsiz süd tozu', 'yağsiz quru süd',
        'nutrilak premium +2; quru süd; uyğunlaşdırılmış; davam südü;',
        'domik v derevne nişanlı süd',
        'ultra pasterizə edilmiş 1.5% yağliliqda  süd  tba 1lt'       


'qati sud 600qr pet 1x12', 
        'quru sud tozu'       
]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Süd və süd məhsulları (kərə yağı, süd, pendir, qatıq, qaymaq, şor, kefir, sırok, kəsmik, yoqurt, ayran, pudinq)" if x in sud else x)

### Et Kolbasa konser

In [ ]:
searchfor = ['olbas', 'osis', 'ardel', "etçina", "etcina"]
mehsul_not_na = df[pd.notna(df['Məhsul'])]
len(mehsul_not_na[mehsul_not_na['Məhsul'].str.contains("|".join(searchfor))])

1675

In [ ]:
see_similar('doktor'.lower())

(2, array(['cerkiz var doktorskaya gubernskaya 0.5 kq kolbasa',
        'cerkiz var doktorskaya 1.6 kolbasa'], dtype=object))

In [ ]:
et_ve_et_mehsullari_mal_toyuq_ordek_kolbasa_sosiler = [
'”pro plan nutrisavour”(sousda mal əti ilə)',
 'quzu əti',
 'cerkiz servelat astoriya 0.6 kq',
 'salame nostranello g500 atm crtx11 ngr - hissə verilmiş kolbasa',
 'vetçina donuz ətindən',
 'toyuq əti̇ndən kabanos',
 'cerkiz var doktorskaya gubernskaya 0.5 kq kolbasa',
 'mal əti konservi 525',
 'çerki̇z var russkaya gubernskaya 1 kq kolbasa',
 'ət məhsullari ( kolbasa)',
 'file acılı',
 'ostankino var fileynaya  kolbasa',
 'dondurulmuş hindquşu farşı',
 'sosiska  üçün suni qabıq',
 'farş',
 'toyuq əti yarımfabrikatı',
 'dondurulmuş toyuq',
 'mal əti konservi 290 qr',
 'sosis  mal ətindən',
 'toyuq əti yarımfbri̇ktı',
 'sosis orqinal quş ətindən',
 'cerkiz var russkaya po çerkizovski 1.6 kq kolbasa',
 'bişmış mal ətindən sosiska',
 'wurstel golosino multipack 100x5 x12 ngr - sosiska',
 'dondurulumuş toyuq budu',
 'cerkiz var moloçnaya po çerkizovski 1 kq kolbasa',
 'kolbasa yarım-hisə verilmiş',
 'çolpa əti',
 'qoyun əti konservi 290 qr',
 'cerkiz vk imperskaya  kolbasa',
 'toyuq əti̇ndən pendi̇r əlavəli̇ kabanos',
 'donuz əti yarimfabrikatlari',
 'dondurulmuş mal əti',
 'cerkiz var doktorskaya 1.6 kolbasa',
 'wurstel go0g p.s. crtx15 ngr - sosiska',
 'kolbasa pendirli',
 'donuz əti məhsulları',
 'kolbasa (salam)',
 'pro plan quru tam rasionlu iri cinsli it balalari üçün, böyük miqdarda toyuq əti ilə. həmçinin süd verən və hamilə itlər üçündə məsləhət görülür.',
 'donuz əti məhsulu',
 '“ pro plan veterinary diets” (toyuq əti ilə)',
 'qaymaqlı sosiska',
 'kolbasa bişmiş və hisə verilmiş',
 'kamar tavuk etli salam kolbasa',
 'toyuq əti',
 'mal ətindən toppinq',
 'dondurulmuş toyuq əti̇',
 'kolbasa hisə verilmiş',
 'friskies (sousda mal, quzu və toyuq ətli tikələr)',
 '“pedigree” it balalari üçün sousda mal və quzu əti ilə',
 'dondurulmuş telapiya filesi',
 'çiy qurudulmuş kolbasalar',
 'panqasius file',
 'purina one (yüksək tərki̇bli̇ toyuq ətli̇ və bütün taxil i̇lə)',
 'donuzun qol əti',
 'cerkiz servelat stolicniy',
 '”pro plan” (toyuq əti ilə)',
 'mal əti̇',
 'whiskas toyuq,quzu ətli raqu',
 'quş əti kolbasa.sosislər',
 'kolbasa bişmiş',
 'ətotu',
 'doğranmış peperoni kolbasaları',
 'toyuq əti̇ndən kolbasa',
 'kolbasa (hind quşu əti)',
 'sosis',
 'mal əti (əla növ)',
 'donuz əti konservi 338 qr.',
 'sosiska mal ətindən',
 'optimealtm miniatyür və kiçik cinsli küçüklər üçün konservləşdirilmiş dənli bitkisiz tam rasion yem dana və cücə əti ilə souslu',
 'mal əti̇ hi̇ssələri̇',
 'donuz əti konserv',
 'yarım-hisə verilmiş kolbasa',
 'toyuq',
 'bi̇şmi̇ş kolbasalar',
 'ördək əti',
 'ostanki̇no gri̇l-master 0,45 kq sosi̇sləri̇',
 'pediqri jiledə quzu əti̇ və ya dana əti̇ və ya qara ci̇yər i̇lə yetki̇n i̇tlər üçün',
 'mal əti yarımfabrikatları',
 'soyudulmuş mal əti',
 'konservlesdirilmis ət',
 'bişmiş kolbasa',
 'dondurulmuş toyuq farşı',
 'sal.ungherese atm gr250 cal.crtx16 ngr - hissə verilmiş kolbasa',
 'mal farşı',
 'dondurulmuş mal ətinin farşı',
 'çiy dönər (mal əti)',
 'servelat',
 'kolbasa lyubitelskaya prima',
 'g+g deli schinkenwuerstchen nişanlı sosiska',
 'toyuq əti yarımfabrikatları',
 'ət',
 'bişmiş sosislər”student” çəki',
 'kolbasa krakouskaya',
 '”friskies” (ət, toyuq əti və faydali tərəvəzlərlə)',
 'qəlyanaltılıq ət çubuqları',
 'file acısız',
 'mal əti konservası',
 'sosi̇ski̇ eli̇tni̇ye s si̇rom',
 'kətən toxumu',
 'toyuq əti̇ hi̇ssələri̇',
 'sosis mal ətindən',
 'toyuq filesi',
 'mal ətindən qəssab kolbasa',
 'donuz əti̇ndən kabanos',
 'gourmet perle sousda yumuşaq file( toyuq əti ilə)',
 'hind quşu farşı',
 'toyuq sosisi',
 '“pro plan” (quzu əti ilə)',
 'mal əti konservləri',
 'ət məhsulu(donuz əti)',
 'kolbasa,sosislər',
 'donuzun sinə əti',
 'dondurulmuş sümüksüz mal əti( tərkibində bonfile yoxdur)(2 ci sort)',
 'kolbasa',
 'cücə',
 'kolbasa üzlüyü',
 'dondurulmuş toyuq sosi̇sləri̇',
 'sosiska',
 'ət aromalı bulyon',
 'mal əti yarımfabrikatı',
 'mal ətindən sardelka',
 'dondurulmuş mal farşı',
 'hisə verilmiş toyuq hissələri',
 '”dog chow” (quzu əti ilə)',
 'bratwurst sosi̇ska',
 'kolbasa və sosiska məhsullari qarışıq( mal əti donuz əti)',
 'cerkiz servelat imperskaya  0.5 kq',
 'dana əti',
 'donuz əti',
 'toyuq əti̇ yarimfabri̇kati',
 'cerkiz var russkaya gubernskaya 0.5 kq kolbasa',
 'sals. napoli picc.atm g250 cal.ctx12 ngr - hissə verilmiş kolbasa',
 'kolbasa yarı hisə verilmiş',
 'mal əti̇ yarimfabri̇kati',
 'kolbasa və sosiska istehsali üçün süni qabiqlar',
 'dondurulmuş toyuq filesi',
 'bişmiş və hisə verilmiş kolbasa',
 'çerki̇z var moloçnaya  po çerki̇zovski̇ 0.4 kq kolbasa',
 'darling” (ətli və tərəvəz əlavəsi ilə)-',
 'bi̇şmi̇ş donuz əti̇ndən kolbasa',
 'dlya druqa (itlər üçün, mal ətli, tam rasionlu quru yem)',
 'bişmiş-hisə verilmiş kolbasa utrennay',
 'əsas yemək ətli',
 'çerkiz moloçniye 0.45 kq sosisləri',
 'sosis  quş ətindən',
 'vetçina',
 'bişmi̇ş kolbasa klassi̇k kanqal 225 qr',
 'mal əti',
 'kolbasa və sosiska istehsali üçün süni qabıqlar',
 'kətan toxumu tane 250 gr',
 'toyuq kolbasa 500gr',
 'soyudulmuş toyuq əti',
 'qoyun əti',
 'frankfurter sosi̇ska',
 'file (acısız strips)',
 'mal əti konservi',
 'gourmet perle sousda yumuşaq file( mal əti ilə)',
 'sosislər',
 'donuzun sümükdən ayrılmış filesi'
]

et_ve_et_mehsullari_mal_toyuq_ordek_kolbasa_sosiler = list(set(et_ve_et_mehsullari_mal_toyuq_ordek_kolbasa_sosiler))

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Mal, toyuq, ördək, hind quşu əti və onlardan hazırlanan məhsullar (kolbasa, sosislər, yarımfabrikat, konservlər)" if x in et_ve_et_mehsullari_mal_toyuq_ordek_kolbasa_sosiler else x)

### Alkoqollu içki

In [ ]:
see_similar("balt".lower())

(2, array(['"balti̇ka 0" 0.47l', '"baltika 0" alkoqolsuz buğda pivəsi  0,45l'],
       dtype=object))

In [ ]:
alkoqollu_icki = ['"balti̇ka 0" 0.47l',
                  'alkoqollu içki',
'oneqin 0.7l alkoqollu içki',
        'oneqi̇n 0.5l alkoqollu içki',
        'tekila, otlar əsasında alkoqollu içkilər  aperol,espolon blanco,jose cuervo,milagro,sierra,xenta absenta',


'konyak spirtli saxlanılmış(6 illik)',
        'konyaklar martell cohi̇ba,cordon,vsop',
        'cognac vs louis royer 40% 70cl - konyak',
        'deau cognac privilege 70cl fr/ang -+/ith gift box - konyak',
        'cognac gautier vs with gift box 0.7 40% - konyak',

  'araq absolut standart,kurant,ci̇tron,mandari̇n',
        'araq stoliçnaya,stoli̇çnaya sever spetsi̇alnaya,myaqkaya',
        'araq ciroc, ketel one vodka',
        'araq absolut ci̇tron,elyx,kurant,standart',
        'zeppelin emtee nishanli araq', 'kookla vodka  premium 40 - araq',
        'vodka-moskovskaya osobaya 40% 0.7 l (nrf) nişanlı araq',
        'araq-fortuna premi̇um 0.5',
        'vodka barska premium 1.0 40% ek - araq', 'araq 0.7lt',
        'briliantovıy lyod 1 - araq', 'vodka yaric 1.0 l - araq',
        '"bear force powerful" araq',
        'belorusski̇y standart əmtəə ni̇şanli araq',
        'president vodka gumuş 0.7 nişanlı araq',

                  'sparkl.wine-bottega rose gold pinot nerd brut 11.5% 0.75l nişanlı şampan',
                  'vi̇ski̇',
        'vi̇ski̇lər aberlour 12yo ,ballantines finest,passport scotch,jameson',
        'vi̇ski̇lər jack dani̇els,balveni̇a doublewood,clan macgregor,glenfi̇ddi̇ch grants,monkey shoulder,tullamore',
'louders finest blended scotch whisky 12x70 cl botles 40% vol - viski',
        'viski', 'whiskey-tullamore dew  40% 0.7 l nişanlı viski',


'qırmızı şərab', 'ağ şərab', 'şərab', 'şərab çəhrayi',
        'şərab çəhray',
        'şərablar campo vi̇ejo,jacobs creak,brancot estate,jacobs creak classi̇c',
        'col de salici brut doc köpüklənən  ağ şərab',
        'col de sali̇ci̇ extra dry doc köpüklənən ağ şərab',
        'şərablar campo vi̇ejo cava,jacobs creek,campo vi̇ejo,domi̇ni̇o de campo,jacobs creek,',
        'ağ kəmşirin şərab', 'sarafin chardonnay ağ şərab',
        'sarafin cabernet sauvignon qırmızı şərab',
        'sarafin merlot qırmızı şərab', 'sarafin sauvignon blanc ağ şərab',
        'alazanskaya dalina krasnoye polu sladkoye - qırmızı kəmşirin şərab',
        'şərab besini rose', 'saperavi qvevri turş qırmızı şərab',
        'valpolicella classico doc 2019 - şərab',
        'pinot grigio delle v. doc 2019 12.5% alc - şərab',
        'amarone valp . cl. docg 2016 15.5% alc - şərab',
        'bellissima əmtəə nişanlı köpüklü şərab', 'qirmizi şərab',
        'şərab-prosecco doc spu ext dry anti̇ca vi̇gna orange 0,7',

                  'pasterizə edilmiş pivə', 'pasterizə edilməyən pivə',
        '"baltika 0" alkoqolsuz buğda pivəsi  0,45l',
        'alivariya nişanlı pivə', 'holsten pilsener ds dpg nişanlı pivə',
        'guinness draught ds dpg 0,44l nişanlı pivə',
        'erdinger weissbier alk-frei  nişanlı alkoqolsuz pivə',
        'erdinger weissbier hefe  nişanlı pivə'
      
      ]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Alkoqollu içkilər (pivə, şərab, araq, viski, konyak)" if x in alkoqollu_icki else x)

### Karamel konfet

In [ ]:
see_similar("sokolad")

(4, array(['sokolad qara teneke 250 gr',
        "isti sokolad choko-la' - chocolate beverage 1 kg jar",
        'səhər yeməyi amarant "di end di" tund sokolad 250qr',
        'nutella sokoladi ile doldurulmus vafli'], dtype=object))

In [ ]:
konfet_karamel_sokolad = [
'sokolad qara teneke 250 gr',
        "isti sokolad choko-la' - chocolate beverage 1 kg jar",
        'səhər yeməyi amarant "di end di" tund sokolad 250qr',                          
'karamel',
'içlikli və ya içliksiz şəkərləmə karameli',

'anata karamel', 'shirin tak karamel', 'i̇ran karameli̇',
        'vestelia karamelli peçenye 0.120 kq',
        'karamel (shoniz, chichak alton)', 'südlü karamel',
        'şokolend nişanli meyvəli karamel', 'jako nişanli meyvəli karamel',

'şokoladli oyuncaq yumurta', 'ağ kokoli̇n şokoladlari', 'şokolad',
        'sirop şokoladlı',  'ağ şokolad',
        'bi̇tter kokoli̇n şokoladlari', 'şokolad pastası',
        'şokoladli konfet', 'südlü şokolad', 'qara şokolad',
        'şokolad qlazuru', 'şokolad yaği',
        'kakao və kakao məhsullari, şokolad və şokoladməhsullari',
        'ağ şokolad tozu', 'südlü şokolad kütləsi', 'şokolad kütləsi',
        'şokoladlı toppinqlər', 'şokolad tozu', 'i̇sti şokolad',
        'şokoladlı sous', 'şokolad yağı', 'şokolad (dekorativ)',
        'kupiec nişanlı tünd şokolad ilə düyü vaflisi',
        'şokoladli və fi̇sti̇qli krem', 'şokolad siropu', 'şokolad kremi',
        'şokoladlar', 'bitter draje şokoladlar',
        'şokoladli baton roşen', 'şokolad premi̇um 240 gr',
        'sos şokoladli 350 gr 1x6', 'kokteyl şokoladlı',
        'şokoladli kruasan', 'şokoladlı konfet', 'şokoladli içki',
        'dondurma eski̇mo şokoladli', 'nutella nişanlı şokolad kremi',
        'şokolad (qlazur)', 
        'kakao - şokolad i̇stehsali', 'ağ şokoladlar',
        'peçenye arçi̇ şokolad', 'ülker cokokrem nişanli  şokoladlar',
        'oreski v qlazuri-şokolad', 'şokolad yumurta', 'çərəzli şokolad',
        'milk chokolate - şokolad', 'mini choko dark crt 9kg - şokolad',
        'şokoladlı məmulatlar', 'şokolad qlazurlu konfet',
        '"ja! " nişanlı şokolad kremi', '"reber" nişanlı  şokolad konfet',
        'vestelia şokolad qlazurlu peçenye 0.200 kq',
        'dippo para konfise (şokolad)',
        'specialty box 25 pc nişanlı şokolad',
        'specialty box 11 pc nişanlı şokolad',
        'şokolad ağ poristiy roşen karamelli', 'şokolad roşen poristiy ağ',
        'qara şokolad roşen klassik orginal', 'şokoladlı soyuq jele',
        'şokoladlı konfetlər pobeda',
        'dondurma"bolşoy pinqvin,straçatella,tsar-plombir,elitnoye,kaştan şokoladnıy,semeynoye,slivoçnoye,şokoladnoye,staçatella"',
        'qarişiq şokolad', 'şokolad sos 128 qr',
        'şokolad dəst tünd və südlü (şəkərsi̇z)', 'şokoladlı konfetlər',
        'şokolad konfet apelsin', 'şokolad konfet squşenka kokos ilə',
        'g+g pfefferminz taefelchen nişanlı şokolad',
        'pli̇tka şokolad " umni̇ye sladosti̇ " findiqli 90qr',
        'şokoladli̇ un məmulati̇', 'şokolad (maitre truffout)',
        'şokoladlı sufle', 'dash şokolad', 'i̇sti̇ şokolad',
        'qara şokolad 2,5 kg', 'kreker 2 crack şokoladlı', 'krem şokolad',
        'albali ile şokolad dadli quru qarışıq', 'şokoladli konfetlər',
        'kokolin şokolad', 'şokolad vitrini', 'şokoladlı uşaq vafliləri',
        'nurefşan nişanli kakaolu mexmix  şokoladlari 150gq',                        
                      
'konfet', 'şokoladli konfet', 'şokaladlı konfetler',
        'konfet (noğul)', 'konfet (daş)', 'konfet nuttini',
        'portagallı partlayan konfet', 'konfetlər', 'konfet djelli',
        'şokoladlı konfet', 'şəkərli qənnadı  məmulatları (konfet)',
        'konfet sotaş', 'konfet jaklin fransız zefiri çiyələk-bazilik',
        'anata konfet', '"pszczolka" nişanlı karamel konfet',
        'şokolad qlazurlu konfet', '"reber" nişanlı  şokolad konfet',
        'konfet korivka roşen', 'konfet muds',
        'şokoladlı konfetlər pobeda', 'sotaş konfeti', 'yumşaq konfet',
        'şokoladlı konfetlər', 'şokolad konfet apelsin',
        'şokolad konfet squşenka kokos ilə',
        'g+g schoko dragee nişanlı konfet', 'konfet karamel',
        'karamel konfet', 'sladosti dona nişanli halvali konfet',
        'sladosti dona nişanli kozinakli konfet',
        'lop lop domi̇nka konfet', 'şokoladli konfetlər',
        'konfet candy nut nuqa və yumşaq karamel araxis',
        'konfet bonny fruit yaqodniy miks', 'nevski nişanlı konfet',
        'konfet (paşmaq)', 'qlazurlu konfet'                          
]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Konfet, karamel, şokolad" if x in konfet_karamel_sokolad else x)

### Corek pecenye pryanik

In [ ]:
see_similar("xəmir".lower())

(10, array(['dondurulmuş xəmir və qatli xəmir',
        'i̇stehlakahazir hərnöv xəmir xörəkləri', 'xəmir', 'şəkər xəmiri',
        'ət, tərəvəz və digər məhsullarladoldurulmuş makaron və pelmenformali xəmirməmulatlari',
        'xəmir mayasi', 'pinza xəmiri-kiçik', 'pinza xəmiri-böyük',
        'dekor xəmiri', 'şəkər xəmiri (ağ)'], dtype=object))

In [ ]:
pecenye_corek_bulka = [
                       'dondurulmuş xəmir və qatli xəmir',
        'i̇stehlakahazir hərnöv xəmir xörəkləri', 'xəmir', 'şəkər xəmiri',
        'ət, tərəvəz və digər məhsullarladoldurulmuş makaron və pelmenformali xəmirməmulatlari',
        'xəmir mayasi', 'pinza xəmiri-kiçik', 'pinza xəmiri-böyük',
        'dekor xəmiri', 'şəkər xəmiri (ağ)',
                       'makaron',
        'ət, tərəvəz və digər məhsullarladoldurulmuş makaron və pelmenformali xəmirməmulatlari',
        'makaron, əriştə və makaronməhsullari',
        'makaron məmulati-spagetti̇',
        'makaron məhsullari, a qrupp, əla növ spagetti̇',
        'makaron məhsullari, a qrupp, əla növ vermi̇şel',
        'noxuddan və qarğıdalı unundan makaron',
        'kartal dk nişanlı makaronlar', 'makaron üçün sous',
        'tez hazırlanan makaron məhsulları rolton', 'makaron memulatları',
        'makaron məmulatı', 'burğu makaron 500 qr',
        'makaron məhsullari "eş zdorovo" amarant unu qlutensiz vermisel 250qr',
        'barilla spaghetti no 5 nişanlı makaron', 'makaron məlumatlari',
        'ferişte nişanlı makaronlar',
'vafli', 'dondurma üçün vafli', 'vafli̇ trubuçka', 'vafli̇',
'dondurma üçün vafli̇', 'vafli̇ beliy maq',
'morojennoe 20 kopeek  plombi̇r s aromatom vani̇li̇ vafelnom stakanci̇ke - dondurma 20 kopeek vani̇lli̇ plombi̇r vafli̇ stəkanda',
'vaflili tort', 'dr schaer magdal kuechl aprik nişanlı vafli',
'dr schaer pausa ciok nişanlı vafli',
'nutella sokoladi ile doldurulmus vafli',
'vafli̇ trubuçka', 'vafli̇', 'dondurma üçün vafli̇',
'vafli̇ beliy maq',
'morojennoe 20 kopeek  plombi̇r s aromatom vani̇li̇ vafelnom stakanci̇ke - dondurma 20 kopeek vani̇lli̇ plombi̇r vafli̇ stəkanda',
'şirniyyat',
'qənnadi məmulati', 
'qənnadi məmulatlar', 'qənnadi məmulatlari ( oyuncaqlı yumurta)',
'qənnadi məmulatlari', 'qənnadi un məmulati',
'qənnadi məmulatı',

'unlu məmulat', 'unlu məmulatlari (mararon manti)',
'unlu məmulat ( vafli)', 'unlu qənnadı məmulat',
'unlu məmulatla vafli', 'unlu məmulatlar',
'unlu qənnadı məmulatları (pryanik)', 'unlu qara zeytun',
'qara qaragatli "belqiyskiy desert"  unlu məmulat 0.672 kq',
'unlu məmulatlari', 'unlu qənnadı məmulatları',
'unlu qənnadi məmulatlar', 'unlu məmulatlari (rulet)',
'korona vkusa nişanlı unlu məmulat',
'korona vkusa nişanlı qlazurlu unlu məmulat', 'unlu məmulatları',
'unlu qənnadı məmulatlar', 'unlu məmulatlar(tort)',
'unlu məmulatlari (kek)',

'suşka (südlü)',
'pryani̇k südlü',
'pryanik', 'pryaniklər', 'unlu qənnadı məmulatları (pryanik)',
'korona vkusa nişanlı qlazurlu pryanik',
'korona vkusa nişanlı pryanik', 'pryanik san bane apelsinli',
'şokoladlı maffinlər', 'maffinlər',
'biskvit','bi̇skvi̇t har rooz', 'bi̇skvi̇t',
'tort biskvit',
'sadə keks, sadə biskvit, sadə krekerlər və s., qlazurlu, dolğuluvə/və ya ədviyyatli biskvitlər, kekslər, krekerlər və vafli',
'biskvit məhsulları', 'biskvit məhsullari',
'minisnack pastry cream 125gr əmtəə nişanli biskvitlər',
'millefoglie puff pastry 125 gr əmtəə nişanli biskvitlər',
'vicenzovo gr.400əmtəə nişanlibiskvitlər',
'minisnack hazelnut cream 125gr əmtəə nişanli biskvitlər'

'bulka',   'keks', 'sadə keks, sadə biskvit, sadə krekerlər və s., qlazurlu, dolğuluvə/və ya ədviyyatli biskvitlər, kekslər, krekerlər və vafli',
'ulker kekstra ni̇şanli keks', 'cassy keks',

'çörək bulka məmulatları', 'çörək',
'çörək-bulka məmulatlari',
'burger çörəklər küncütlü 6 ədəd 300 qr',
'çörək-frozen round breadwi& reground durum ùeat foulls3g',

'sapore aroldo bag in box 10kg eu çörək ücün qatı maya',
'əla növ çörək unu', 'edeka bio dinkelbrot nişanlı çörək',
'çörək çubuqlar və kakaolu pasta seti', 'hazır çörəkçilik tozları','xirçildayan çörək',

'unlu məmulat ( pecenya))',
'pecenye obsyanoe klassicekoe ves 2.7 kq', 'unlu məmulat pecenya',
'unlu məmulatalr ( pecenya )', 'hei̇nz pecenye',
'peçenye', 'şirin quru peçenye', 'şokoladlı peçenye',
'peçenye  çiyəlkli steviya', 'şirin peçenyalar', 'peçenyelər',
'örtüklü şirin quru peçenye',
'coffret 1888 sables pur beurre 500gr ct*12-peçenye', 'peçenya',
'şokoladli peçenye', 'şəkərli̇ peçenye',
'vestelia çokoladlı peçenye 0.250 kq',
'vestelia ənənəvi peçenye 0.300 kq',
'uşaq peçenyesi koqda ya virastu 180 qr 5 taxil bitkisi kalsium ilə',
'uşaq peçenyesi koqda ya virastu 150 qr banan ilə',
'bahlsen leibniz minis nişanlı peçenye',
'g+g butterkeks nişanlı peçenye', 'peçenye stevi̇yali',
'rulet vafli  peçenye', 'sladosti dona nişanli qlazurlu peçenye',
'peçenye (fei̇ny bi̇skui̇t)', 'moskva-peçenye', "Unlu məmulatlar (çörək, peçenye, bulka, keks, biskvit)"                       

                       
]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Unlu məmulatlar (çörək, peçenye, bulka, keks, biskvit, makaron)" if x in pecenye_corek_bulka else x)

### Cay

In [ ]:
see_similar("çayl")

(3, array(['çay, bitki və meyvə çaylari və bunlarin qarişiqlari',
        'qara çaylar', 'ledi grey, earl grey  çayları'], dtype=object))

In [ ]:
cay = [
'qara cay', 
'bitki çayi', 'çay', 'qara çay', 'yaşıl çay', 'soyuq çay',
        'aloe blossom herbal tea / aloe çiçəklərindən çay, filtr-paket №25',
        'bai̇kal tea collecti̇on yabani otlardan hazirlanmiş fi̇toçay №9 (karbohi̇dratlara nəzarət)',
        'otaci pastil adaçayi, sorma pastil №24',
        'çay hazirlamaq üçün toz', 'çay (qara, xam)',
        'turboslim təmizləmə çayı,çay (filtr-paketdə) no 20',
        'qırmizı kök,kökümsov köklərlə birlikdə,çay(paketdə 30q) no 1',
        'baikal tea collection yabanı otlardan hazırlanmış fitoçay №9 (karbohidratlara nəzarət)',
        'bai̇kal tea collecti̇on yabani otlardan hazirlanmiş fi̇toçay №3 (təbi̇i̇ anti̇stress), fitoçay (filtr-paketdə) №30',
        'sabelnik çay/bataqlıq gördəvər çayı,çay,50 qr',
        'baikal tea collection yabani otlardan hazirlanmiş fitoçay n 7/yüngül hərəkət',
        'çay, bitki və meyvə çaylari və bunlarin qarişiqlari',
        'yabani otlardan hazirlanmiş fitoçay temizleme və drenaj, filtr-paket 1.5 qr №30',
        'digər yaşıl çay (fermentləşdirilməmiş)', 'çaytikanı meyvəsi',
        'yaşıl çay (gunpower sortu)', 'xam qara çay',
        'yaşıl çay (fermentləşdirilməmiş), 3 kq-dan çox çəkisi olmayan ilkin bağlamalarda',
        'çay əlavəsi',
        'baikal tea collection yabanı otlardan hazırlanmış fitoçay №5 (komfort həzm)',
        'yoogo təmi̇zləyi̇ci̇ turbo çay, filtr-paket №30',
        'baikal tea collection yabanı otlardan hazırlanmış fitoçay №6 (qaraciyər qorunması)',
        'baikal tea collection yabanı otlardan hazırlanmış fitoçay №8 (ürək rahatlığı)',
        'bai̇kal tea collecti̇on yabani otlardan hazirlanmiş fi̇toçay №4 (yüngül nəsəf), filtr-paket 1.5 qr №30',
        'paket çay üçün kağız', 'yasəmən çayı', 'soyuq çay ekstraktı',
        'ceylon ni̇şanli qara çay',
        'aloe blossom herbal tea/aloe çi̇çəkləri̇ndən çay, fi̇ltr-paket n25',
        'i̇ki̇nci̇ növ i̇ran çayi. lahi̇jan refah tea', 'qara çaylar',
        'meyvə-bitki çay', 'cran tea / mərcani çayı, №25  filtr-paketdə',
        'teekanne assam nişanlı çay', 'ledi grey, earl grey  çayları',
        'jasmine çayı 12*25*1.8gr',
        'baikal tea collection yabanı otlardan hazırlanmış fitoçay №5/komfort həzm, filtr-paket 1.5 qr №30',
        'baikal tea collection yabanı otlardan hazırlanmış fitoçay №4/yüngül nəfəs, filtr-paket 1.5 qr №30',
        'baikal tea collection yabanı otlardan hazırlanmış fitoçay №9/karbohidratlara nəzarət, filtr-paket 1.5 qr №30',
        'ersağ li̇si̇yum çayi (goji̇ berry) (goji̇ beere tee) 5 li̇ form çayi',
        'ersağ mori̇nga çayi', 'lipton nişanli soyuq çay', 'çay-latte',
        'uşaq çayi 85 qr koqda ya virastu  - gecəniz xeyrə (alma.melissa və s )',
        'koqda ya virastu uşaq çayi - moruq itburnu c vitamini ilə',
        'çaykur tiryaki çayı 1000 qr', 'ofçay tiryakisine qara çay',
        'çay içkisi karkade albalı ilə', 'qara çay ədvi̇yyatli',
        'yaşıl çay moruq ilə', 'çay yaşıl',
        'meyvəli,ədviyyatlı , bitki çay assorti',
        'alnatura bio gruener tee sencha nişanlı çay',
        'qara çay(giləmeyvə və meyvə garişiğı ilə)'       
]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Qida və müalicəvi çaylar" if x in cay else x)

### Qehve 

In [ ]:
see_similar("kaka")

(12, array(['kakao', 'kakao yağı', 'kakao tozu', 'kakao yaği əvəzedicisi',
        'kakaolu krem şanti', 'ultra chococ-kakao yağı əvəzedicisi',
        'nesquik nişanlı kakao', 'krem kakao içlikli kurassan 40 qr',
        'toz kakao 50 qr',
        'qənnadi məmulatlarinin istehsali üçün "solpro" temperləşdirməmış kakao yağ əvəzedici -33931',
        'kakao yağı əvəzedicisi', 'naturel kakao tozu'], dtype=object))

In [ ]:
qehve_kakao = [
'kakao', 'kakao yağı', 'kakao tozu', 'kakao yaği əvəzedicisi',
        'kakaolu krem şanti', 'ultra chococ-kakao yağı əvəzedicisi',
        'nesquik nişanlı kakao', 'krem kakao içlikli kurassan 40 qr',
        'toz kakao 50 qr',
        'qənnadi məmulatlarinin istehsali üçün "solpro" temperləşdirməmış kakao yağ əvəzedici -33931',
        'kakao yağı əvəzedicisi', 'naturel kakao tozu',
  'kofe qovrulmuş',
        'qovrulmuş kofe çəyirdəyi, qovrulmuş və üyüdülmüş kofe, kofe ekstrakti və içərisində dadlandirici kofebirləşməsi olanistehlaka hazir kofe',
        'kofe qovrulmamış kofe', 'kofein benzoat natrium 20%-li',
        'kofe qovrulmamış kofeinli', 'hazir kofe i̇çki̇si̇',
        'kofein benzoat 20%', 'kofeli noxud', 'kofeinofarm 20%',
        'kofein-benzoat natrium 20%-li',
        'qovrulmuş kofei̇nsi̇z espresso kofe', 'kofe',
        'ki̇mbo nişanlı kofe', 'kofe "aroma gold espresso" 1 kq',
        'wellness kokteyli̇ hazirlamaq ü quruçün qatişiq kofe kappuçi̇no dadı ilə',
        'kofe çibo',

'qəhvə',"qehve",
        '"vertera® forte" ("vertera® forte") qəhvəyi dəniz yosunlarından (laminariyadan, fukusdan) hazırlanan qida məhsulu.',
        'ti̇ryaki̇si̇ne qəhvə ersağ ti̇ryaki̇si̇ne', 'qəhvə (osmanlı)',
        'türk qəhvəsi̇',
        'qəhvəyi və yabanı düyü qarışığı "bravolli" 350 qr',
        'kurukahveci mehmed efendi türk qəhvəsi',
        'coffi̇to 3 - ü 1 - də , coffi̇to 4 - ü 1 - də ,  ala turk qəhvəsi poşet 100 qr',
        'hazır qəhvə', 'qovrulmuş qəhvə',
        'today qar topu qəhvəli 50 gr1x24x6', 'qəhvə (dibek)',
        'qəhvə (gold)'               
]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Qəhvə və kakao" if x in qehve_kakao else x)

### Duyu

In [ ]:
see_similar('düyü')

(20, array(['düyü', 'düyü unu', 'düyü lopası', 'düyü yarması',
        'çips qarğıdalı-düyülü naço pendiri ilə',
        'qənnadi məmulatlarinin istehsalı üçün partladılmış düyü (şişildilmiş) -1-2,5 mm',
        'çips qarğıdalı-düyülü tomat və reyhan ilə',
        '1121 ağ sella düyüsü', 'müxtəlif növ düyü unları', 'uzun düyü',
        '"sella basmatı" adlı düyü', 'düyü unu 250 qr', 'düyü kağizi',
        'müxtəlif növ yumru düyü', 'düyü yumru',
        'uzun qırmızı düyü "bravolli" 350 qr',
        'yumuru düyü “yarmarka” 1p/ 800 q',
        'plov üçün basmati uzun düyü "bravolli" 350 qr',
        'düyü "jasmin" "bravolli" 350 qr', 'ağ sella düyüsü'], dtype=object))

In [ ]:
duyu = [
'düyü', 'düyü unu', 'düyü lopası', 'düyü yarması',
        'qənnadi məmulatlarinin istehsalı üçün partladılmış düyü (şişildilmiş) -1-2,5 mm',
        '1121 ağ sella düyüsü', 'müxtəlif növ düyü unları', 'uzun düyü',
        '"sella basmatı" adlı düyü', 'düyü unu 250 qr', 'düyü kağizi',
        'müxtəlif növ yumru düyü', 'düyü yumru',
        'uzun qırmızı düyü "bravolli" 350 qr',
        'yumuru düyü “yarmarka” 1p/ 800 q',
        'plov üçün basmati uzun düyü "bravolli" 350 qr',
        'düyü "jasmin" "bravolli" 350 qr', 'ağ sella düyüsü',

]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Düyü və ondan hazırlanmış məhsullar'" if x in duyu else x)

### Qargidali

In [ ]:
see_similar('QARgIDALI'.lower())

(1, array(['xam qargidali yagi'], dtype=object))

In [ ]:
qargidali = [
             'qarğidali unu', 'dondurulmuş qarğidali dənələri',
        'qovrulmuş qarğidali', 'qarğidali', 'souslu qarğidali',
        'konservləşdirilmiş qarğidali', 'qarğidali dənələri̇',
        'qarğidali valideyni', 'şirin qarğidali dənələri',
        'ərzaqliq qarğidali', 'konservləşdi̇ri̇lmi̇ş qarğidali,hei̇nz',
        'şəkər qarğidalisi konservi', 'qarğidali yarmasi n 5',
        'konservləşdi̇ri̇lmi̇ş qarğidali konservi̇', 'qarğidali yaği',
        'qarğidali ni̇şastasi 200 qr', 'qarğidali unu 250 qr',
        'qarğidali ni̇şastasi', 'corn qarğidali yaği',
        'qarğidalidan hazirlanmiş hazir ərzaq', 'qarğidali yaği 1 lt',
        'qarğidalili cubuq (rafaello)', 'qarğidali çubuqlari 38 gr',
        
             'xam qargidali yagi',
'qarğıdalı konservi', 'qarğıdalı popkornu', 'qarğıdalı lopası',
'qarğıdalı çubuğu', 'qarğıdalı nişastası', 'qarğıdalı yağı',
'qarğıdalı özü, qarğıdalının istehsal qalıqları',
'qarğıdalı kəpəyi', 'qarğıdalı toxumu', 'qarğıdalı unu',
'qarğıdalı', 'qarğıdalı yarması',
'çips qarğıdalı-düyülü naço pendiri ilə',
'çips qarğıdalı-düyülü tomat və reyhan ilə',
'qarğıdalı lopaları start', 'çin qarğıdalısı',
'qarğıdalı yarmasi n 5', 'saflaşdırılmış qarğıdalı yağı',
'ərzaqlıq qarğıdalı', 'qarğıdalı çi̇psi̇ləri̇', 'souslu qarğıdalı',
'qarğıdalı çubuqları', 'qarğıdalı popcorn', 'şirin qarğıdalı',
'qovrulmuş qarğıdalı', 'qarğıdalı çipsi'
]

In [ ]:
df['mehsul'] = df['mehsul'].apply(lambda x: "Qarğıdalı və ondan hazırlanmış məhsullar" if x in qargidali else x)

### See top 

In [ ]:
df['mehsul'].value_counts().reset_index().iloc[20:40]

,index,mehsul
20,Arpa və arpa məhsulları,2637
21,Meşə materialı (taxta) və bəzək dekorativ ağac...,2221
22,Armud və armuddan hazırlanan məhsullar,2170
23,"Balıq, balıq məhsulları və balıqlara aid nümun...",2093
24,BFMQƏ,2073
25,Soğan və soğandan hazırlanan məhsullar,1786
26,Bal,1712
27,Qida və müalicəvi çaylar,1645
28,Gavalı və gavalıdan hazırlanan məhsullar,1635
29,"Üzüm, üzümdən hazırlanan məhsullar və üzüməçül...",1590


In [ ]:
df['mehsul'].value_counts().reset_index().head(40)

,index,mehsul
0,Qan,74465
1,"Pomidor, pomidor məhsulları və pomidorçuluğa a...",53176
2,Alma və alma məhsulları,32236
3,"Kapsul, flakon, damcı məhlul və tablet forması...",13580
4,Kartof və ondan hazırlanmış məhsullar,13436
5,Pambıq lifi və pambıqçılığa aid nümunələr,10082
6,"Buğda, buğda unu və digər məhsullar",10072
7,"Süd və süd məhsulları (kərə yağı, süd, pendir,...",9582
8,Xurma və ondan hazırlanan məhsullar,7843
9,"Mal, toyuq, ördək, hind quşu əti və onlardan h...",7581


In [ ]:
df['mehsul'].value_counts().reset_index().iloc[240:260]

,index,mehsul
240,çörək otu,35
241,bulyon,35
242,üyüdülmüş qara i̇stiot,35
243,böyürtkən,35
244,i̇ris,35
245,yulaf,35
246,"“pedigree” sousda toyuq, mal, dovşan və hinduş...",35
247,avokado,35
248,penbex,34
249,badımcan toxumu,34


In [ ]:
df['mehsul'].value_counts().reset_index().head(40)['mehsul'].sum(axis=0)

299720

In [ ]:
laboratoriyalar = df['Laboratoriya'].unique()

In [ ]:
laboratoriyalar

array(['Mərkəzi Baytarlıq Laboratoriyası',
       'MFL Meyvəli Karantin - ekspertiza bölməsi',
       'Milli Referens Laboratoriyası',
       'Abşeron Regional Sınaq Laboratoriyası',
       'Kompleks Tədqiqatlar Laboratoriyası',
       'Bakı Regional Sınaq Laboratoriyası',
       'Mərkəzi Fitosanitar Laboratoriyası',
       'Tovuz Regional Sınaq Laboratoriyası',
       'Lənkəran Regional Sınaq Laboratoriyası',
       'Xaçmaz Regional Sınaq Laboratoriyası',
       'Mərkəzi Baytarlıq Laboratoriyasının Bərdə Regional bölməsi',
       'Mərkəzi Baytarlıq Laboratoriyasının Qax Regional bölməsi',
       'Göygöl Regional Sınaq Laboratoriyası',
       'Mərkəzi Baytarlıq Laboratoriyasının Quba Regional bölməsi',
       'Mərkəzi Baytarlıq Laboratoriyasının Sabirabad Regional bölməsi',
       'Meyvəli Karantin-ekspertiza bölməsi',
       'Qax Regionl Sınaq Laboratoriyası',
       'Bərdə Regional Sınaq Laboratoriyası',
       'Sabirabad Regional Sınaq Laboratoriyası',
       'Quba Reqional Sınaq La

In [ ]:
n=-9
print(laboratoriyalar[n])
df[df['Laboratoriya']==laboratoriyalar[n]]['mehsul'].value_counts().tail(20).reset_index()

Mərkəzi Baytarlıq Laboratoriyasının Quba Regional bölməsi


,index,mehsul
0,Qan,29
1,damazliq yumurta,17
2,soya şrotu,12
3,"Buğda, buğda unu və digər məhsullar",10
4,Patoloji material,1


In [ ]:
n=0
print(laboratoriyalar[n])
df[df['Laboratoriya']==laboratoriyalar[n]]['mehsul'].value_counts().head(30).reset_index()

Mərkəzi Baytarlıq Laboratoriyası


,index,mehsul
0,Qan,37268
1,nan,4398
2,Yaxma,3780
3,Patoloji material,1041
4,"Mal, toyuq, ördək, hind quşu əti və onlardan h...",1013
5,"Süd və süd məhsulları (kərə yağı, süd, pendir,...",54
6,Bal,54
7,"Buğda, buğda unu və digər məhsullar",38
8,hind quşu qiyməsi,34
9,toyuq qanadı,27


In [ ]:
# orqan, patoloji material, toyuq eti, mal eti, qus, baliq, bal, dondurulmus, 

In [ ]:
df[
   (df['Laboratoriya']=='Mərkəzi Baytarlıq Laboratoriyası') &
   (df['mehsul'] == 'nan')
].iloc[:, 9].value_counts()

Müxtəlif növ heyvan xəstəlikləri (İBH və XBH) Leykoz, Şmallenberq,Xlamidioz , Dabaq, Bruselyoz, XBH kataral isitməsi,Nodulyar dermatit -ElİSA metod                                          181
Koronovirus müayinəsi                                                                                                                                                                         88
XBH taun xəstəliyi- ELİSA metod                                                                                                                                                               87
İri və xırdabuynuzlu heyvanların bruselyoz xəstəliyi-  ELİSA metod                                                                                                                            71
Bruselyoz xəstəliyi  (İBH, XBH,at və s. heyvanlar) -klassik seroloji metod-Roz-Benqal sınağı,KBR,AR                                                                                           20
Müxtəlif növ heyvanların ümumi xəst

In [ ]:
df[
   (df['Laboratoriya']=='Mərkəzi Baytarlıq Laboratoriyası') 
   #&
   #(df['mehsul'] == 'nan')
].iloc[:, 9].value_counts()

Bruselyoz xəstəliyi  (İBH, XBH,at və s. heyvanlar) -klassik seroloji metod-Roz-Benqal sınağı,KBR,AR                                                                                          4678
XBH taun xəstəliyi- ELİSA metod                                                                                                                                                              3563
immunoferment analizi (ELİSA) vasitəsilə müxtəlif növ heyvan xəstəliklərinin törədicilərinin (diaqnostik nümunələrdə, spermada, embrionda və s.) təyini (1nümunə)                            1918
Quş xəstəlikləri Quş qripi,Nyukasl, İnfeksion bronxit,İnf.larinqatraxet, İnf.bursal , Reovirus, Mikoplazmoz və s. - ElİSA metod                                                              1337
Müxtəlif növ heyvan xəstəlikləri (İBH və XBH) Leykoz, Şmallenberq,Xlamidioz , Dabaq, Bruselyoz, XBH kataral isitməsi,Nodulyar dermatit -ElİSA metod                                           888
molekulyar-genetik üsulla (Pol

In [ ]:
df[df['mehsul']== "nan" ].iloc[:, 10].value_counts()

Heyvan xəstəliklərinin diaqnostikası                                                                                                                   291
Yoluxucu xəstəliklərin siyahısına daxil edilmiş xüsusi təhlükəli heyvan xəstəlikləri istisna olmaqla, digər xəstəliklərin diaqnostikası (müayinəsi)    109
Digər                                                                                                                                                   88
Qida mikrobiologiyası                                                                                                                                   51
Fiziki və hüquqi şəxslərin müraciətləri əsasında fitosanitar müayinəsinin və ekspertizasının aparılması                                                 41
Aqrokimyəvi maddələrin dövlət sınağı, təkrar qeydiyyatı və ekspertizası                                                                                 38
Baytarlıq preparatlarının, yem və yem əlavələrinin istifadə üçün təqdi

In [ ]:
df[df['mehsul']=='İbh dərisi'.lower()]#['Laboratoriya'].value_counts()

,Akt nömrəsi,Subyektin ID kodu,Subyekt,İstinad sənəd nömrəsi,Nümunənin təyinatı,Ölkə,Məhsul,Laboratoriya,Region,Analizin adı,Analiz qrupu,Analizin tipi,Analiz parametri,Tətbiq edilən sınaq metodu,Texniki normativ hüquqi sənəd,Analiz nəticəsi,NGA tarixi,NQA tarixi,Analizin başlama tarixi,Analizin bitmə tarixi,Laboratoriya göndərilmə tarixi,Regional bölməyə göndərilmə tarixi,nga_tarixi,ay_sirasi,groups_advanced_analiz_tipi,analiz_neticesi,groups_advanced_analiz_tipi_nas,final_groups,numunenin_teyinati,numunenin_novu,mehsul


### Products

In [ ]:
df = df[~(df['analiz_neticesi'] == "Qeyd olunmayıb")]

In [ ]:
df['mehsul'].value_counts().reset_index().head(39)

,index,mehsul
0,"Pomidor, pomidor məhsulları və pomidorçuluğa a...",52485
1,Qan,39687
2,Alma və alma məhsulları,32169
3,Kartof və ondan hazırlanmış məhsullar,13346
4,"Kapsul, flakon, damcı məhlul və tablet forması...",13123
5,"Buğda, buğda unu və digər məhsullar",9998
6,Pambıq lifi və pambıqçılığa aid nümunələr,9996
7,"Süd və süd məhsulları (kərə yağı, süd, pendir,...",9350
8,Xurma və ondan hazırlanan məhsullar,7615
9,"Mal, toyuq, ördək, hind quşu əti və onlardan h...",6854


In [ ]:
selectde_prodcts = df['mehsul'].value_counts().reset_index().head(39)['index']

In [ ]:
df_selected_products = df[df['mehsul'].isin(selectde_prodcts)]

In [ ]:
product_stats = pd.crosstab(df_selected_products['mehsul'], df_selected_products['analiz_neticesi']).reset_index()

In [ ]:
product_stats['mehsul'] = pd.Categorical(product_stats['mehsul'], selectde_prodcts, True)

In [ ]:
product_stats = product_stats.sort_values(by='mehsul').reset_index(drop=True)

In [ ]:
product_stats['Cəmi analiz sayı'] = product_stats.iloc[:, 1:10].sum(axis=1)

In [ ]:
product_stats['Uyğunluq faizi'] = round(product_stats['Uyğundur']/product_stats['Cəmi analiz sayı']*100, 1)

In [ ]:
muraciet_sayi = []

for i in selectde_prodcts:
  muraciet_sayi.append(len(df[df['mehsul'] == i]["Akt nömrəsi"].unique()))

In [ ]:
max_analiz_sayi = []

for i in selectde_prodcts:
  max_analiz_sayi.append(df[df['mehsul'] == i]["Akt nömrəsi"].value_counts()[0])

In [ ]:
product_stats['Müraciət sayı'] = muraciet_sayi
product_stats['Bir müraciət üzrə aparılmış ən çox analiz sayı'] = max_analiz_sayi

In [ ]:
product_stats['Bir müraciətə düşən ortalama analiz sayı'] = round(
    product_stats["Cəmi analiz sayı"]/product_stats["Müraciət sayı"]
)

In [ ]:
# temp_sum = df[df['mehsul'] == selectde_prodcts[0]]['numunenin_teyinati'].value_counts().reset_index()['numunenin_teyinati'].sum()
# temp_say = df[df['mehsul'] == selectde_prodcts[0]]['numunenin_teyinati'].value_counts().reset_index()['numunenin_teyinati'][0]
# temp_numune_teyinati = df[df['mehsul'] == selectde_prodcts[0]]['numunenin_teyinati'].value_counts().reset_index()['index'][0]
# temp_faiz = temp_say/temp_sum*100
# temp_result = f'{temp_numune_teyinati} üzrə {temp_faiz}%'

In [ ]:
numunenin_teyinati_faizi = []

for i in selectde_prodcts:
  temp_sum = df[df['mehsul'] == i ]['numunenin_teyinati'].value_counts().reset_index()['numunenin_teyinati'].sum()
  temp_say = df[df['mehsul'] == i ]['numunenin_teyinati'].value_counts().reset_index()['numunenin_teyinati'][0]
  temp_numune_teyinati = df[df['mehsul'] == i ]['numunenin_teyinati'].value_counts().reset_index()['index'][0]
  temp_faiz = round(temp_say/temp_sum*100)
  temp_say_2 = df[df['mehsul'] == i ]['numunenin_teyinati'].value_counts().reset_index()['numunenin_teyinati'][1]
  temp_numune_teyinati_2 = df[df['mehsul'] == i ]['numunenin_teyinati'].value_counts().reset_index()['index'][1]
  temp_faiz_2 = round(temp_say_2/temp_sum*100)
  temp_result = f'{temp_numune_teyinati} üzrə {temp_faiz}%\n{temp_numune_teyinati_2} üzrə {temp_faiz_2}%'
  numunenin_teyinati_faizi.append(temp_result)

In [ ]:
product_stats["Analizlər üzrə aparılan ən çox məhsulun təyinatı"] = numunenin_teyinati_faizi

In [ ]:
df['Analiz qrupu'].value_counts()

Fiziki və hüquqi şəxslərin müraciətləri əsasında fitosanitar müayinəsinin və ekspertizasının aparılması                                                149211
Qida mikrobiologiyası                                                                                                                                   45509
Heyvan xəstəliklərinin diaqnostikası                                                                                                                    31314
Yoluxucu xəstəliklərin siyahısına daxil edilmiş xüsusi təhlükəli heyvan xəstəlikləri istisna olmaqla, digər xəstəliklərin diaqnostikası (müayinəsi)     17185
Bitki və bitkiçilik məhsullarında toksikoloji analizlər                                                                                                 14925
Qida məhsullarının toksikologiyası                                                                                                                      13996
Qida məhsullarının fiziki-kimyəvi göstəriciləri     

In [ ]:
analiz_qrupu = []

for i in selectde_prodcts:
  temp_sum = df[df['mehsul'] == i ]['Analiz qrupu'].value_counts().reset_index()['Analiz qrupu'].sum()
  temp_say = df[df['mehsul'] == i ]['Analiz qrupu'].value_counts().reset_index()['Analiz qrupu'][0]
  temp_numune_teyinati = df[df['mehsul'] == i ]['Analiz qrupu'].value_counts().reset_index()['index'][0]
  temp_faiz = round(temp_say/temp_sum*100)
  temp_result = f'{temp_numune_teyinati} üzrə {temp_faiz}%'
  analiz_qrupu.append(temp_result)

In [ ]:
product_stats['Analiz qrupu'] = analiz_qrupu

In [ ]:
product_stats.to_excel("Products.xlsx")

In [ ]:
product_stats[['mehsul',"Müraciət sayı", "Cəmi analiz sayı", 
               'Bir müraciət üzrə aparılmış ən çox analiz sayı',
               # 'Bir müraciətə düşən ortalama analiz sayı',
               "Uyğunluq faizi",
               "Analizlər üzrə aparılan ən çox məhsulun təyinatı",
               'Analiz qrupu']].to_excel("products.xlsx")

In [ ]:
df['Analiz qrupu'].value_counts()

Fiziki və hüquqi şəxslərin müraciətləri əsasında fitosanitar müayinəsinin və ekspertizasının aparılması                                                149211
Qida mikrobiologiyası                                                                                                                                   45509
Heyvan xəstəliklərinin diaqnostikası                                                                                                                    31314
Yoluxucu xəstəliklərin siyahısına daxil edilmiş xüsusi təhlükəli heyvan xəstəlikləri istisna olmaqla, digər xəstəliklərin diaqnostikası (müayinəsi)     17185
Bitki və bitkiçilik məhsullarında toksikoloji analizlər                                                                                                 14925
Qida məhsullarının toksikologiyası                                                                                                                      13996
Qida məhsullarının fiziki-kimyəvi göstəriciləri     

### Idxal

In [ ]:
df['Ölkə'] = df['Ölkə'].apply(lambda x: "Türkiyə" if x=="Turkiyə" else x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_idxal = df[df['numunenin_teyinati'] == "İdxal"]

In [ ]:
necessary_countries = [
 'Rusiya Federasiyası',
 'Türkiyə',
 'İran İslam Respublikası',
 'Ukrayna',
 'Belarus Respublikası',]

In [ ]:
df_idxal = df_idxal[df['Ölkə'].isin(necessary_countries)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
countries_stats = df_idxal['Ölkə'].value_counts().reset_index()

In [ ]:
muraciet_sayi = []

for i in necessary_countries:
  muraciet_sayi.append(len(df_idxal[df_idxal['Ölkə'] == i]['Akt nömrəsi'].value_counts()))


In [ ]:
muracietde_en_cox_analiz_sayi = []

for i in necessary_countries:
  muracietde_en_cox_analiz_sayi.append(df_idxal[df_idxal['Ölkə'] == i]['Akt nömrəsi'].value_counts()[0])

countries_stats['Müraciət üzrə ən çox analiz sayı'] = muracietde_en_cox_analiz_sayi

In [ ]:
countries_stats['Müraciət sayı'] = muraciet_sayi

In [ ]:
mehsul_faizi = []

for i in necessary_countries:
  cemi_analiz = df_idxal[df_idxal['Ölkə'] == i]['mehsul'].value_counts().sum()

  mehsulun_adi = df_idxal[df_idxal['Ölkə'] == i]['mehsul'].value_counts().index[0]
  analiz_sayi = df_idxal[df_idxal['Ölkə'] == i]['mehsul'].value_counts()[0]  
  faizi = round(analiz_sayi/cemi_analiz*100, 1)

  mehsulun_adi_1 = df_idxal[df_idxal['Ölkə'] == i]['mehsul'].value_counts().index[1]
  analiz_sayi_1 = df_idxal[df_idxal['Ölkə'] == i]['mehsul'].value_counts()[1]  
  faiz_1 = round(analiz_sayi_1/cemi_analiz*100, 1)

  mehsulun_adi_2 = df_idxal[df_idxal['Ölkə'] == i]['mehsul'].value_counts().index[2]
  analiz_sayi_2 = df_idxal[df_idxal['Ölkə'] == i]['mehsul'].value_counts()[2]  
  faiz_2 = round(analiz_sayi_2/cemi_analiz*100, 1)

  result = f'{mehsulun_adi} üzrə - {faizi}%\n{mehsulun_adi_1} üzrə - {faiz_1}%\n{mehsulun_adi_2} üzrə - {faiz_2}%'
  mehsul_faizi.append(result)

In [ ]:
df_idxal[df_idxal['Ölkə'] == necessary_countries[0]]['mehsul'].value_counts().head(7)#.index[2]

Buğda, buğda unu və digər məhsullar                                                                                 8036
Konfet, karamel, şokolad                                                                                            1395
Kapsul, flakon, damcı məhlul və tablet formasında olan dərmanlar                                                    1345
Süd və süd məhsulları (kərə yağı, süd, pendir, qatıq, qaymaq, şor, kefir, sırok, kəsmik, yoqurt, ayran, pudinq)     1283
Mal, toyuq, ördək, hind quşu əti və onlardan hazırlanan məhsullar (kolbasa, sosislər, yarımfabrikat, konservlər)    1233
Unlu məmulatlar (çörək, peçenye, bulka, keks, biskvit, makaron)                                                     1021
Balıq, balıq məhsulları və balıqlara aid nümunələr                                                                   313
Name: mehsul, dtype: int64

In [ ]:
countries_stats["Məhsul üzrə analiz payı"] = mehsul_faizi

In [ ]:
uygunluq = []

for i in necessary_countries:

  cemi_analiz = df_idxal[df_idxal['Ölkə'] == i]['analiz_neticesi'].value_counts().sum()
  analiz_sayi = df_idxal[df_idxal['Ölkə'] == i]['analiz_neticesi'].value_counts()[0]  
  faizi = round(analiz_sayi/cemi_analiz*100, 1)

  uygunluq.append(f'{faizi}%')

In [ ]:
countries_stats['Uyğunluq'] = uygunluq

In [ ]:
df_idxal[df['Ölkə'] == 'Rusiya Federasiyası']['mehsul'].value_counts().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


21170

In [ ]:
df_idxal[df_idxal['Ölkə'] == 'Rusiya Federasiyası']['mehsul'].value_counts().head(7)

Buğda, buğda unu və digər məhsullar                                                                                 8036
Konfet, karamel, şokolad                                                                                            1395
Kapsul, flakon, damcı məhlul və tablet formasında olan dərmanlar                                                    1345
Süd və süd məhsulları (kərə yağı, süd, pendir, qatıq, qaymaq, şor, kefir, sırok, kəsmik, yoqurt, ayran, pudinq)     1283
Mal, toyuq, ördək, hind quşu əti və onlardan hazırlanan məhsullar (kolbasa, sosislər, yarımfabrikat, konservlər)    1233
Unlu məmulatlar (çörək, peçenye, bulka, keks, biskvit, makaron)                                                     1021
Balıq, balıq məhsulları və balıqlara aid nümunələr                                                                   313
Name: mehsul, dtype: int64

In [ ]:
countries_stats[["index","Ölkə","Müraciət sayı","Müraciət üzrə ən çox analiz sayı",
                 "Uyğunluq","Məhsul üzrə analiz payı",]] .to_excel("idxal.xlsx")

In [ ]:
countries_stats

,index,Ölkə,Müraciət üzrə ən çox analiz sayı,Müraciət sayı,Məhsul üzrə analiz payı,Uyğunluq
0,Rusiya Federasiyası,21170,120,4063,"Buğda, buğda unu və digər məhsullar üzrə - 38....",94.8%
1,Türkiyə,15432,41,3216,"Kapsul, flakon, damcı məhlul və tablet forması...",98.4%
2,İran İslam Respublikası,12007,30,4729,Kartof və ondan hazırlanmış məhsullar üzrə - 5...,97.6%
3,Ukrayna,3735,25,1211,"Mal, toyuq, ördək, hind quşu əti və onlardan h...",81.2%
4,Belarus Respublikası,2250,30,482,"Süd və süd məhsulları (kərə yağı, süd, pendir,...",90.1%


### Ixrac

In [ ]:
df_ixrac = df[df['numunenin_teyinati'] == "İxrac"]

In [ ]:
list(df_ixrac['Ölkə'].value_counts().reset_index().head(5)["index"])

['Rusiya Federasiyası',
 'Türkiyə',
 'Ukrayna',
 'İraq Respublikası',
 'Gürcüstan Respublikası']

In [ ]:
ixrac_states = ['Rusiya Federasiyası',
 'Türkiyə',
 'Ukrayna',
 'İraq Respublikası',
 'Gürcüstan Respublikası']
 
df_ixrac = df_ixrac[df_ixrac['Ölkə'].isin(ixrac_states)]

In [ ]:
ixrac_stats = df_ixrac['Ölkə'].value_counts().reset_index()

In [ ]:
ixrac_stats

,index,Ölkə
0,Rusiya Federasiyası,92984
1,Türkiyə,4985
2,Ukrayna,3564
3,İraq Respublikası,3353
4,Gürcüstan Respublikası,2347


In [ ]:
muraciet_sayi = []

for i in ixrac_states:
  muraciet_sayi.append(len(df_ixrac[df_ixrac['Ölkə'] == i]['Akt nömrəsi'].value_counts()))

ixrac_stats['Müraciət sayı'] = muraciet_sayi

mehsul_faizi = []

for i in ixrac_states:
  cemi_analiz = df_ixrac[df_ixrac['Ölkə'] == i]['mehsul'].value_counts().sum()

  mehsulun_adi = df_ixrac[df_ixrac['Ölkə'] == i]['mehsul'].value_counts().index[0]
  analiz_sayi = df_ixrac[df_ixrac['Ölkə'] == i]['mehsul'].value_counts()[0]  
  faizi = round(analiz_sayi/cemi_analiz*100, 1)

  mehsulun_adi_1 = df_ixrac[df_ixrac['Ölkə'] == i]['mehsul'].value_counts().index[1]
  analiz_sayi_1 = df_ixrac[df_ixrac['Ölkə'] == i]['mehsul'].value_counts()[1]  
  faiz_1 = round(analiz_sayi_1/cemi_analiz*100, 1)

  mehsulun_adi_2 = df_ixrac[df_ixrac['Ölkə'] == i]['mehsul'].value_counts().index[2]
  analiz_sayi_2 = df_ixrac[df_ixrac['Ölkə'] == i]['mehsul'].value_counts()[2]  
  faiz_2 = round(analiz_sayi_2/cemi_analiz*100, 1)

  result = f'{mehsulun_adi} üzrə - {faizi}%\n{mehsulun_adi_1} üzrə - {faiz_1}%\n{mehsulun_adi_2} üzrə - {faiz_2}%'
  mehsul_faizi.append(result)

ixrac_stats["Məhsul üzrə analiz payı"] = mehsul_faizi

uygunluq = []

for i in ixrac_states:

  cemi_analiz = df_ixrac[df_ixrac['Ölkə'] == i]['analiz_neticesi'].value_counts().sum()
  analiz_sayi = df_ixrac[df_ixrac['Ölkə'] == i]['analiz_neticesi'].value_counts()[0]  
  faizi = round(analiz_sayi/cemi_analiz*100, 1)

  uygunluq.append(f'{faizi}%')

ixrac_stats["Uyğunluq"] = uygunluq

muracietde_en_cox_analiz_sayi = []

for i in ixrac_states:
  muracietde_en_cox_analiz_sayi.append(df_ixrac[df_ixrac['Ölkə'] == i]['Akt nömrəsi'].value_counts()[0])

ixrac_stats['Müraciət üzrə ən çox analiz sayı'] = muracietde_en_cox_analiz_sayi

list(ixrac_stats['Məhsul üzrə analiz payı'])

['Pomidor, pomidor məhsulları və pomidorçuluğa aid nümunələr üzrə - 39.2%\nAlma və alma məhsulları üzrə - 32.6%\nNektarin üzrə - 3.0%',
 'Pambıq lifi və pambıqçılığa aid nümunələr üzrə - 94.6%\nBuğda, buğda unu və digər məhsullar üzrə - 1.9%\nQarğıdalı və ondan hazırlanmış məhsullar üzrə - 1.5%',
 'Xurma və ondan hazırlanan məhsullar üzrə - 33.6%\nNar və nardan hazırlanan məhsullar üzrə - 17.3%\nQarpız və toxumları üzrə - 11.8%',
 'Arpa və arpa məhsulları üzrə - 61.9%\nQarğıdalı və ondan hazırlanmış məhsullar üzrə - 12.0%\nKapsul, flakon, damcı məhlul və tablet formasında olan dərmanlar üzrə - 10.7%',
 'Soğan və soğandan hazırlanan məhsullar üzrə - 33.3%\nBuğda, buğda unu və digər məhsullar üzrə - 26.1%\nPomidor, pomidor məhsulları və pomidorçuluğa aid nümunələr üzrə - 8.6%']

In [ ]:
ixrac_stats[["index","Ölkə","Müraciət sayı","Müraciət üzrə ən çox analiz sayı",
             "Uyğunluq","Məhsul üzrə analiz payı",]].to_excel("ixrac.xlsx")

In [ ]:
df_ixrac[df_ixrac['Ölkə'] == necessary_countries[0]]['mehsul'].value_counts().head(7)

Pomidor, pomidor məhsulları və pomidorçuluğa aid nümunələr    36463
Alma və alma məhsulları                                       30341
Nektarin                                                       2835
Şaftalı və şaftalıdan hazırlanan məhsullar                     2547
Gilas və gilasdan hazırlan məhsullar                           2545
Xurma və ondan hazırlanan məhsullar                            2481
Armud və armuddan hazırlanan məhsullar                         1904
Name: mehsul, dtype: int64